In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 Unified Manus System - Combined Core, Tools, Server, and GUI (v7.0.x)                               ║
# ║ Combines elements from Box 2 (Agent Core & Tools), Box 3 (Server & GUI), and Agent/Role/UI snippets.   ║
# ║ Assumes Box 1 config is handled externally or defaults are used.                                       ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

# --- STEP 1: SETUP AND IMPORTS ---
print("🔧 Unified Manus System: Initializing...")
import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import re
import urllib.parse
import urllib.request
import inspect
import shlex
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable

# --- STEP 2: LOAD CONFIGURATION (Simulated/Fallback) ---
# In a real scenario, this would load from Box 1's exports or a config file.
print("📥 Loading configuration...")
try:
    # Try common paths (adjust as needed)
    config_paths = [
        Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json"),
        Path("./UnifiedManusSystem/config/box1_exports.json")
    ]
    box1_config = None
    for config_file in config_paths:
        if config_file.exists():
            with open(config_file, "r") as f:
                box1_config = json.load(f)
            break

    if box1_config:
        BASE_DIR = Path(box1_config["BASE_DIR"])
        WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
        LOG_FILE = Path(box1_config["LOG_FILE"])
        public_url = box1_config["public_url"]
        dashboard_url = box1_config["dashboard_url"]
        IS_COLAB = box1_config["IS_COLAB"]
        print("✅ Box 1 configuration loaded successfully")
        print(f"📁 Base Directory: {BASE_DIR}")
        print(f"🌍 Public URL: {public_url}")
    else:
        raise FileNotFoundError("Box 1 config not found")
except Exception as e:
    print(f"⚠️ Failed to load Box 1 config: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True # Adjust based on environment

# Ensure directories exist
BASE_DIR.mkdir(parents=True, exist_ok=True)
WORKSPACE_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)

# Change to base directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

# Apply nest_asyncio for Jupyter compatibility
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")

# --- STEP 3: IMPORT MODULES FOR FASTAPI AND GUI ---
print("📦 Importing required modules for API and GUI...")
# Core FastAPI imports
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from pydantic import BaseModel
import requests # For proxying calls if needed

# GUI imports (attempt to import, handle if not available)
JUPYTER_AVAILABLE = False
GRADIO_AVAILABLE = False
try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️ Jupyter widgets not available")

try:
    import gradio as gr
    GRADIO_AVAILABLE = True
    print("✅ Gradio available")
except ImportError:
    print("⚠️ Gradio not available")


# --- STEP 4: LOGGING SYSTEM ---
print("📝 Setting up logging...")
def log_activity(activity_type: str, message: str, details: Optional[Dict[str, Any]] = None):
    """Log an activity to the system log file."""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "type": activity_type,
        "message": message,
        "details": details or {}
    }
    try:
        with open(LOG_FILE, "a") as f:
            f.write(json.dumps(log_entry) + "\n")
    except Exception as e:
        print(f"⚠️ Failed to log activity: {e}")

# --- STEP 5: AGENT ROLE SYSTEM (from UI snippets) ---
print("🎭 Setting up Agent Role system...")
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        """Simulate role processing. In a full implementation, this would call an LLM."""
        # Placeholder logic - replace with actual LLM call
        response = f"[{self.name}] Processing task: {task_description}"
        if stream_callback:
             stream_callback(response + "\n")
        time.sleep(0.5) # Simulate processing time
        return response

# Define roles (from UI snippets)
ROLES = {
    "planner": ManusRole(
        "Planner",
        "Creates plans",
        "You are an expert planner. Break down complex tasks into smaller, manageable steps."
    ),
    "researcher": ManusRole(
        "Researcher",
        "Gathers information",
        "You are a research assistant. When asked to research or find information, use the provided tools effectively. Summarize findings clearly and concisely. Only provide the summary, not the tool call itself."
    ),
    "coder": ManusRole(
        "Coder",
        "Writes code",
        "You are a skilled software engineer. Write clean, efficient, and well-documented code based on the plan provided."
    ),
    "reviewer": ManusRole(
        "Reviewer",
        "Reviews code",
        "You are a senior engineer reviewing code. Check for correctness, efficiency, and best practices."
    ),
    "debugger": ManusRole(
        "Debugger",
        "Fixes code",
        "You are an expert debugger. When given code and error messages, identify the root cause and provide fixed code. Explain your reasoning clearly."
    ),
}

# --- STEP 6: FILE SYSTEM MANAGER (from UI snippets) ---
print("📁 Setting up File System Manager...")
class FileSystemManager:
    def __init__(self, base_path: Path = WORKSPACE_DIR):
        self.base_path = base_path.resolve()
        self.base_path.mkdir(parents=True, exist_ok=True)

    def safe_join(self, *paths) -> Path:
        """Join paths and ensure the result is within the base path."""
        full_path = Path(self.base_path, *paths).resolve()
        try:
            full_path.relative_to(self.base_path) # Raises ValueError if not relative
            return full_path
        except ValueError:
            raise PermissionError(f"Path traversal attempt: {full_path}")

    def read_file(self, file_path: str) -> str:
        """Read a file safely."""
        full_path = self.safe_join(file_path)
        with open(full_path, 'r', encoding='utf-8') as f:
            return f.read()

    def write_file(self, file_path: str, content: str) -> str:
        """Write content to a file safely."""
        full_path = self.safe_join(file_path)
        full_path.parent.mkdir(parents=True, exist_ok=True) # Ensure parent dirs exist
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ File written to {full_path}"

    def list_files(self, dir_path: str = ".") -> List[str]:
        """List files in a directory safely."""
        full_path = self.safe_join(dir_path)
        if full_path.is_dir():
            return [str(p.relative_to(self.base_path)) for p in full_path.iterdir() if p.is_file()]
        else:
            return [str(full_path.relative_to(self.base_path))] if full_path.is_file() else []

# --- STEP 7: CORE AGENT CLASS ---
print("🤖 Setting up Core Manus Agent...")
class ManusAgent:
    def __init__(self):
        self.roles = ROLES # Use the roles defined above
        self.fs = FileSystemManager() # Use the FS manager
        self.memory: List[Dict[str, Any]] = []
        self.session_id = f"session_{int(time.time())}"
        log_activity("system", "Manus Agent initialized")

    def solve_task(self, task_description: str, stream_callback=None):
        """Main task solving logic using roles."""
        log_activity("agent", "Task started", {"task": task_description})
        self.memory.append({"type": "task_start", "content": task_description, "timestamp": datetime.now().isoformat()})
        if stream_callback:
            stream_callback(f"🧠 Starting task: {task_description}\n")

        # Example multi-step process (replace with your logic)
        plan_output = self.roles["planner"].process(f"Create a plan for: {task_description}", stream_callback)
        self.memory.append({"type": "thought", "role": "planner", "content": plan_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"📝 Plan: {plan_output}\n")

        # Simulate execution steps (e.g., coding, review)
        code_output = self.roles["coder"].process(f"Write code based on plan: {plan_output}", stream_callback)
        self.memory.append({"type": "action", "role": "coder", "content": code_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"💻 Code: {code_output}\n")

        review_output = self.roles["reviewer"].process(f"Review code: {code_output}", stream_callback)
        self.memory.append({"type": "thought", "role": "reviewer", "content": review_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"🔍 Review: {review_output}\n")

        final_result = f"Task '{task_description}' completed.\nPlan: {plan_output}\nCode: {code_output}\nReview: {review_output}"
        self.memory.append({"type": "task_end", "content": final_result, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"✅ Task completed: {final_result}\n")
        log_activity("agent", "Task completed", {"task": task_description})
        return final_result

    def run_task(self, goal: str, context: Optional[str] = None) -> str:
        """Wrapper for solve_task, potentially adding context."""
        task_with_context = f"{goal}\nContext: {context}" if context else goal
        # For streaming, we'd need a different approach in a web context,
        # but for direct tool call, we can capture output.
        captured_output = []
        def capture_stream(text):
            captured_output.append(text)
        result = self.solve_task(task_with_context, stream_callback=capture_stream)
        # Return combined captured stream and result if needed, or just result
        # For simplicity, returning the final result string
        return result # Or '\n'.join(captured_output) + '\n' + result

# Initialize the global agent
agent = ManusAgent()
print("✅ ManusAgent system ready")


# --- STEP 8: TOOL REGISTRY AND EXECUTION SYSTEM ---
print("🛠️ Setting up tool registry and safety functions...")
# Tool registry
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str):
    """Decorator to register tools"""
    def decorator(func: Callable):
        TOOL_REGISTRY[name] = func
        print(f"🔧 Registered tool: {name}")
        return func
    return decorator

def safe_path(file_path: str) -> Path:
    """Ensure file operations stay within safe directory"""
    if not file_path:
        raise ValueError("File path cannot be empty")
    base_path = WORKSPACE_DIR.resolve()
    full_path = (base_path / file_path).resolve()
    try:
        full_path.relative_to(base_path)
        return full_path
    except ValueError:
        raise PermissionError(f"Access denied: {file_path} is outside the workspace")

# --- STEP 9: REGISTERING TOOLS ---
print("🧰 Registering tools...")

@register_tool("write_file")
def write_file(file_path: str, content: str) -> str:
    """Write content to a file"""
    try:
        safe_file_path = safe_path(file_path)
        safe_file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(safe_file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        log_activity("tool", "File written", {"file": file_path})
        return f"✅ Successfully wrote to {safe_file_path}"
    except Exception as e:
        error_msg = f"❌ Failed to write file {file_path}: {e}"
        log_activity("tool", "File write failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("read_file")
def read_file(file_path: str) -> str:
    """Read content from a file"""
    try:
        safe_file_path = safe_path(file_path)
        with open(safe_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        log_activity("tool", "File read", {"file": file_path})
        return content
    except Exception as e:
        error_msg = f"❌ Failed to read file {file_path}: {e}"
        log_activity("tool", "File read failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("list_files")
def list_files(path: str = ".") -> List[str]:
    """List files in a directory"""
    try:
        safe_dir_path = safe_path(path)
        if safe_dir_path.is_dir():
            files = [str(p.relative_to(WORKSPACE_DIR)) for p in safe_dir_path.iterdir() if p.is_file()]
            log_activity("tool", "Directory listed", {"path": path})
            return files
        elif safe_dir_path.is_file():
            log_activity("tool", "File listed", {"path": path})
            return [str(safe_dir_path.relative_to(WORKSPACE_DIR))]
        else:
            return []
    except Exception as e:
        log_activity("tool", "Directory listing failed", {"path": path, "error": str(e)})
        return [f"❌ Error listing {path}: {e}"]

@register_tool("run_agent_task")
def run_agent_task(goal: str, context: Optional[str] = None) -> str:
    """Run a task through the agent system"""
    result = agent.run_task(goal, context)
    log_activity("tool", "Agent task completed", {"goal": goal})
    return result

@register_tool("get_agent_memory")
def get_agent_memory() -> Dict[str, Any]:
    """Get the agent's memory"""
    return {
        "memory": agent.memory,
        "session_id": agent.session_id,
        "memory_count": len(agent.memory)
    }

@register_tool("clear_agent_memory")
def clear_agent_memory() -> str:
    """Clear the agent's memory"""
    old_count = len(agent.memory)
    agent.memory.clear()
    log_activity("tool", "Agent memory cleared", {"old_count": old_count})
    return f"🧹 Cleared {old_count} memory entries"

@register_tool("install_package")
def install_package(package_name: str) -> str:
    """Install a Python package using pip"""
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package_name],
                                capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            log_activity("tool", "Package installed", {"package": package_name})
            return f"✅ Successfully installed {package_name}\n{result.stdout}"
        else:
            log_activity("tool", "Package installation failed", {"package": package_name, "error": result.stderr})
            return f"❌ Failed to install {package_name}\n{result.stderr}"
    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Installation of {package_name} timed out"
        log_activity("tool", "Package installation timed out", {"package": package_name})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error installing {package_name}: {e}"
        log_activity("tool", "Package installation error", {"package": package_name, "error": str(e)})
        return error_msg

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30) -> str:
    """Execute Python code in a subprocess"""
    try:
        start_time = time.time()
        # Write code to a temporary file
        temp_file = WORKSPACE_DIR / f"temp_exec_{int(time.time())}.py"
        with open(temp_file, 'w') as f:
            f.write(code)

        # Run the code
        result = subprocess.run([sys.executable, str(temp_file)],
                                capture_output=True, text=True, timeout=timeout,
                                cwd=str(WORKSPACE_DIR))

        # Clean up
        temp_file.unlink(missing_ok=True)

        execution_time = time.time() - start_time
        if result.returncode == 0:
            output = f"✅ Code executed successfully (in {execution_time:.2f}s):\n{result.stdout}"
            if result.stderr:
                output += f"\n⚠️ Stderr:\n{result.stderr}"
            log_activity("tool", "Python code executed", {"execution_time": execution_time})
        else:
            output = f"❌ Code execution failed (in {execution_time:.2f}s):\n{result.stderr}"
            if result.stdout:
                output += f"\n_stdout:\n{result.stdout}"
            log_activity("tool", "Python code execution failed", {"execution_time": execution_time, "error": result.stderr})

        return output

    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Code execution timed out after {timeout} seconds"
        log_activity("tool", "Python code timeout", {"timeout": timeout})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error executing code: {e}\n{traceback.format_exc()}"
        log_activity("tool", "Python code execution error", {"error": str(e)})
        return error_msg

print(f"✅ Registered {len(TOOL_REGISTRY)} tools")


# --- STEP 10: SETTING UP FASTAPI APPLICATION ---
print("🌐 Setting up FastAPI application...")
# Pydantic models
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# Initialize FastAPI app (Box 2 style)
app = FastAPI(
    title="Unified Manus MCP Server",
    description="Multi-agent coding assistant and tool API",
    version="7.0.0"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Adjust for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# API Endpoints (Box 2 style)
@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "status": "Unified Manus MCP System v7.0.0 online",
        "box": "2 - Agent Core",
        "docs": "/docs",
        "tool_call": "/mcp/tools/call",
        "tool_list": "/mcp/tools/list",
        "agent_status": f"Active - Session {agent.session_id}",
        "tools_available": len(TOOL_REGISTRY),
        "public_url": public_url,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "box": 2,
        "agent_memory_size": len(agent.memory),
        "tools_registered": len(TOOL_REGISTRY),
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/tools/call")
async def call_tool(tool_call: ToolCall):
    """Execute a registered tool"""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found"})

    try:
        # Execute the tool
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("tool_error", f"Tool '{tool_name}' failed", {"error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Tool execution failed: {str(e)}", "details": tb_str})

@app.get("/mcp/tools/list")
async def list_tools():
    """List all available tools"""
    tools_info = []
    for name, func in TOOL_REGISTRY.items():
        description = func.__doc__.strip() if func.__doc__ else "No description provided."
        # Get function signature
        try:
            sig = inspect.signature(func)
            parameters = {}
            for param_name, param in sig.parameters.items():
                param_info = {
                    "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any",
                    "required": param.default == inspect.Parameter.empty
                }
                if param.default != inspect.Parameter.empty:
                    param_info["default"] = param.default
                parameters[param_name] = param_info
        except Exception:
            parameters = {"error": "Could not parse parameters"}

        tools_info.append({
            "name": name,
            "description": description,
            "parameters": parameters
        })
    return {
        "tools": tools_info,
        "count": len(tools_info),
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/agent/task")
async def run_agent_task_endpoint(request: TaskRequest):
    """Run a task through the agent system"""
    try:
        result = agent.run_task(request.task, request.context)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("agent_error", "Agent task failed", {"task": request.task, "error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Agent task failed: {str(e)}", "details": tb_str})

@app.get("/mcp/agent/memory")
async def get_agent_memory_endpoint():
    """Get the agent's memory"""
    return get_agent_memory()

@app.post("/mcp/agent/memory/clear")
async def clear_agent_memory_endpoint():
    """Clear the agent's memory"""
    return {"result": clear_agent_memory()}

# --- STEP 11: SERVER LAUNCH AND GUI SYSTEMS (Box 3 Style) ---
print("🚀 Setting up Server Launch and GUI Systems...")

# Box 3 specific setup (merged)
box2_running = True # Assume Box 2 components are available in this unified script

# Create plugin manifests (Box 3 feature)
def create_plugin_manifests():
    """Create plugin manifest files for AI integration"""
    print("📄 Creating plugin manifest files...")
    site_dir = BASE_DIR / "site"
    site_dir.mkdir(exist_ok=True)

    # AI Plugin manifest
    ai_plugin_manifest = {
        "schema_version": "v1",
        "name_for_human": "Unified Manus MCP",
        "name_for_model": "unified_manus",
        "description_for_human": "Multi-agent coding assistant with comprehensive tool support",
        "description_for_model": "A unified agent system with file operations, Python execution, and multi-role thinking capabilities",
        "auth": {"type": "none"},
        "api": {"type": "openapi", "url": f"{public_url}/openapi.json"},
        "logo_url": f"{public_url}/static/logo.png",
        "contact_email": "support@manus.ai",
        "legal_info_url": f"{public_url}/legal"
    }
    with open(site_dir / "ai-plugin.json", "w") as f:
        json.dump(ai_plugin_manifest, f, indent=2)

    # OpenAPI/Swagger manifest snippet (FastAPI auto-generates)
    # Claude-compatible manifest
    claude_manifest = {
        "name": "unified_manus",
        "description": "Multi-agent coding assistant with comprehensive tool support",
        "version": "7.0.0",
        "endpoints": {
            "tool_call": f"{public_url}/mcp/tools/call",
            "tool_list": f"{public_url}/mcp/tools/list",
            "stream": f"{public_url}/mcp/tools/stream" # Assuming stream endpoint exists or will be added
        },
        "capabilities": ["file_operations", "python_execution", "package_management", "agent_thinking", "memory_management"]
    }
    try:
        import yaml
        with open(site_dir / "claude.yaml", "w") as f:
            yaml.dump(claude_manifest, f, default_flow_style=False)
    except ImportError:
        print("⚠️ PyYAML not found, skipping Claude manifest YAML creation.")

create_plugin_manifests()

# --- Gradio Interface ---
def setup_gradio_interface():
    """Setup Gradio interface"""
    if not GRADIO_AVAILABLE:
        print("⚠️ Gradio not available, skipping Gradio interface")
        return None

    print("🎨 Setting up Gradio interface...")

    def run_agent_task(task, context):
        """Run a task through the agent"""
        try:
            if box2_running_in_session:
                # Direct call to in-session agent
                result_dict = box2_agent.run_task(task, context)
                if result_dict.get("status") == "success":
                     return result_dict.get("final_output", "No final output found.")
                else:
                     return f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}"
            elif box2_api_accessible:
                 payload = {"task": task, "context": context}
                 response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                 if response.status_code == 200:
                     result_dict = response.json()
                     if result_dict.get("status") == "success":
                          return result_dict.get("final_output", "No final output found in API response.")
                     else:
                          return f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}"
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Agent Core) is not available."
        except Exception as e:
            return f"❌ Failed to run agent task: {str(e)}"

    def list_tools():
        """List available tools"""
        try:
             if box2_running_in_session:
                 from __main__ import TOOL_REGISTRY
                 tools_text = f"🛠️ Available Tools ({len(TOOL_REGISTRY)}):\n"
                 for name, func in TOOL_REGISTRY.items():
                     desc = func.__doc__.split('\n')[0] if func.__doc__ else "No description"
                     tools_text += f"• {name}: {desc}\n"
                 return tools_text
             elif box2_api_accessible:
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", timeout=10)
                 if response.status_code == 200:
                     result = response.json()
                     tools_text = f"🛠️ Available Tools ({result['count']}):\n"
                     for tool in result['tools']:
                         tools_text += f"• {tool['name']}: {tool['description']}\n"
                     return tools_text
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
             else:
                  return "❌ Box 2 (Tool Registry) is not available."
        except Exception as e:
            return f"❌ Failed to fetch tools: {str(e)}"

    def execute_tool(tool_name, tool_input):
        """Execute a specific tool"""
        try:
            if isinstance(tool_input, str) and tool_input.strip():
                try:
                    input_data = json.loads(tool_input)
                except json.JSONDecodeError:
                    input_data = {"content": tool_input}
            else:
                input_data = {}

            if box2_running_in_session:
                from __main__ import TOOL_REGISTRY
                if tool_name in TOOL_REGISTRY:
                    result = TOOL_REGISTRY[tool_name](**input_data)
                    if isinstance(result, dict):
                        return json.dumps(result, indent=2)
                    return str(result)
                else:
                    return f"❌ Tool '{tool_name}' not found."
            elif box2_api_accessible:
                payload = {"tool_name": tool_name, "tool_input": input_data}
                response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                if response.status_code == 200:
                    res_data = response.json()
                    result_content = res_data.get("result", "No result field")
                    if isinstance(result_content, dict):
                        return json.dumps(result_content, indent=2)
                    return str(result_content)
                else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Tool Execution) is not available."
        except Exception as e:
            return f"❌ Failed to execute tool: {str(e)}"

    # --- New Chat Functionality for Gradio ---
    def chat_with_model(message_history):
        """
        Chat with the model via the new /mcp/chat endpoint.
        Gradio's ChatInterface passes message_history as a list of [user_msg, bot_msg, user_msg, ...]
        We need to convert it to the format expected by /mcp/chat: [{"role": "...", "content": "..."}]
        """
        # Convert Gradio history to Ollama format
        ollama_messages = []
        for i, msg in enumerate(message_history):
            if i % 2 == 0: # User message
                ollama_messages.append({"role": "user", "content": msg})
            else: # Bot message
                ollama_messages.append({"role": "assistant", "content": msg})

        if box2_running_in_session and box2_api_accessible:
            # Prefer direct API call for streaming
            try:
                payload = {"messages": ollama_messages}
                # Note: Streaming responses from external APIs to Gradio chatbot can be complex.
                # A simpler approach is to make a non-streaming call.
                response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120)
                if response.status_code == 200:
                    # The /mcp/chat endpoint returns a stream. If we get a non-stream response here,
                    # it might be an aggregated final message or an error.
                    # Let's try to parse JSON. If it fails, return raw text.
                    try:
                        data = response.json()
                        # Check for common fields in the final aggregated response
                        if "message" in data:
                            # If it's the final message structure from the streaming endpoint
                            return data.get("message", {}).get("content", "Received message, content unclear (format 1).")
                        elif "content" in data:
                             # If it's a simple content response
                             return data["content"]
                        elif "final_output" in data: # Maybe the action_agent format was used somehow
                            return data["final_output"]
                        else:
                            # Return the whole JSON if structure is unknown
                            return f"Received JSON, structure unclear: {data}"
                    except json.JSONDecodeError:
                        # If response isn't JSON, return text content
                        return response.text or "Received response, but it was empty."
                else:
                    return f"❌ Chat API Error ({response.status_code}): {response.text}"
            except Exception as e:
                 return f"❌ Chat API Call Failed: {str(e)}"
        elif box2_running_in_session:
            # If only running in session, we'd need a way to call the Ollama chat function directly
            # and handle streaming. This is more complex in Gradio without async support in this context.
            # Fallback: Use a simple non-streaming direct call (if such a function exists or is adapted).
            # For now, indicate it's not fully supported via direct call in this GUI setup.
            return "⚠️ Direct chat not fully implemented in this GUI mode. Use API or Jupyter GUI for full chat."
        else:
             return "❌ Box 2 Chat API is not accessible."

    with gr.Blocks(title="Unified Manus MCP System (Ollama)", theme=gr.themes.Default()) as demo:
        gr.Markdown("# 🤖 Unified Manus MCP System v7.0.x (Ollama-powered)")
        gr.Markdown("Multi-Agent Coding Assistant with LLM Capabilities")

        with gr.Tab("Agent Tasks"):
            with gr.Row():
                with gr.Column():
                    task_input = gr.Textbox(label="Task", placeholder="Enter a task for the agent...")
                    context_input = gr.Textbox(label="Context (optional)", placeholder="Additional context...", lines=3)
                    run_btn = gr.Button("Run Task with Action Agent", variant="primary")
                with gr.Column():
                    task_output = gr.Textbox(label="Agent Output", lines=20, max_lines=30, show_copy_button=True)
            run_btn.click(fn=run_agent_task, inputs=[task_input, context_input], outputs=task_output)

        with gr.Tab("Tool Execution"):
            with gr.Row():
                with gr.Column():
                    tool_name_input = gr.Dropdown(label="Tool Name", choices=tools_available if tools_available else [], allow_custom_value=True)
                    tool_input_input = gr.Textbox(label="Tool Input (JSON)", placeholder='{"file_path": "test.txt", "content": "Hello"}', lines=5)
                    exec_tool_btn = gr.Button("Execute Tool", variant="secondary")
                with gr.Column():
                    tool_output = gr.Textbox(label="Tool Output", lines=15, max_lines=20, show_copy_button=True)
            exec_tool_btn.click(fn=execute_tool, inputs=[tool_name_input, tool_input_input], outputs=tool_output)

        with gr.Tab("Direct Chat (via /mcp/chat)"):
             # Gradio's ChatInterface is a convenient way to handle chat
             chatbot = gr.Chatbot(label="Conversation")
             msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
             clear_chat = gr.Button("Clear Chat")

             def respond(message, chat_history):
                 # Append user message to history
                 chat_history.append((message, None))
                 # Get response from chat function
                 bot_message = chat_with_model([item for sublist in chat_history for item in sublist if item is not None])
                 # Update the last entry in history with the bot's response
                 chat_history[-1] = (message, bot_message)
                 return "", chat_history

             msg.submit(respond, [msg, chatbot], [msg, chatbot])
             clear_chat.click(fn=lambda: ([], []), inputs=[], outputs=[chatbot, msg], queue=False)


        with gr.Tab("System Info"):
            with gr.Row():
                tools_btn = gr.Button("Refresh Tool List")
                tools_output = gr.Textbox(label="Available Tools", lines=15, max_lines=20)
                tools_btn.click(fn=list_tools, outputs=tools_output)

            info_text = (
                f"**System Information:**\n"
                f"- Public API URL: {public_url}\n"
                f"- Dashboard URL: {dashboard_url}\n"
                f"- Agent Session: {agent_session}\n"
                f"- Ollama Model: {ollama_model}\n"
                f"- Base Directory: {BASE_DIR}\n"
                f"- Workspace Directory: {WORKSPACE_DIR}\n"
                f"- Tools Available: {len(tools_available)}\n"
                f"- Box 2 Status: {'Integrated/Running' if box2_running else 'Not Accessible'}"
            )
            gr.Markdown(info_text)

    return demo

# Jupyter Interface (Box 3 feature)
def setup_jupyter_interface():
    """Setup Jupyter widget interface"""
    if not JUPYTER_AVAILABLE:
        print("⚠️ Jupyter widgets not available, skipping Jupyter interface")
        return None

    print("📓 Setting up Jupyter interface...")
    # This would typically involve creating widgets and displaying them.
    # A simplified version or placeholder:
    def create_jupyter_ui():
        clear_output(wait=True)
        display(HTML(f"""
        <div style="background: #2c3e50; color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px;">
            <h2>🤖 Unified Manus MCP System v7.0.x</h2>
            <p>Multi-Agent Coding Assistant - Jupyter Interface</p>
        </div>
        <div style="background: #ecf0f1; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
            <strong>System Status:</strong><br>
            🌍 Public URL: <a href="{public_url}" target="_blank">{public_url}</a><br>
            📊 Dashboard: <a href="{dashboard_url}" target="_blank">{dashboard_url}</a><br>
            🤖 Agent Session: {agent.session_id}<br>
            🛠️ Tools Available: {len(TOOL_REGISTRY)}
        </div>
        <p>Use the Gradio interface or API endpoints for full interaction.</p>
        """))
    return create_jupyter_ui

# Save Box 2/3 state (Box 2/3 feature)
def save_box2_state():
    """Save Box 2 state for other boxes"""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "memory_count": len(agent.memory),
        "api_endpoints": [
            "/mcp/tools/call",
            "/mcp/tools/list",
            # "/mcp/tools/stream", # Add if streaming endpoint is implemented
            "/mcp/agent/task",
            "/mcp/agent/memory"
        ],
        "timestamp": datetime.now().isoformat()
    }
    config_file = BASE_DIR / "config" / "box2_exports.json" # Save in Box 2 format
    config_file.parent.mkdir(parents=True, exist_ok=True)
    with open(config_file, "w") as f:
        json.dump(state, f, indent=2)
    print(f"✅ Box 2 state saved to {config_file}")

save_box2_state() # Save initial state

# Verification
def verify_setup():
    """Verify setup"""
    checks = {
        "Agent Initialized": agent is not None,
        "Tools Registered": len(TOOL_REGISTRY) > 0,
        "FastAPI App": app is not None,
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
    }
    print("🔍 Unified Manus System verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_setup()
if verification_passed:
    print("🎉 UNIFIED MANUS SYSTEM SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"🤖 Agent Session: {agent.session_id}")
    print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)}")
    print(f"🧠 Memory Entries: {len(agent.memory)}")
    print(f"🌐 API Ready at: {public_url}")
    print("=" * 60)
    print("🚀 System is ready! Launch server or GUI as needed.")
    log_activity("system", "Unified Manus System setup completed successfully", {"tools_count": len(TOOL_REGISTRY), "agent_session": agent.session_id})
else:
    print("❌ UNIFIED MANUS SYSTEM SETUP HAD ISSUES!")
    print("Please check the errors above.")
    log_activity("system", "Unified Manus System setup failed", {"errors": "See logs or output"})


# --- FINAL INSTRUCTIONS ---
print("\n🔧 To start the FastAPI server, run in a separate cell or terminal:")
print("```python")
print("import uvicorn")
print("uvicorn.run('this_script_name:app', host='0.0.0.0', port=8000, reload=True) # Adjust filename")
print("```")

if GRADIO_AVAILABLE:
    print("\n🎨 To launch the Gradio interface, run in a separate cell:")
    print("```python")
    print("demo = setup_gradio_interface()")
    print("if demo: demo.launch(share=True) # share=False for local only")
    print("```")

if JUPYTER_AVAILABLE:
    print("\n📓 To display the Jupyter interface, run in a cell:")
    print("```python")
    print("ui_func = setup_jupyter_interface()")
    print("if ui_func: ui_func()")
    print("```")

# Export for potential external use (like Box 3 importing)
__all__ = ['app', 'agent', 'TOOL_REGISTRY', 'setup_gradio_interface', 'setup_jupyter_interface']

print("✅ Unified Manus System initialization complete.")

🔧 Unified Manus System: Initializing...
📥 Loading configuration...
✅ Box 1 configuration loaded successfully
📁 Base Directory: /content/drive/MyDrive/UnifiedManusSystem
🌍 Public URL: http://localhost:8000
🔄 nest_asyncio applied
📦 Importing required modules for API and GUI...
✅ Jupyter widgets available
✅ Gradio available
📝 Setting up logging...
🎭 Setting up Agent Role system...
📁 Setting up File System Manager...
🤖 Setting up Core Manus Agent...
✅ ManusAgent system ready
🛠️ Setting up tool registry and safety functions...
🧰 Registering tools...
🔧 Registered tool: write_file
🔧 Registered tool: read_file
🔧 Registered tool: list_files
🔧 Registered tool: run_agent_task
🔧 Registered tool: get_agent_memory
🔧 Registered tool: clear_agent_memory
🔧 Registered tool: install_package
🔧 Registered tool: execute_python
✅ Registered 8 tools
🌐 Setting up FastAPI application...
🚀 Setting up Server Launch and GUI Systems...
📄 Creating plugin manifest files...
✅ Box 2 state saved to /content/drive/MyDriv

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 1: Core Infrastructure Setup - v7.0.x (Updated for Unified System)                              ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - Google Drive mount and folder structure                                                               ║
# ║ - Package installation with progress tracking (Updated for Unified System)                              ║
# ║ - FRP tunnel setup for public access                                                                    ║
# ║ - Base configuration initialization                                                                     ║
# ║ - Logging and configuration setup                                                                       ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 1: Initializing Core Infrastructure (Updated for Unified System)...")

import os
import sys
import time
import json
import signal
import subprocess
import traceback
import urllib.parse
import urllib.request
from datetime import datetime, timedelta
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📦 Step 1: Checking and installing core packages (Updated)...")

def install_with_progress(package):
    """Install a package using pip and show basic progress."""
    print(f"📦 Installing {package}...")
    try:
        # Use sys.executable to ensure we're using the correct Python/pip
        result = subprocess.run([sys.executable, "-m", "pip", "install", package],
                                capture_output=True, text=True, check=True)
        print(f"✅ Successfully installed {package}")
        # Optionally print stdout for detailed logs if needed
        # print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e.stderr}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error installing {package}: {e}")
        return False

# --- List of required packages for the Unified Manus System ---
# Includes packages for FastAPI, GUIs (Gradio, Jupyter), file handling, etc.
REQUIRED_PACKAGES = [
    "fastapi",
    "uvicorn[standard]", # Includes uvloop and httptools for better performance
    "pydantic",
    "requests",
    "nest_asyncio",
    # GUI Packages - Install if needed, handle potential errors gracefully
    # Gradio is large, consider installing only if explicitly needed later
    # "gradio", # Optional: Install via Box 3 or on-demand
    # Jupyter widgets are often present in notebook environments
    # "ipywidgets", # Optional: Check availability in Box 3
    # PyYAML for potential manifest creation (Box 3)
    "PyYAML"
]

# --- Installation Process ---
installation_success = True
for package in REQUIRED_PACKAGES:
    # Skip gradio/ipywidgets here, install in Box 3 if needed
    if package in ["gradio", "ipywidgets"]:
        print(f"⏭️ Skipping optional GUI package '{package}' for now. Will check/install in Box 3.")
        continue
    if not install_with_progress(package):
        installation_success = False

if not installation_success:
    print("⚠️ Some core packages failed to install. This might cause issues later.")
    # Depending on requirements, you might want to halt here or continue cautiously.
    # For now, let's warn but proceed.
else:
    print("✅ All core non-GUI packages installed successfully.")

print("📂 Step 2: Setting up directory structure...")

# --- Define Base Directory Structure ---
# Use a common path that works in Colab and potentially locally
BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
WORKSPACE_DIR = BASE_DIR / "workspace"
LOGS_DIR = BASE_DIR / "logs"
CONFIG_DIR = BASE_DIR / "config"
SITE_DIR = BASE_DIR / "site" # For plugin manifests, static files (Box 3)
STATIC_DIR = SITE_DIR / "static" # For static assets (Box 3)

# --- Create Directories ---
for directory in [BASE_DIR, WORKSPACE_DIR, LOGS_DIR, CONFIG_DIR, SITE_DIR, STATIC_DIR]:
    try:
        directory.mkdir(parents=True, exist_ok=True)
        print(f"📁 Created/Verified directory: {directory}")
    except Exception as e:
        print(f"❌ Error creating directory {directory}: {e}")
        installation_success = False # Mark as failure if critical dirs fail

# --- Change Working Directory ---
if BASE_DIR.exists():
    os.chdir(BASE_DIR)
    print(f"🔄 Changed working directory to: {BASE_DIR}")
else:
    print(f"❌ Base directory {BASE_DIR} does not exist. Cannot change directory.")
    installation_success = False

# --- Apply nest_asyncio (Important for Jupyter environments) ---
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied successfully.")
except ImportError:
    print("⚠️ nest_asyncio not found or failed to apply. Might be needed in Jupyter.")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")


print("🌐 Step 3: Setting up FRP tunnel for public access...")

# --- FRP Tunnel Setup (Example logic, adjust paths/commands as needed) ---
FRP_AVAILABLE = False
try:
    frp_check = subprocess.run(["frpc", "--version"], capture_output=True, text=True)
    if frp_check.returncode == 0:
        print("✅ FRP client found.")
        FRP_AVAILABLE = True
    else:
        print("⚠️ FRP client command failed.")
except FileNotFoundError:
    print("⚠️ FRP client ('frpc') not found in PATH.")
except Exception as e:
    print(f"⚠️ Error checking FRP: {e}")

public_url = "http://localhost:8000" # Default fallback
dashboard_url = "http://localhost:5000" # Default fallback

if FRP_AVAILABLE:
    # Example: Check for a specific FRP process or config file
    # This part is highly environment-dependent. Simplified example:
    frp_process_name = "frpc"
    try:
        # Check if frpc process is already running (Linux/macOS)
        result = subprocess.run(["pgrep", "-f", frp_process_name], capture_output=True, text=True)
        if result.stdout.strip():
            print("✅ FRP tunnel appears to be running.")
            # Getting the actual public URL from FRP config or logs is complex.
            # Assume a standard freefrp.net setup for demonstration.
            # You'd need to parse the FRP config or logs to get the real URL dynamically.
            # Placeholder - replace with actual logic to determine URL
            public_url = "http://your-assigned-subdomain.frp.freefrp.net" # Example
            dashboard_url = "http://your-assigned-subdomain.frp.freefrp.net:5000" # Example
            print(f"🌍 Public URL (assumed): {public_url}")
            print(f"📊 Dashboard URL (assumed): {dashboard_url}")
        else:
            print("🔄 FRP client found but not running. Starting tunnel...")
            # Example command to start FRP in background - ADJUST FOR YOUR CONFIG
            # Ensure 'frpc.ini' exists in BASE_DIR or specify the path correctly
            frp_config_path = BASE_DIR / "frpc.ini"
            if frp_config_path.exists():
                 # Use Popen to run in background
                 frp_process = subprocess.Popen(["frpc", "-c", str(frp_config_path)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                 print("🔄 FRP process started in background.")
                 # Give it a moment to initialize
                 time.sleep(5)
                 # Re-check status or assume it's working if config is correct
                 public_url = "http://your-assigned-subdomain.frp.freefrp.net" # Update logic needed
                 dashboard_url = "http://your-assigned-subdomain.frp.freefrp.net:5000" # Update logic needed
                 print(f"🌍 Public URL (assumed after start): {public_url}")
                 print(f"📊 Dashboard URL (assumed after start): {dashboard_url}")
            else:
                 print(f"⚠️ FRP config file not found at {frp_config_path}. Skipping tunnel start.")
    except Exception as e:
        print(f"⚠️ Error managing FRP tunnel: {e}")
else:
    print("⏭️ FRP not available/configured. Using localhost URLs.")
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"


print("📝 Step 4: Setting up logging...")

# --- Logging Setup ---
LOG_FILE = LOGS_DIR / "manus_log.json"
try:
    LOG_FILE.parent.mkdir(parents=True, exist_ok=True) # Ensure logs dir exists
    # Create empty log file if it doesn't exist
    if not LOG_FILE.exists():
        with open(LOG_FILE, "w") as f:
            json.dump([], f) # Initialize with an empty list
    print(f"📝 Log file initialized at: {LOG_FILE}")
except Exception as e:
    print(f"❌ Error setting up log file {LOG_FILE}: {e}")
    installation_success = False

def log_activity(category: str, message: str, data: Optional[Dict] = None):
    """Log activity to the JSON file."""
    try:
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "category": category,
            "message": message,
            "data": data or {}
        }
        # Read existing logs
        logs = []
        if LOG_FILE.exists():
            try:
                with open(LOG_FILE, "r") as f:
                    content = f.read()
                    if content:
                        logs = json.loads(content)
            except json.JSONDecodeError:
                print("⚠️ Log file corrupted, starting fresh.")
                logs = []

        logs.append(log_entry)

        # Write back logs (consider log rotation for large files)
        with open(LOG_FILE, "w") as f:
            json.dump(logs, f, indent=2)

        print(f"[LOG] {timestamp} [{category}] {message}")
    except Exception as e:
        print(f"❌ Failed to log activity: {e}")

log_activity("system", "Box 1 initialization started")

print("🛡️ Step 5: Setting up safety and path validation...")

# --- Environment Detection ---
IS_COLAB = "COLAB_RELEASE_TAG" in os.environ
IS_KAGGLE = "KAGGLE_CONTAINER_NAME" in os.environ
print(f"🖥️ Environment Detected: Colab={IS_COLAB}, Kaggle={IS_KAGGLE}")

# --- Safe Path Resolution Function ---
def safe_path(file_path: str) -> Path:
    """
    Resolve a file path safely within the BASE_DIR.
    Prevents directory traversal attacks.
    """
    if not file_path:
        print("⚠️ Empty file path provided, redirecting to workspace default.")
        return WORKSPACE_DIR / "default.txt"

    path = Path(file_path)

    # Handle absolute paths
    if path.is_absolute():
        try:
            # Try to make it relative to root, then prepend BASE_DIR
            path = path.relative_to(Path("/"))
            path = BASE_DIR / path
        except ValueError:
            # If it can't be made relative to root, put it in workspace by name
            print(f"⚠️ Absolute path {file_path} outside allowed structure, redirecting to workspace by name.")
            path = WORKSPACE_DIR / path.name
    else:
        # Relative path, prepend workspace
        path = WORKSPACE_DIR / path

    resolved = path.resolve()

    # Ensure the resolved path is within BASE_DIR
    try:
        resolved.relative_to(BASE_DIR.resolve())
    except ValueError:
        print(f"⚠️ Path {file_path} outside safe directory, redirecting to workspace")
        return WORKSPACE_DIR / Path(file_path).name

    return resolved

print("📊 Step 6: Creating system state and monitoring...")

# --- System State Tracking ---
SYSTEM_STATE = {
    "start_time": datetime.now().isoformat(),
    "base_dir": str(BASE_DIR),
    "workspace_dir": str(WORKSPACE_DIR),
    "logs_dir": str(LOGS_DIR),
    "config_dir": str(CONFIG_DIR),
    "public_url": public_url,
    "dashboard_url": dashboard_url,
    "is_colab": IS_COLAB,
    "is_kaggle": IS_KAGGLE,
    "components_ready": {
        "directories": True, # Set based on earlier checks
        "logging": LOG_FILE.exists(),
        "frp_tunnel": FRP_AVAILABLE, # Simplified
        "packages_installed": installation_success,
        "safety": True # Assuming path function works
    }
}

print("🔍 Step 7: Verification...")

def verify_installation():
    """Verify the Box 1 setup."""
    checks = {
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Logs Directory": LOGS_DIR.exists(),
        "Config Directory": CONFIG_DIR.exists(),
        "Site Directory": SITE_DIR.exists(),
        "Static Directory": STATIC_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
        "Core Packages Installed": installation_success, # Based on earlier result
        "nest_asyncio Applied": True, # Assume success if no exception
        # Add more checks as needed
    }
    print("🔍 Box 1 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_installation()

if verification_passed:
    print("🎉 BOX 1 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"📁 Base Directory: {BASE_DIR}")
    print(f"📂 Workspace Directory: {WORKSPACE_DIR}")
    print(f"🌍 Public URL: {public_url}")
    print(f"📊 Dashboard URL: {dashboard_url}")
    print(f"📝 Logs: {LOG_FILE}")
    print("=" * 60)
    print("🔄 Ready for Unified Manus System (Agent Core, Tools, Server, GUI)")
    log_activity("system", "Box 1 setup completed successfully")
else:
    print("❌ BOX 1 SETUP HAD ISSUES!")
    print("Please check the errors above before proceeding.")
    log_activity("system", "Box 1 setup failed", {"errors": "See output above"})

print("📤 Exporting essential variables for Unified System...")

# --- Export Configuration for Unified System ---
# This replaces the old 'box1_exports.json' content to be more suitable
essential_vars = {
    "BASE_DIR": str(BASE_DIR),
    "WORKSPACE_DIR": str(WORKSPACE_DIR),
    "LOG_FILE": str(LOG_FILE),
    "public_url": public_url,
    "dashboard_url": dashboard_url,
    "IS_COLAB": IS_COLAB,
    "IS_KAGGLE": IS_KAGGLE,
    "VERSION": "7.0.x-unified"
}

try:
    CONFIG_DIR.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    config_export_path = CONFIG_DIR / "box1_exports.json"
    with open(config_export_path, "w") as f:
        json.dump(essential_vars, f, indent=2)
    print(f"✅ Box 1 variables exported to {config_export_path}")
    log_activity("system", "Configuration exported", {"path": str(config_export_path)})
except Exception as e:
    print(f"❌ Failed to export Box 1 variables: {e}")
    log_activity("system", "Configuration export failed", {"error": str(e)})

if verification_passed:
    print("🚀 PROCEED TO RUN THE UNIFIED MANUS SYSTEM CODE!")
else:
    print("🛑 CRITICAL ERRORS IN BOX 1. UNIFIED SYSTEM MAY NOT FUNCTION CORRECTLY!")

🔧 BOX 1: Initializing Core Infrastructure (Updated for Unified System)...
📦 Step 1: Checking and installing core packages (Updated)...
📦 Installing fastapi...
✅ Successfully installed fastapi
📦 Installing uvicorn[standard]...
✅ Successfully installed uvicorn[standard]
📦 Installing pydantic...
✅ Successfully installed pydantic
📦 Installing requests...
✅ Successfully installed requests
📦 Installing nest_asyncio...
✅ Successfully installed nest_asyncio
📦 Installing PyYAML...
✅ Successfully installed PyYAML
✅ All core non-GUI packages installed successfully.
📂 Step 2: Setting up directory structure...
📁 Created/Verified directory: /content/drive/MyDrive/UnifiedManusSystem
📁 Created/Verified directory: /content/drive/MyDrive/UnifiedManusSystem/workspace
📁 Created/Verified directory: /content/drive/MyDrive/UnifiedManusSystem/logs
📁 Created/Verified directory: /content/drive/MyDrive/UnifiedManusSystem/config
📁 Created/Verified directory: /content/drive/MyDrive/UnifiedManusSystem/site
📁 Create

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 2: Agent Core and Tools - v7.0.x (Updated for Revised Box 1)                                    ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - ManusAgent with role-based thinking system                                                            ║
# ║ - Complete tool registry and execution system                                                           ║
# ║ - File operations, Python execution, package management                                                 ║
# ║ - FastAPI app with all tool endpoints                                                                   ║
# ║ - Real-time streaming and dashboard integration                                                         ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 2: Initializing Agent Core and Tools (Updated for Revised Box 1)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📥 Step 1: Loading Box 1 configuration (Updated Path)...")

# Load configuration from Box 1 - Simplified path based on updated Box 1
try:
    # Standardized path from updated Box 1
    config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
    # Fallback for local runs (if needed, but Box 1 sets this path)
    if not config_file.exists():
         config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

    if config_file.exists():
        with open(config_file, "r") as f:
            box1_config = json.load(f)

        BASE_DIR = Path(box1_config["BASE_DIR"])
        WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
        LOG_FILE = Path(box1_config["LOG_FILE"])
        public_url = box1_config["public_url"]
        dashboard_url = box1_config["dashboard_url"]
        IS_COLAB = box1_config["IS_COLAB"]
        print("✅ Box 1 configuration loaded successfully")
        print(f"📁 Base Directory: {BASE_DIR}")
        print(f"🌍 Public URL: {public_url}")
    else:
        raise FileNotFoundError("Box 1 config not found at expected location")

except Exception as e:
    print(f"❌ Failed to load Box 1 config: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True # Adjust based on environment

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules...")

# Import all necessary modules
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")

# Core FastAPI imports
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from pydantic import BaseModel
import requests # For proxying calls if needed

# YAML import - Box 1 now guarantees PyYAML is installed
try:
    import yaml
    YAML_AVAILABLE = True
    print("✅ PyYAML available (as installed by updated Box 1)")
except ImportError:
    YAML_AVAILABLE = False
    print("⚠️ PyYAML not available")

print("✅ All Box 2 modules imported successfully")

print("📝 Step 3: Setting up logging...")

def log_activity(category: str, message: str, data: Optional[Dict[str, Any]] = None):
    """Log activity to the JSON file."""
    try:
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "category": category,
            "message": message,
            "data": data or {},
            "box": "2"
        }

        # Read existing logs
        logs = []
        if LOG_FILE.exists():
            try:
                with open(LOG_FILE, "r") as f:
                    content = f.read()
                    if content.strip(): # Check if file is not empty
                         logs = json.loads(content)
            except json.JSONDecodeError:
                print("⚠️ Log file corrupted, starting fresh.")
                logs = []
        else:
            # Create parent directories if needed
            LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
            # Create an empty log file
            LOG_FILE.write_text("[]")

        logs.append(log_entry)

        # Keep last 1000 entries
        if len(logs) > 1000:
            logs = logs[-1000:]

        with open(LOG_FILE, "w") as f:
            json.dump(logs, f, indent=2)

        print(f"[LOG] {timestamp} [{category}] {message}")
    except Exception as e:
        print(f"❌ Logging failed: {e}")

log_activity("system", "Box 2 initialization started")

print("🛡️ Step 4: Setting up safety and path validation...")

def safe_path(file_path: str) -> Path:
    """Ensure file operations stay within safe directory"""
    if not file_path:
        raise ValueError("File path cannot be empty")

    base_path = WORKSPACE_DIR.resolve()
    full_path = (base_path / file_path).resolve()

    try:
        full_path.relative_to(base_path) # Raises ValueError if not relative
        return full_path
    except ValueError:
        raise PermissionError(f"Access denied: {file_path} is outside the workspace")

print("🔧 Step 5: Registering all core tools...")

# Tool registry
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str):
    """Decorator to register tools"""
    def decorator(func: Callable):
        TOOL_REGISTRY[name] = func
        print(f"🔧 Registered tool: {name}")
        log_activity("system", f"Tool registered: {name}")
        return func
    return decorator

# --- Registering Tools ---

@register_tool("write_file")
def write_file(file_path: str, content: str) -> str:
    """Write content to a file"""
    try:
        safe_file_path = safe_path(file_path)
        safe_file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(safe_file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        log_activity("tool", "File written", {"file": file_path})
        return f"✅ Successfully wrote to {safe_file_path}"
    except Exception as e:
        error_msg = f"❌ Failed to write file {file_path}: {e}"
        log_activity("tool", "File write failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("read_file")
def read_file(file_path: str) -> str:
    """Read content from a file"""
    try:
        safe_file_path = safe_path(file_path)
        with open(safe_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        log_activity("tool", "File read", {"file": file_path})
        return content
    except Exception as e:
        error_msg = f"❌ Failed to read file {file_path}: {e}"
        log_activity("tool", "File read failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("list_files")
def list_files(path: str = ".") -> List[str]:
    """List files in a directory"""
    try:
        safe_dir_path = safe_path(path)
        if safe_dir_path.is_dir():
            files = [str(p.relative_to(WORKSPACE_DIR)) for p in safe_dir_path.iterdir() if p.is_file()]
            log_activity("tool", "Directory listed", {"path": path})
            return files
        elif safe_dir_path.is_file():
            log_activity("tool", "File listed", {"path": path})
            return [str(safe_dir_path.relative_to(WORKSPACE_DIR))]
        else:
            return []
    except Exception as e:
        log_activity("tool", "Directory listing failed", {"path": path, "error": str(e)})
        return [f"❌ Error listing {path}: {e}"]

# --- Agent Role System ---
print("🎭 Setting up Agent Role system...")
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        """Simulate role processing."""
        # Placeholder logic - replace with actual LLM call
        response = f"[{self.name}] Processing task: {task_description}"
        if stream_callback:
             stream_callback(response + "\n")
        time.sleep(0.5) # Simulate processing time
        return response

# Define roles
ROLES = {
    "planner": ManusRole(
        "Planner",
        "Creates plans",
        "You are an expert planner. Break down complex tasks into smaller, manageable steps."
    ),
    "researcher": ManusRole(
        "Researcher",
        "Gathers information",
        "You are a research assistant. When asked to research or find information, use the provided tools effectively. Summarize findings clearly and concisely. Only provide the summary, not the tool call itself."
    ),
    "coder": ManusRole(
        "Coder",
        "Writes code",
        "You are a skilled software engineer. Write clean, efficient, and well-documented code based on the plan provided."
    ),
    "reviewer": ManusRole(
        "Reviewer",
        "Reviews code",
        "You are a senior engineer reviewing code. Check for correctness, efficiency, and best practices."
    ),
    "debugger": ManusRole(
        "Debugger",
        "Fixes code",
        "You are an expert debugger. When given code and error messages, identify the root cause and provide fixed code. Explain your reasoning clearly."
    ),
}

# --- File System Manager ---
print("📁 Setting up File System Manager...")
class FileSystemManager:
    def __init__(self, base_path: Path = WORKSPACE_DIR):
        self.base_path = base_path.resolve()
        self.base_path.mkdir(parents=True, exist_ok=True)

    def safe_join(self, *paths) -> Path:
        """Join paths and ensure the result is within the base path."""
        full_path = Path(self.base_path, *paths).resolve()
        try:
            full_path.relative_to(self.base_path) # Raises ValueError if not relative
            return full_path
        except ValueError:
            raise PermissionError(f"Path traversal attempt: {full_path}")

    def read_file(self, file_path: str) -> str:
        """Read a file safely."""
        full_path = self.safe_join(file_path)
        with open(full_path, 'r', encoding='utf-8') as f:
            return f.read()

    def write_file(self, file_path: str, content: str) -> str:
        """Write content to a file safely."""
        full_path = self.safe_join(file_path)
        full_path.parent.mkdir(parents=True, exist_ok=True) # Ensure parent dirs exist
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ File written to {full_path}"

    def list_files(self, dir_path: str = ".") -> List[str]:
        """List files in a directory safely."""
        full_path = self.safe_join(dir_path)
        if full_path.is_dir():
            return [str(p.relative_to(self.base_path)) for p in full_path.iterdir() if p.is_file()]
        else:
            return [str(full_path.relative_to(self.base_path))] if full_path.is_file() else []

# --- Core Agent Class ---
print("🤖 Setting up Core Manus Agent...")
class ManusAgent:
    def __init__(self):
        self.roles = ROLES # Use the roles defined above
        self.fs = FileSystemManager() # Use the FS manager
        self.memory: List[Dict[str, Any]] = []
        self.session_id = f"session_{int(time.time())}"
        log_activity("system", "Manus Agent initialized")

    def solve_task(self, task_description: str, stream_callback=None):
        """Main task solving logic using roles."""
        log_activity("agent", "Task started", {"task": task_description})
        self.memory.append({"type": "task_start", "content": task_description, "timestamp": datetime.now().isoformat()})
        if stream_callback:
            stream_callback(f"🧠 Starting task: {task_description}\n")

        # Example multi-step process (replace with your logic)
        plan_output = self.roles["planner"].process(f"Create a plan for: {task_description}", stream_callback)
        self.memory.append({"type": "thought", "role": "planner", "content": plan_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"📝 Plan: {plan_output}\n")

        code_output = self.roles["coder"].process(f"Write code based on plan: {plan_output}", stream_callback)
        self.memory.append({"type": "action", "role": "coder", "content": code_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"💻 Code: {code_output}\n")

        review_output = self.roles["reviewer"].process(f"Review code: {code_output}", stream_callback)
        self.memory.append({"type": "thought", "role": "reviewer", "content": review_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"🔍 Review: {review_output}\n")

        final_result = f"Task '{task_description}' completed.\nPlan: {plan_output}\nCode: {code_output}\nReview: {review_output}"
        self.memory.append({"type": "task_end", "content": final_result, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"✅ Task completed: {final_result}\n")
        log_activity("agent", "Task completed", {"task": task_description})
        return final_result

    def run_task(self, goal: str, context: Optional[str] = None) -> str:
        """Wrapper for solve_task, potentially adding context."""
        task_with_context = f"{goal}\nContext: {context}" if context else goal
        captured_output = []
        def capture_stream(text):
            captured_output.append(text)
        result = self.solve_task(task_with_context, stream_callback=capture_stream)
        # Return combined captured stream and result if needed, or just result
        return result # Or '\n'.join(captured_output) + '\n' + result

# Initialize the global agent
agent = ManusAgent()
print("✅ ManusAgent system ready")

@register_tool("run_agent_task")
def run_agent_task(goal: str, context: Optional[str] = None) -> str:
    """Run a task through the agent system"""
    result = agent.run_task(goal, context)
    log_activity("tool", "Agent task completed", {"goal": goal})
    return result

@register_tool("get_agent_memory")
def get_agent_memory() -> Dict[str, Any]:
    """Get the agent's memory"""
    return {
        "memory": agent.memory,
        "session_id": agent.session_id,
        "memory_count": len(agent.memory)
    }

@register_tool("clear_agent_memory")
def clear_agent_memory() -> str:
    """Clear the agent's memory"""
    old_count = len(agent.memory)
    agent.memory.clear()
    log_activity("tool", "Agent memory cleared", {"old_count": old_count})
    return f"🧹 Cleared {old_count} memory entries"

@register_tool("install_package")
def install_package(package_name: str) -> str:
    """Install a Python package using pip"""
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package_name],
                                capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            log_activity("tool", "Package installed", {"package": package_name})
            return f"✅ Successfully installed {package_name}\n{result.stdout}"
        else:
            log_activity("tool", "Package installation failed", {"package": package_name, "error": result.stderr})
            return f"❌ Failed to install {package_name}\n{result.stderr}"
    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Installation of {package_name} timed out"
        log_activity("tool", "Package installation timed out", {"package": package_name})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error installing {package_name}: {e}"
        log_activity("tool", "Package installation error", {"package": package_name, "error": str(e)})
        return error_msg

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30) -> str:
    """Execute Python code in a subprocess"""
    try:
        start_time = time.time()
        # Write code to a temporary file
        temp_file = WORKSPACE_DIR / f"temp_exec_{int(time.time())}.py"
        with open(temp_file, 'w') as f:
            f.write(code)

        # Run the code
        result = subprocess.run([sys.executable, str(temp_file)],
                                capture_output=True, text=True, timeout=timeout,
                                cwd=str(WORKSPACE_DIR))

        # Clean up
        temp_file.unlink(missing_ok=True)

        execution_time = time.time() - start_time
        if result.returncode == 0:
            output = f"✅ Code executed successfully (in {execution_time:.2f}s):\n{result.stdout}"
            if result.stderr:
                output += f"\n⚠️ Stderr:\n{result.stderr}"
            log_activity("tool", "Python code executed", {"execution_time": execution_time})
        else:
            output = f"❌ Code execution failed (in {execution_time:.2f}s):\n{result.stderr}"
            if result.stdout:
                output += f"\n_stdout:\n{result.stdout}"
            log_activity("tool", "Python code execution failed", {"execution_time": execution_time, "error": result.stderr})

        return output

    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Code execution timed out after {timeout} seconds"
        log_activity("tool", "Python code timeout", {"timeout": timeout})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error executing code: {e}\n{traceback.format_exc()}"
        log_activity("tool", "Python code execution error", {"error": str(e)})
        return error_msg


print(f"✅ Registered {len(TOOL_REGISTRY)} tools")

print("🌐 Step 6: Setting up FastAPI application...")

# Pydantic models
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# Initialize FastAPI app
app = FastAPI(
    title="Unified Manus MCP Server",
    description="Multi-agent coding assistant and tool API",
    version="7.0.x"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Adjust for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# API Endpoints
@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "status": "Unified Manus MCP System v7.0.x online",
        "box": "2 - Agent Core",
        "docs": "/docs",
        "tool_call": "/mcp/tools/call",
        "tool_list": "/mcp/tools/list",
        "agent_status": f"Active - Session {agent.session_id}",
        "tools_available": len(TOOL_REGISTRY),
        "public_url": public_url,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "box": 2,
        "agent_memory_size": len(agent.memory),
        "tools_registered": len(TOOL_REGISTRY),
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/tools/call")
async def call_tool(tool_call: ToolCall):
    """Execute a registered tool"""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found"})

    try:
        # Execute the tool
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("tool_error", f"Tool '{tool_name}' failed", {"error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Tool execution failed: {str(e)}", "details": tb_str})

@app.get("/mcp/tools/list")
async def list_tools():
    """List all available tools"""
    tools_info = []
    for name, func in TOOL_REGISTRY.items():
        description = func.__doc__.strip() if func.__doc__ else "No description provided."
        # Get function signature
        try:
            sig = inspect.signature(func)
            parameters = {}
            for param_name, param in sig.parameters.items():
                param_info = {
                    "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any",
                    "required": param.default == inspect.Parameter.empty
                }
                if param.default != inspect.Parameter.empty:
                    param_info["default"] = param.default
                parameters[param_name] = param_info
        except Exception:
            parameters = {"error": "Could not parse parameters"}

        tools_info.append({
            "name": name,
            "description": description,
            "parameters": parameters
        })
    return {
        "tools": tools_info,
        "count": len(tools_info),
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/agent/task")
async def run_agent_task_endpoint(request: TaskRequest):
    """Run a task through the agent system"""
    try:
        result = agent.run_task(request.task, request.context)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("agent_error", "Agent task failed", {"task": request.task, "error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Agent task failed: {str(e)}", "details": tb_str})

@app.get("/mcp/agent/memory")
async def get_agent_memory_endpoint():
    """Get the agent's memory"""
    return get_agent_memory()

@app.post("/mcp/agent/memory/clear")
async def clear_agent_memory_endpoint():
    """Clear the agent's memory"""
    return {"result": clear_agent_memory()}

@app.get("/mcp/system/info")
async def get_system_info():
    """Get system information"""
    return {
        "box": 2,
        "name": "Agent Core and Tools",
        "version": "7.0.x",
        "base_dir": str(BASE_DIR),
        "workspace_dir": str(WORKSPACE_DIR),
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "is_colab": IS_COLAB,
        "agent_session": agent.session_id,
        "tools_count": len(TOOL_REGISTRY),
        "memory_entries": len(agent.memory),
        "uptime": datetime.now().isoformat(),
        "python_version": sys.version,
        "working_directory": os.getcwd()
    }


print("🔄 Step 7: Setting up streaming and real-time features...")
# Real-time queues for streaming (placeholder for potential future use)
thought_queue = queue.Queue()
output_queue = queue.Queue()

# If you plan to add streaming endpoints, define them here
# Example (requires `sse-starlette`):
# from sse_starlette.sse import EventSourceResponse
# @app.get("/mcp/tools/stream")
# async def stream_tool_call(tool_call: ToolCall):
#     async def event_generator():
#         # ... streaming logic ...
#     return EventSourceResponse(event_generator())

print("💾 Step 8: Setting up data persistence...")

def save_box2_state():
    """Save Box 2 state for other boxes"""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "memory_count": len(agent.memory),
        "api_endpoints": [
            "/mcp/tools/call",
            "/mcp/tools/list",
            # "/mcp/tools/stream", # Add if streaming endpoint is implemented
            "/mcp/agent/task",
            "/mcp/agent/memory"
        ],
        "timestamp": datetime.now().isoformat()
    }
    config_file = BASE_DIR / "config" / "box2_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    with open(config_file, "w") as f:
        json.dump(state, f, indent=2)
    print(f"✅ Box 2 state saved to {config_file}")
    log_activity("system", "Box 2 state saved", {"path": str(config_file)})

save_box2_state()

print("🔍 Step 9: Verification and testing...")

def verify_box2_setup():
    """Verify Box 2 setup"""
    checks = {
        "Agent Initialized": agent is not None,
        "Tools Registered": len(TOOL_REGISTRY) > 0,
        "FastAPI App": app is not None,
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
    }
    print("🔍 Box 2 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box2_setup()

if verification_passed:
    print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"🤖 Agent Session: {agent.session_id}")
    print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)}")
    print(f"🧠 Memory Entries: {len(agent.memory)}")
    print(f"🌐 API Ready at: {public_url}")
    print("=" * 60)
    print("🔄 Ready for Box 3: Server Launch and GUI")
    log_activity("system", "Box 2 setup completed successfully", {"tools_count": len(TOOL_REGISTRY), "agent_session": agent.session_id})

    # Test a simple tool
    try:
        test_result = TOOL_REGISTRY["run_agent_task"]("Test task for verification")
        print("✅ Agent test completed successfully")
        log_activity("system", "Agent test completed successfully")
    except Exception as e:
        print(f"⚠️ Agent test failed: {e}")
        log_activity("system", "Agent test failed", {"error": str(e)})

else:
    print("❌ BOX 2 SETUP HAD ISSUES!")
    print("Please check the errors above before proceeding to Box 3")
    log_activity("system", "Box 2 setup failed", {"errors": "See output above"})

print("📤 Box 2 ready for integration with Box 3!")
print("🚀 PROCEED TO BOX 3 to launch the complete system!")

# Make the app available for starting the server
print("🔧 To start the server manually, use:")
print("```python")
print("import uvicorn")
print("uvicorn.run('this_script_filename:app', host='0.0.0.0', port=8000, reload=True)")
print("```")
print("(Replace 'this_script_filename' with the actual name of this Python file without .py)")

# Export the app for Box 3 to use
__all__ = ['app', 'agent', 'TOOL_REGISTRY']


🔧 BOX 2: Initializing Agent Core and Tools (Updated for Revised Box 1)...
📥 Step 1: Loading Box 1 configuration (Updated Path)...
✅ Box 1 configuration loaded successfully
📁 Base Directory: /content/drive/MyDrive/UnifiedManusSystem
🌍 Public URL: http://localhost:8000
📦 Step 2: Importing required modules...
🔄 nest_asyncio applied
✅ PyYAML available (as installed by updated Box 1)
✅ All Box 2 modules imported successfully
📝 Step 3: Setting up logging...
⚠️ Log file corrupted, starting fresh.
[LOG] 2025-07-27T14:28:31.996936 [system] Box 2 initialization started
🛡️ Step 4: Setting up safety and path validation...
🔧 Step 5: Registering all core tools...
🔧 Registered tool: write_file
[LOG] 2025-07-27T14:28:31.997720 [system] Tool registered: write_file
🔧 Registered tool: read_file
[LOG] 2025-07-27T14:28:31.998088 [system] Tool registered: read_file
🔧 Registered tool: list_files
[LOG] 2025-07-27T14:28:31.998475 [system] Tool registered: list_files
🎭 Setting up Agent Role system...
📁 Setting 

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 2: Agent Core and Tools - v7.0.x (Updated for Revised Box 1 & Callable Fix)                     ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - ManusAgent with role-based thinking system (Integrated as 'action_agent' tool)                        ║
# ║ - Complete tool registry and execution system                                                           ║
# ║ - File operations, Python execution, package management                                                 ║
# ║ - FastAPI app with all tool endpoints                                                                   ║
# ║ - Real-time streaming and dashboard integration                                                         ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 2: Initializing Agent Core and Tools (Updated for Revised Box 1 & Callable Fix)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable # Ensure Callable is imported

print("📥 Step 1: Loading Box 1 configuration (Updated Path)...")

# Load configuration from Box 1 - Simplified path based on updated Box 1
try:
    # Standardized path from updated Box 1
    config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
    # Fallback for local runs (if needed, but Box 1 sets this path)
    if not config_file.exists():
         config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

    if config_file.exists():
        with open(config_file, "r") as f:
            box1_config = json.load(f)

        BASE_DIR = Path(box1_config["BASE_DIR"])
        WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
        LOG_FILE = Path(box1_config["LOG_FILE"])
        public_url = box1_config["public_url"]
        dashboard_url = box1_config["dashboard_url"]
        IS_COLAB = box1_config["IS_COLAB"]
        print("✅ Box 1 configuration loaded successfully")
        print(f"📁 Base Directory: {BASE_DIR}")
        print(f"🌍 Public URL: {public_url}")
    else:
        raise FileNotFoundError("Box 1 config not found at expected location")

except Exception as e:
    print(f"❌ Failed to load Box 1 config: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True # Adjust based on environment

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules...")

# Import all necessary modules
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")

# Core FastAPI imports
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from pydantic import BaseModel
import requests # For proxying calls if needed

# YAML import - Box 1 now guarantees PyYAML is installed
try:
    import yaml
    YAML_AVAILABLE = True
    print("✅ PyYAML available (as installed by updated Box 1)")
except ImportError:
    YAML_AVAILABLE = False
    print("⚠️ PyYAML not available")

print("✅ All Box 2 modules imported successfully")

print("📝 Step 3: Setting up logging...")

def log_activity(category: str, message: str, data: Optional[Dict[str, Any]] = None):
    """Log activity to the JSON file."""
    try:
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "category": category,
            "message": message,
            "data": data or {},
            "box": "2"
        }

        # Read existing logs
        logs = []
        if LOG_FILE.exists():
            try:
                with open(LOG_FILE, "r") as f:
                    content = f.read()
                    if content.strip(): # Check if file is not empty
                         logs = json.loads(content)
            except json.JSONDecodeError:
                print("⚠️ Log file corrupted, starting fresh.")
                logs = []
        else:
            # Create parent directories if needed
            LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
            # Create an empty log file
            LOG_FILE.write_text("[]")

        logs.append(log_entry)

        # Keep last 1000 entries
        if len(logs) > 1000:
            logs = logs[-1000:]

        with open(LOG_FILE, "w") as f:
            json.dump(logs, f, indent=2)

        print(f"[LOG] {timestamp} [{category}] {message}")
    except Exception as e:
        print(f"❌ Logging failed: {e}")

log_activity("system", "Box 2 initialization started")

print("🛡️ Step 4: Setting up safety and path validation...")

def safe_path(file_path: str) -> Path:
    """Ensure file operations stay within safe directory"""
    if not file_path:
        raise ValueError("File path cannot be empty")

    base_path = WORKSPACE_DIR.resolve()
    full_path = (base_path / file_path).resolve()

    try:
        full_path.relative_to(base_path) # Raises ValueError if not relative
        return full_path
    except ValueError:
        raise PermissionError(f"Access denied: {file_path} is outside the workspace")

print("🎭 Step 5: Setting up Agent Role system...")
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        """Simulate role processing."""
        # Placeholder logic - replace with actual LLM call
        response = f"[{self.name}] Processing task: {task_description}"
        if stream_callback:
             stream_callback(response + "\n")
        time.sleep(0.5) # Simulate processing time
        return response

# Define roles
ROLES = {
    "planner": ManusRole(
        "Planner",
        "Creates plans",
        "You are an expert planner. Break down complex tasks into smaller, manageable steps."
    ),
    "researcher": ManusRole(
        "Researcher",
        "Gathers information",
        "You are a research assistant. When asked to research or find information, use the provided tools effectively. Summarize findings clearly and concisely. Only provide the summary, not the tool call itself."
    ),
    "coder": ManusRole(
        "Coder",
        "Writes code",
        "You are a skilled software engineer. Write clean, efficient, and well-documented code based on the plan provided."
    ),
    "reviewer": ManusRole(
        "Reviewer",
        "Reviews code",
        "You are a senior engineer reviewing code. Check for correctness, efficiency, and best practices."
    ),
    "debugger": ManusRole(
        "Debugger",
        "Fixes code",
        "You are an expert debugger. When given code and error messages, identify the root cause and provide fixed code. Explain your reasoning clearly."
    ),
}

print("📁 Step 6: Setting up File System Manager...")
class FileSystemManager:
    def __init__(self, base_path: Path = WORKSPACE_DIR):
        self.base_path = base_path.resolve()
        self.base_path.mkdir(parents=True, exist_ok=True)

    def safe_join(self, *paths) -> Path:
        """Join paths and ensure the result is within the base path."""
        full_path = Path(self.base_path, *paths).resolve()
        try:
            full_path.relative_to(self.base_path) # Raises ValueError if not relative
            return full_path
        except ValueError:
            raise PermissionError(f"Path traversal attempt: {full_path}")

    def read_file(self, file_path: str) -> str:
        """Read a file safely."""
        full_path = self.safe_join(file_path)
        with open(full_path, 'r', encoding='utf-8') as f:
            return f.read()

    def write_file(self, file_path: str, content: str) -> str:
        """Write content to a file safely."""
        full_path = self.safe_join(file_path)
        full_path.parent.mkdir(parents=True, exist_ok=True) # Ensure parent dirs exist
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ File written to {full_path}"

    def list_files(self, dir_path: str = ".") -> List[str]:
        """List files in a directory safely."""
        full_path = self.safe_join(dir_path)
        if full_path.is_dir():
            return [str(p.relative_to(self.base_path)) for p in full_path.iterdir() if p.is_file()]
        else:
            return [str(full_path.relative_to(self.base_path))] if full_path.is_file() else []

print("🤖 Step 7: Setting up Core Manus Agent (Action Agent)...")
class ManusAgent:
    def __init__(self):
        self.roles = ROLES # Use the roles defined above
        self.fs = FileSystemManager() # Use the FS manager
        self.memory: List[Dict[str, Any]] = []
        self.session_id = f"session_{int(time.time())}"
        log_activity("system", "Manus Agent (Action Agent) initialized")

    def solve_task(self, task_description: str, stream_callback=None):
        """Main task solving logic using roles."""
        log_activity("agent", "Task started", {"task": task_description})
        self.memory.append({"type": "task_start", "content": task_description, "timestamp": datetime.now().isoformat()})
        if stream_callback:
            stream_callback(f"🧠 Starting task: {task_description}\n")

        # Example multi-step process (replace with your logic)
        plan_output = self.roles["planner"].process(f"Create a plan for: {task_description}", stream_callback)
        self.memory.append({"type": "thought", "role": "planner", "content": plan_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"📝 Plan: {plan_output}\n")

        code_output = self.roles["coder"].process(f"Write code based on plan: {plan_output}", stream_callback)
        self.memory.append({"type": "action", "role": "coder", "content": code_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"💻 Code: {code_output}\n")

        review_output = self.roles["reviewer"].process(f"Review code: {code_output}", stream_callback)
        self.memory.append({"type": "thought", "role": "reviewer", "content": review_output, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"🔍 Review: {review_output}\n")

        final_result = f"Task '{task_description}' completed.\nPlan: {plan_output}\nCode: {code_output}\nReview: {review_output}"
        self.memory.append({"type": "task_end", "content": final_result, "timestamp": datetime.now().isoformat()})
        if stream_callback:
             stream_callback(f"✅ Task completed: {final_result}\n")
        log_activity("agent", "Task completed", {"task": task_description})
        return final_result

    def run_task(self, goal: str, context: Optional[str] = None) -> str:
        """Wrapper for solve_task, potentially adding context."""
        task_with_context = f"{goal}\nContext: {context}" if context else goal
        # For direct tool call, we can capture output if needed.
        # The current solve_task doesn't stream to a return value in a way easily captured here.
        # Let's simplify and just return the final result string.
        result = self.solve_task(task_with_context) # Pass stream_callback if needed
        return result

# Initialize the global agent instance (the "action agent")
agent = ManusAgent()
print("✅ ManusAgent (Action Agent) system ready")

print("🔧 Step 8: Registering all core tools (including Action Agent)...")

# Tool registry - Now Callable is properly defined
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str):
    """Decorator to register tools"""
    def decorator(func: Callable):
        TOOL_REGISTRY[name] = func
        print(f"🔧 Registered tool: {name}")
        log_activity("system", f"Tool registered: {name}")
        return func
    return decorator

# --- Registering Core Tools ---

@register_tool("write_file")
def write_file(file_path: str, content: str) -> str:
    """Write content to a file"""
    try:
        safe_file_path = safe_path(file_path)
        safe_file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(safe_file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        log_activity("tool", "File written", {"file": file_path})
        return f"✅ Successfully wrote to {safe_file_path}"
    except Exception as e:
        error_msg = f"❌ Failed to write file {file_path}: {e}"
        log_activity("tool", "File write failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("read_file")
def read_file(file_path: str) -> str:
    """Read content from a file"""
    try:
        safe_file_path = safe_path(file_path)
        with open(safe_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        log_activity("tool", "File read", {"file": file_path})
        return content
    except Exception as e:
        error_msg = f"❌ Failed to read file {file_path}: {e}"
        log_activity("tool", "File read failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("list_files")
def list_files(path: str = ".") -> List[str]:
    """List files in a directory"""
    try:
        safe_dir_path = safe_path(path)
        if safe_dir_path.is_dir():
            files = [str(p.relative_to(WORKSPACE_DIR)) for p in safe_dir_path.iterdir() if p.is_file()]
            log_activity("tool", "Directory listed", {"path": path})
            return files
        elif safe_dir_path.is_file():
            log_activity("tool", "File listed", {"path": path})
            return [str(safe_dir_path.relative_to(WORKSPACE_DIR))]
        else:
            return []
    except Exception as e:
        log_activity("tool", "Directory listing failed", {"path": path, "error": str(e)})
        return [f"❌ Error listing {path}: {e}"]

@register_tool("install_package")
def install_package(package_name: str) -> str:
    """Install a Python package using pip"""
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package_name],
                                capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            log_activity("tool", "Package installed", {"package": package_name})
            return f"✅ Successfully installed {package_name}\n{result.stdout}"
        else:
            log_activity("tool", "Package installation failed", {"package": package_name, "error": result.stderr})
            return f"❌ Failed to install {package_name}\n{result.stderr}"
    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Installation of {package_name} timed out"
        log_activity("tool", "Package installation timed out", {"package": package_name})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error installing {package_name}: {e}"
        log_activity("tool", "Package installation error", {"package": package_name, "error": str(e)})
        return error_msg

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30) -> str:
    """Execute Python code in a subprocess"""
    try:
        start_time = time.time()
        # Write code to a temporary file
        temp_file = WORKSPACE_DIR / f"temp_exec_{int(time.time())}.py"
        with open(temp_file, 'w') as f:
            f.write(code)

        # Run the code
        result = subprocess.run([sys.executable, str(temp_file)],
                                capture_output=True, text=True, timeout=timeout,
                                cwd=str(WORKSPACE_DIR))

        # Clean up
        temp_file.unlink(missing_ok=True)

        execution_time = time.time() - start_time
        if result.returncode == 0:
            output = f"✅ Code executed successfully (in {execution_time:.2f}s):\n{result.stdout}"
            if result.stderr:
                output += f"\n⚠️ Stderr:\n{result.stderr}"
            log_activity("tool", "Python code executed", {"execution_time": execution_time})
        else:
            output = f"❌ Code execution failed (in {execution_time:.2f}s):\n{result.stderr}"
            if result.stdout:
                output += f"\n_stdout:\n{result.stdout}"
            log_activity("tool", "Python code execution failed", {"execution_time": execution_time, "error": result.stderr})

        return output

    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Code execution timed out after {timeout} seconds"
        log_activity("tool", "Python code timeout", {"timeout": timeout})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error executing code: {e}\n{traceback.format_exc()}"
        log_activity("tool", "Python code execution error", {"error": str(e)})
        return error_msg

# --- Register the Action Agent as a Tool ---
@register_tool("action_agent")
def action_agent(goal: str, context: Optional[str] = None) -> str:
    """
    Execute a complex task using the internal ManusAgent (Action Agent).
    This is the core reasoning and multi-step execution tool.
    """
    try:
        log_activity("tool", "Action Agent invoked", {"goal": goal})
        result = agent.run_task(goal, context)
        log_activity("tool", "Action Agent task completed", {"goal": goal})
        return result
    except Exception as e:
        error_msg = f"❌ Action Agent failed: {e}\n{traceback.format_exc()}"
        log_activity("tool", "Action Agent error", {"goal": goal, "error": str(e)})
        return error_msg

# --- Other Agent-related Tools (using the global 'agent' instance) ---
@register_tool("get_agent_memory")
def get_agent_memory() -> Dict[str, Any]:
    """Get the agent's memory"""
    return {
        "memory": agent.memory,
        "session_id": agent.session_id,
        "memory_count": len(agent.memory)
    }

@register_tool("clear_agent_memory")
def clear_agent_memory() -> str:
    """Clear the agent's memory"""
    old_count = len(agent.memory)
    agent.memory.clear()
    log_activity("tool", "Agent memory cleared", {"old_count": old_count})
    return f"🧹 Cleared {old_count} memory entries"


print(f"✅ Registered {len(TOOL_REGISTRY)} tools, including 'action_agent'")

print("🌐 Step 9: Setting up FastAPI application...")

# Pydantic models
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# Initialize FastAPI app
app = FastAPI(
    title="Unified Manus MCP Server",
    description="Multi-agent coding assistant and tool API",
    version="7.0.x"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Adjust for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# API Endpoints
@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "status": "Unified Manus MCP System v7.0.x online",
        "box": "2 - Agent Core",
        "docs": "/docs",
        "tool_call": "/mcp/tools/call",
        "tool_list": "/mcp/tools/list",
        "agent_status": f"Active - Session {agent.session_id}",
        "tools_available": len(TOOL_REGISTRY),
        "public_url": public_url,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "box": 2,
        "agent_memory_size": len(agent.memory),
        "tools_registered": len(TOOL_REGISTRY),
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/tools/call")
async def call_tool(tool_call: ToolCall):
    """Execute a registered tool"""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found"})

    try:
        # Execute the tool
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("tool_error", f"Tool '{tool_name}' failed", {"error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Tool execution failed: {str(e)}", "details": tb_str})

@app.get("/mcp/tools/list")
async def list_tools():
    """List all available tools"""
    tools_info = []
    for name, func in TOOL_REGISTRY.items():
        description = func.__doc__.strip() if func.__doc__ else "No description provided."
        # Get function signature
        try:
            sig = inspect.signature(func)
            parameters = {}
            for param_name, param in sig.parameters.items():
                param_info = {
                    "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any",
                    "required": param.default == inspect.Parameter.empty
                }
                if param.default != inspect.Parameter.empty:
                    param_info["default"] = param.default
                parameters[param_name] = param_info
        except Exception:
            parameters = {"error": "Could not parse parameters"}

        tools_info.append({
            "name": name,
            "description": description,
            "parameters": parameters
        })
    return {
        "tools": tools_info,
        "count": len(tools_info),
        "timestamp": datetime.now().isoformat()
    }

# --- Endpoint specifically for the Action Agent ---
@app.post("/mcp/agent/action")
async def run_action_agent_endpoint(request: TaskRequest):
    """Run a task through the internal ManusAgent (Action Agent)"""
    try:
        result = agent.run_task(request.task, request.context)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("agent_error", "Action Agent task failed", {"task": request.task, "error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Action Agent task failed: {str(e)}", "details": tb_str})

@app.get("/mcp/agent/memory")
async def get_agent_memory_endpoint():
    """Get the agent's memory"""
    return get_agent_memory()

@app.post("/mcp/agent/memory/clear")
async def clear_agent_memory_endpoint():
    """Clear the agent's memory"""
    return {"result": clear_agent_memory()}

@app.get("/mcp/system/info")
async def get_system_info():
    """Get system information"""
    return {
        "box": 2,
        "name": "Agent Core and Tools",
        "version": "7.0.x",
        "base_dir": str(BASE_DIR),
        "workspace_dir": str(WORKSPACE_DIR),
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "is_colab": IS_COLAB,
        "agent_session": agent.session_id,
        "tools_count": len(TOOL_REGISTRY),
        "memory_entries": len(agent.memory),
        "uptime": datetime.now().isoformat(),
        "python_version": sys.version,
        "working_directory": os.getcwd()
    }


print("🔄 Step 10: Setting up streaming and real-time features...")
# Real-time queues for streaming (placeholder for potential future use)
thought_queue = queue.Queue()
output_queue = queue.Queue()

# If you plan to add streaming endpoints, define them here
# Example (requires `sse-starlette`):
# from sse_starlette.sse import EventSourceResponse
# @app.get("/mcp/tools/stream")
# async def stream_tool_call(tool_call: ToolCall):
#     async def event_generator():
#         # ... streaming logic ...
#     return EventSourceResponse(event_generator())

print("💾 Step 11: Setting up data persistence...")

def save_box2_state():
    """Save Box 2 state for other boxes"""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "memory_count": len(agent.memory),
        "api_endpoints": [
            "/mcp/tools/call",
            "/mcp/tools/list",
            # "/mcp/tools/stream", # Add if streaming endpoint is implemented
            "/mcp/agent/action", # Specific endpoint for the agent
            "/mcp/agent/memory"
        ],
        "timestamp": datetime.now().isoformat()
    }
    config_file = BASE_DIR / "config" / "box2_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    with open(config_file, "w") as f:
        json.dump(state, f, indent=2)
    print(f"✅ Box 2 state saved to {config_file}")
    log_activity("system", "Box 2 state saved", {"path": str(config_file)})

save_box2_state()

print("🔍 Step 12: Verification and testing...")

def verify_box2_setup():
    """Verify Box 2 setup"""
    checks = {
        "Agent Initialized": agent is not None,
        "Tools Registered": len(TOOL_REGISTRY) > 0,
        "Action Agent Tool Available": "action_agent" in TOOL_REGISTRY,
        "FastAPI App": app is not None,
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
    }
    print("🔍 Box 2 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box2_setup()

if verification_passed:
    print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"🤖 Agent Session: {agent.session_id}")
    print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)} (including 'action_agent')")
    print(f"🧠 Memory Entries: {len(agent.memory)}")
    print(f"🌐 API Ready at: {public_url}")
    print("=" * 60)
    print("🔄 Ready for Box 3: Server Launch and GUI")
    log_activity("system", "Box 2 setup completed successfully", {"tools_count": len(TOOL_REGISTRY), "agent_session": agent.session_id})

    # Test the action_agent tool
    try:
        test_result = TOOL_REGISTRY["action_agent"]("Test task for verification")
        print("✅ Action Agent test completed successfully")
        log_activity("system", "Action Agent test completed successfully")
    except Exception as e:
        print(f"⚠️ Action Agent test failed: {e}")
        log_activity("system", "Action Agent test failed", {"error": str(e)})

else:
    print("❌ BOX 2 SETUP HAD ISSUES!")
    print("Please check the errors above before proceeding to Box 3")
    log_activity("system", "Box 2 setup failed", {"errors": "See output above"})

print("📤 Box 2 ready for integration with Box 3!")
print("🚀 PROCEED TO BOX 3 to launch the complete system!")

# Make the app and agent available for Box 3 to use
__all__ = ['app', 'agent', 'TOOL_REGISTRY']


🔧 BOX 2: Initializing Agent Core and Tools (Updated for Revised Box 1 & Callable Fix)...
📥 Step 1: Loading Box 1 configuration (Updated Path)...
✅ Box 1 configuration loaded successfully
📁 Base Directory: /content/drive/MyDrive/UnifiedManusSystem
🌍 Public URL: http://localhost:8000
📦 Step 2: Importing required modules...
🔄 nest_asyncio applied
✅ PyYAML available (as installed by updated Box 1)
✅ All Box 2 modules imported successfully
📝 Step 3: Setting up logging...
[LOG] 2025-07-27T14:28:42.450091 [system] Box 2 initialization started
🛡️ Step 4: Setting up safety and path validation...
🎭 Step 5: Setting up Agent Role system...
📁 Step 6: Setting up File System Manager...
🤖 Step 7: Setting up Core Manus Agent (Action Agent)...
[LOG] 2025-07-27T14:28:42.452281 [system] Manus Agent (Action Agent) initialized
✅ ManusAgent (Action Agent) system ready
🔧 Step 8: Registering all core tools (including Action Agent)...
🔧 Registered tool: write_file
[LOG] 2025-07-27T14:28:42.453407 [system] Tool 

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 3: Server Launch and GUI - v7.0.x (Updated for Revised Box 1 & Box 2 with Action Agent)          ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - FastAPI server launch with Uvicorn (Integrates Box 2 or runs proxy)                                   ║
# ║ - Multi-interface support: Jupyter, Gradio                                                          ║
# ║ - Plugin manifests for AI integration (Claude, OpenAI)                                                  ║
# ║ - System integration and monitoring                                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 3: Initializing Server Launch and GUI Systems (Updated)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📥 Step 1: Loading configurations from previous boxes (Updated Paths)...")

# --- Load Box 1 and Box 2 configurations ---
try:
    # Standardized config directory path from updated Box 1
    config_dir = Path("/content/drive/MyDrive/UnifiedManusSystem/config")
    # Fallback for local runs
    if not config_dir.exists():
        config_dir = Path("./UnifiedManusSystem/config")

    if not config_dir.exists():
        raise FileNotFoundError("Config directory not found")

    # Load Box 1 config
    box1_config_file = config_dir / "box1_exports.json"
    if box1_config_file.exists():
        with open(box1_config_file, "r") as f:
            box1_config = json.load(f)
        print("✅ Box 1 configuration loaded")
    else:
        raise FileNotFoundError("Box 1 config not found")

    # Load Box 2 config
    box2_config_file = config_dir / "box2_exports.json"
    if box2_config_file.exists():
        with open(box2_config_file, "r") as f:
            box2_config = json.load(f)
        print("✅ Box 2 configuration loaded")
    else:
        print("⚠️ Box 2 config not found, will use defaults/fallbacks")
        box2_config = {"tools_registered": [], "agent_session": "unknown", "api_endpoints": []}

    # Extract configuration
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]

    tools_available = box2_config.get("tools_registered", [])
    agent_session = box2_config.get("agent_session", "unknown")
    box2_api_endpoints = box2_config.get("api_endpoints", [])

    print(f"📁 Base Directory: {BASE_DIR}")
    print(f"🌍 Public URL: {public_url}")
    print(f"🤖 Agent Session: {agent_session}")
    print(f"🛠️ Tools Available: {len(tools_available)}")

except Exception as e:
    print(f"❌ Error loading configurations: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True
    tools_available = []
    agent_session = "unknown"
    box2_api_endpoints = []

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules for Box 3...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core web framework
import uvicorn
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse
import requests # For proxying calls to Box 2 if needed

# GUI frameworks
GRADIO_AVAILABLE = False
JUPYTER_AVAILABLE = False
try:
    import gradio as gr
    GRADIO_AVAILABLE = True
    print("✅ Gradio available")
except ImportError:
    print("⚠️ Gradio not available")

try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️ Jupyter widgets not available")

print("✅ All Box 3 modules imported successfully")

print("🔄 Step 3: Re-establishing connection to Box 2 components...")

# --- Determine if Box 2 is running ---
# Check if Box 2's app and agent are available in the current session (e.g., if this is a unified script)
box2_running_in_session = 'app' in globals() and 'agent' in globals() and 'TOOL_REGISTRY' in globals()
box2_api_accessible = False
box2_api_url = "http://localhost:8000" # Default assumption for internal calls

if box2_running_in_session:
    print("✅ Box 2 components found in current session (unified script mode)")
    box2_running = True
    # Use the in-session components
    try:
        from __main__ import app as box2_app, agent as box2_agent, TOOL_REGISTRY as box2_tool_registry
        print("🔗 Linked to in-session Box 2 components")
    except ImportError:
        print("⚠️ Could not import Box 2 components directly, using global references if available")
        # They are already in globals if the check passed
        box2_app = globals().get('app')
        box2_agent = globals().get('agent')
        box2_tool_registry = globals().get('TOOL_REGISTRY')
else:
    # Check if Box 2 server is running externally
    try:
        response = requests.get(f"{box2_api_url}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Box 2 server is accessible externally")
            box2_running = True
            box2_api_accessible = True
        else:
            print(f"⚠️ Box 2 server health check failed (Status: {response.status_code})")
            box2_running = False
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not connect to Box 2 server at {box2_api_url}: {e}")
        box2_running = False

if not box2_running:
    print("⚠️ Box 2 components not found/runnable. GUI will use fallback methods or fail gracefully.")


print("📄 Step 4: Creating plugin manifest files...")

def create_plugin_manifests():
    """Create plugin manifest files for AI integration"""
    print("📄 Creating plugin manifest files...")
    site_dir = BASE_DIR / "site"
    site_dir.mkdir(exist_ok=True)
    static_dir = site_dir / "static"
    static_dir.mkdir(exist_ok=True)

    # AI Plugin manifest (OpenAI/Claude compatible structure)
    ai_plugin_manifest = {
        "schema_version": "v1",
        "name_for_human": "Unified Manus MCP",
        "name_for_model": "unified_manus",
        "description_for_human": "Multi-agent coding assistant with comprehensive tool support",
        "description_for_model": "A unified agent system with file operations, Python execution, package management, and multi-role thinking capabilities via the 'action_agent' tool.",
        "auth": {"type": "none"},
        "api": {"type": "openapi", "url": f"{public_url}/openapi.json"}, # Points to Box 2's OpenAPI spec
        "logo_url": f"{public_url}/site/static/logo.png", # Placeholder
        "contact_email": "support@example.com",
        "legal_info_url": f"{public_url}/site/legal.html" # Placeholder
    }
    try:
        with open(site_dir / "ai-plugin.json", "w") as f:
            json.dump(ai_plugin_manifest, f, indent=2)
        print("✅ AI Plugin manifest created")
    except Exception as e:
        print(f"❌ Failed to create AI Plugin manifest: {e}")

    # Claude-compatible manifest (YAML)
    claude_manifest = {
        "name": "unified_manus",
        "description": "Multi-agent coding assistant with comprehensive tool support, including an 'action_agent' for complex tasks.",
        "version": "7.0.x",
        "endpoints": {
            "tool_call": f"{public_url}/mcp/tools/call",
            "tool_list": f"{public_url}/mcp/tools/list",
            # "stream": f"{public_url}/mcp/tools/stream" # Add if streaming is implemented
        },
        "capabilities": ["file_operations", "python_execution", "package_management", "agent_thinking", "memory_management"]
    }
    try:
        import yaml # Should be available as installed by updated Box 1
        with open(site_dir / "claude.yaml", "w") as f:
            yaml.dump(claude_manifest, f, default_flow_style=False)
        print("✅ Claude manifest (YAML) created")
    except ImportError:
        print("⚠️ PyYAML not found, skipping Claude manifest YAML creation.")
    except Exception as e:
        print(f"❌ Failed to create Claude manifest: {e}")

    # Simple index.html for /site/
    index_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Unified Manus System</title>
</head>
<body>
    <h1>🤖 Unified Manus MCP System v7.0.x</h1>
    <p>Multi-Agent Coding Assistant</p>
    <ul>
        <li><a href="/docs">API Documentation (FastAPI)</a></li>
        <li><a href="/redoc">API Documentation (ReDoc)</a></li>
        <li>Agent Session: {agent_session}</li>
        <li>Tools Available: {len(tools_available)}</li>
        <li>Public URL: <a href="{public_url}">{public_url}</a></li>
    </ul>
</body>
</html>
"""
    try:
        with open(site_dir / "index.html", "w") as f:
            f.write(index_content)
        print("✅ Basic site index.html created")
    except Exception as e:
        print(f"❌ Failed to create site index.html: {e}")

create_plugin_manifests()


print("🎨 Step 5: Setting up Gradio and Jupyter interfaces...")

# --- Gradio Interface ---
def setup_gradio_interface():
    """Setup Gradio interface"""
    if not GRADIO_AVAILABLE:
        print("⚠️ Gradio not available, skipping Gradio interface")
        return None

    print("🎨 Setting up Gradio interface...")

    def run_agent_task(task, context):
        """Run a task through the agent"""
        try:
            if box2_running_in_session:
                # Direct call to in-session agent
                result = box2_agent.run_task(task, context)
                return result
            elif box2_api_accessible:
                 # Call Box 2 API
                 payload = {"task": task, "context": context}
                 response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120) # Longer timeout for agent tasks
                 if response.status_code == 200:
                     return response.json().get("result", "No result returned")
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Agent Core) is not available."
        except Exception as e:
            return f"❌ Failed to run agent task: {str(e)}"

    def list_tools():
        """List available tools"""
        try:
             if box2_running_in_session:
                 # Direct call
                 from __main__ import TOOL_REGISTRY # Or use global if imported
                 tools_text = f"🛠️ Available Tools ({len(TOOL_REGISTRY)}):\n"
                 for name, func in TOOL_REGISTRY.items():
                     desc = func.__doc__.split('\n')[0] if func.__doc__ else "No description"
                     tools_text += f"• {name}: {desc}\n"
                 return tools_text
             elif box2_api_accessible:
                 # Call Box 2 API
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", timeout=10)
                 if response.status_code == 200:
                     result = response.json()
                     tools_text = f"🛠️ Available Tools ({result['count']}):\n"
                     for tool in result['tools']:
                         tools_text += f"• {tool['name']}: {tool['description']}\n"
                     return tools_text
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
             else:
                  return "❌ Box 2 (Tool Registry) is not available."
        except Exception as e:
            return f"❌ Failed to fetch tools: {str(e)}"

    def execute_tool(tool_name, tool_input):
        """Execute a specific tool"""
        try:
            # Parse tool_input as JSON if it's a string
            if isinstance(tool_input, str) and tool_input.strip():
                try:
                    input_data = json.loads(tool_input)
                except json.JSONDecodeError:
                    # If not valid JSON, treat as simple string content
                    input_data = {"content": tool_input}
            else:
                input_data = {}

            if box2_running_in_session:
                # Direct call
                from __main__ import TOOL_REGISTRY # Or use global
                if tool_name in TOOL_REGISTRY:
                    result = TOOL_REGISTRY[tool_name](**input_data)
                    # Pretty print if result is a dict
                    if isinstance(result, dict):
                        return json.dumps(result, indent=2)
                    return str(result)
                else:
                    return f"❌ Tool '{tool_name}' not found."
            elif box2_api_accessible:
                # Call Box 2 API
                payload = {"tool_name": tool_name, "tool_input": input_data}
                response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                if response.status_code == 200:
                    res_data = response.json()
                    # Pretty print if result is a dict
                    result_content = res_data.get("result", "No result field")
                    if isinstance(result_content, dict):
                        return json.dumps(result_content, indent=2)
                    return str(result_content)
                else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Tool Execution) is not available."
        except Exception as e:
            return f"❌ Failed to execute tool: {str(e)}"

    with gr.Blocks(title="Unified Manus MCP System", theme=gr.themes.Default()) as demo:
        gr.Markdown("# 🤖 Unified Manus MCP System v7.0.x")
        gr.Markdown("Multi-Agent Coding Assistant with Comprehensive Tool Support")

        with gr.Tab("Agent Tasks"):
            with gr.Row():
                with gr.Column():
                    task_input = gr.Textbox(label="Task", placeholder="Enter a task for the agent...")
                    context_input = gr.Textbox(label="Context (optional)", placeholder="Additional context...", lines=3)
                    run_btn = gr.Button("Run Task with Action Agent", variant="primary")
                with gr.Column():
                    task_output = gr.Textbox(label="Agent Output", lines=15, max_lines=20, show_copy_button=True)
            run_btn.click(fn=run_agent_task, inputs=[task_input, context_input], outputs=task_output)

        with gr.Tab("Tool Execution"):
            with gr.Row():
                with gr.Column():
                    tool_name_input = gr.Dropdown(label="Tool Name", choices=tools_available if tools_available else [], allow_custom_value=True)
                    tool_input_input = gr.Textbox(label="Tool Input (JSON)", placeholder='{"file_path": "test.txt", "content": "Hello"}', lines=5)
                    exec_tool_btn = gr.Button("Execute Tool", variant="secondary")
                with gr.Column():
                    tool_output = gr.Textbox(label="Tool Output", lines=10, max_lines=15, show_copy_button=True)
            exec_tool_btn.click(fn=execute_tool, inputs=[tool_name_input, tool_input_input], outputs=tool_output)

        with gr.Tab("System Info"):
            with gr.Row():
                tools_btn = gr.Button("Refresh Tool List")
                tools_output = gr.Textbox(label="Available Tools", lines=15, max_lines=20)
                tools_btn.click(fn=list_tools, outputs=tools_output)

            info_text = (
                f"**System Information:**\n"
                f"- Public API URL: {public_url}\n"
                f"- Dashboard URL: {dashboard_url}\n"
                f"- Agent Session: {agent_session}\n"
                f"- Base Directory: {BASE_DIR}\n"
                f"- Workspace Directory: {WORKSPACE_DIR}\n"
                f"- Tools Available: {len(tools_available)}\n"
                f"- Box 2 Status: {'Integrated/Running' if box2_running else 'Not Accessible'}"
            )
            gr.Markdown(info_text)

    return demo

# --- Jupyter Interface ---
def setup_jupyter_interface():
    """Setup Jupyter widget interface"""
    if not JUPYTER_AVAILABLE:
        print("⚠️ Jupyter widgets not available, skipping Jupyter interface")
        return None

    print("📓 Setting up Jupyter interface...")

    # --- Jupyter UI Logic ---
    def create_jupyter_ui():
        clear_output(wait=True)

        # --- UI Elements ---
        task_input = widgets.Text(
            value='',
            placeholder='Enter a task for the agent (e.g., Write a Python script to calculate Fibonacci numbers)',
            description='Task:',
            disabled=False,
            layout=widgets.Layout(width='100%')
        )

        context_input = widgets.Textarea(
            value='',
            placeholder='Optional context for the task',
            description='Context:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        run_button = widgets.Button(
            description="Run Task with Action Agent",
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Execute the task using the internal ManusAgent',
            icon='play' # (FontAwesome names without the `fa-` prefix)
        )

        tool_name_dropdown = widgets.Dropdown(
            options=tools_available if tools_available else ['No tools loaded'],
            value=tools_available[0] if tools_available else 'No tools loaded',
            description='Tool:',
            disabled=not tools_available,
        )

        tool_input_textarea = widgets.Textarea(
            value='{}', # Default empty JSON object
            placeholder='Enter tool input as JSON (e.g., {"file_path": "test.txt", "content": "Hello"})',
            description='Input (JSON):',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        tool_run_button = widgets.Button(
            description="Execute Tool",
            button_style='info',
            tooltip='Run the selected tool with the provided input',
            icon='cogs'
        )

        clear_button = widgets.Button(
            description="Clear Output",
            button_style='warning',
            tooltip='Clear the output areas below',
            icon='eraser'
        )

        output_area = widgets.Output(
            layout=widgets.Layout(height='400px', border='1px solid black', overflow='auto', padding='10px')
        )

        # --- Event Handlers ---
        def on_run_task(b):
            task = task_input.value
            context = context_input.value
            if not task:
                with output_area:
                    print("⚠️ Please enter a task.")
                return

            with output_area:
                clear_output(wait=True)
                print(f"🚀 Running task: '{task}'")
                if context:
                    print(f"   Context: '{context}'")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        # Direct call to in-session agent
                        result = box2_agent.run_task(task, context)
                        print("✅ Task completed!")
                        print(result)
                    elif box2_api_accessible:
                         # Call Box 2 API
                         payload = {"task": task, "context": context}
                         response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                         if response.status_code == 200:
                             result = response.json().get("result", "No result returned")
                             print("✅ Task completed!")
                             print(result)
                         else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Agent Core) is not available.")
                except Exception as e:
                    print(f"💥 ERROR running task: {e}")
                    traceback.print_exc()

        def on_execute_tool(b):
            tool_name = tool_name_dropdown.value
            tool_input_str = tool_input_textarea.value

            if tool_name == 'No tools loaded':
                 with output_area:
                     print("⚠️ No tools are available to execute.")
                 return

            # Parse input
            try:
                if tool_input_str.strip():
                    tool_input_data = json.loads(tool_input_str)
                else:
                    tool_input_data = {}
            except json.JSONDecodeError as e:
                 with output_area:
                     print(f"❌ Invalid JSON input for tool: {e}")
                 return

            with output_area:
                clear_output(wait=True)
                print(f"🔧 Executing tool: '{tool_name}'")
                print(f"   Input: {json.dumps(tool_input_data, indent=2)}")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        # Direct call
                        if tool_name in box2_tool_registry:
                            result = box2_tool_registry[tool_name](**tool_input_data)
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result, indent=2) if isinstance(result, dict) else str(result))
                        else:
                            print(f"❌ Tool '{tool_name}' not found in registry.")
                    elif box2_api_accessible:
                        # Call Box 2 API
                        payload = {"tool_name": tool_name, "tool_input": tool_input_data}
                        response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                        if response.status_code == 200:
                            res_data = response.json()
                            result_content = res_data.get("result", "No result field")
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result_content, indent=2) if isinstance(result_content, dict) else str(result_content))
                        else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Tool Execution) is not available.")
                except Exception as e:
                    print(f"💥 ERROR executing tool: {e}")
                    traceback.print_exc()

        def on_clear(b):
            output_area.clear_output()

        # Assign event handlers
        run_button.on_click(on_run_task)
        tool_run_button.on_click(on_execute_tool)
        clear_button.on_click(on_clear)

        # --- Display Layout ---
        ui_layout = widgets.VBox([
            widgets.HTML("<h1>🤖 Unified Manus MCP System (Jupyter)</h1>"),
            widgets.HTML("<h2>Agent Task Execution</h2>"),
            task_input,
            context_input,
            run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Tool Execution</h2>"),
            tool_name_dropdown,
            tool_input_textarea,
            tool_run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Output</h2>"),
            clear_button,
            output_area
        ])

        display(ui_layout)

    return create_jupyter_ui


print("🌐 Step 6: Setting up FastAPI server (Integrated or Proxy)...")

# --- Integrated or Proxy FastAPI App for Box 3 ---
def create_integrated_or_proxy_server():
    """Create the FastAPI app for Box 3, either integrated or proxying to Box 2"""
    app = FastAPI(
        title="Unified Manus MCP Server - Box 3",
        description="Integrated server with GUI interfaces and potential proxying to Box 2",
        version="7.0.x"
    )

    # CORS middleware
    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"], # Adjust for production
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    # Static files
    site_dir = BASE_DIR / "site"
    if site_dir.exists():
        app.mount("/site", StaticFiles(directory=str(site_dir)), name="site")

    # --- Root endpoint ---
    @app.get("/")
    async def root():
        """Root endpoint"""
        index_file = site_dir / "index.html"
        if index_file.exists():
            return FileResponse(str(index_file))
        else:
            return {
                "message": "Unified Manus MCP System - Box 3",
                "version": "7.0.x",
                "box2_status": "Integrated" if box2_running_in_session else ("Proxying" if box2_api_accessible else "Unavailable"),
                "public_url": public_url,
                "agent_session": agent_session
            }

    # --- Health check ---
    @app.get("/health")
    async def health():
        """Health check endpoint"""
        return {
            "status": "healthy",
            "box": 3,
            "version": "7.0.x",
            "timestamp": datetime.now().isoformat(),
            "box2_status": "Integrated" if box2_running_in_session else ("Accessible" if box2_api_accessible else "Unavailable"),
            "agent_session": agent_session
        }

    # --- Proxy endpoints to Box 2 if it's running externally ---
    if not box2_running_in_session and box2_api_accessible:
        print("🔄 Setting up proxy endpoints to external Box 2...")

        @app.api_route("/mcp/tools/call", methods=["POST"])
        async def proxy_tool_call(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                response = requests.post(f"{box2_api_url}/mcp/tools/call", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/tools/list", methods=["GET"])
        async def proxy_tool_list(request: Request):
             try:
                 # Forward query params if any
                 params = dict(request.query_params)
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", params=params)
                 return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
             except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/action", methods=["POST"]) # Proxy the new action agent endpoint
        async def proxy_agent_action(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                response = requests.post(f"{box2_api_url}/mcp/agent/action", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/memory", methods=["GET", "POST"]) # Proxy memory endpoints
        async def proxy_agent_memory(request: Request):
            try:
                url = f"{box2_api_url}/mcp/agent/memory"
                if request.method == "POST":
                    url += "/clear" # Assuming POST to /memory is clear in Box 2
                body = await request.body() if request.method in ["POST", "PUT"] else None
                headers = dict(request.headers)
                if body:
                    response = requests.request(request.method, url, data=body, headers=headers)
                else:
                     response = requests.request(request.method, url, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        # Add more proxy routes as needed for other Box 2 endpoints

    elif box2_running_in_session:
        print("🔗 Box 2 is integrated, no proxy needed for its endpoints.")
        # Optionally, you could mount Box 2's app directly under a prefix if they are separate
        # But typically in a unified script, Box 2's routes are already part of the 'app'
    else:
        print("⚠️ Box 2 is not accessible, proxy endpoints will not function.")

    return app

# Create the integrated/proxy app
integrated_app = create_integrated_or_proxy_server()
print("✅ FastAPI server (Box 3) configured")


print("💾 Step 7: Saving Box 3 configuration...")

def save_box3_state():
    """Save Box 3 state"""
    state = {
        "interfaces_available": ["jupyter", "gradio", "api"],
        "web_interface_ready": True,
        "plugin_manifests_created": True,
        "launch_modes": ["jupyter", "gradio", "api", "all"],
        "timestamp": datetime.now().isoformat(),
        "box2_connection": {
            "in_session": box2_running_in_session,
            "api_accessible": box2_api_accessible,
            "status": "Integrated" if box2_running_in_session else ("Accessible" if box2_api_accessible else "Unavailable")
        },
        "gradio_available": GRADIO_AVAILABLE,
        "jupyter_available": JUPYTER_AVAILABLE,
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "agent_session": agent_session
    }
    config_file = BASE_DIR / "config" / "box3_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 3 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 3 state: {e}")

save_box3_state()

print("🔍 Step 8: Final verification...")

def verify_box3_setup():
    """Verify Box 3 setup"""
    checks = {
        "Web Interface": (BASE_DIR / "site" / "index.html").exists(),
        "Plugin Manifests": (BASE_DIR / "site" / "ai-plugin.json").exists(),
        "Configuration Files": (BASE_DIR / "config" / "box3_exports.json").exists(),
        "Site Directory": (BASE_DIR / "site").exists(),
        "Box 1 Config": (BASE_DIR / "config" / "box1_exports.json").exists(),
        "Box 2 Config": (BASE_DIR / "config" / "box2_exports.json").exists(),
        "Box 2 Connection": box2_running_in_session or box2_api_accessible,
        "Gradio Availability": not GRADIO_AVAILABLE or (GRADIO_AVAILABLE and setup_gradio_interface() is not None), # Basic check
        "Jupyter Availability": not JUPYTER_AVAILABLE or (JUPYTER_AVAILABLE and setup_jupyter_interface() is not None) # Basic check
    }
    print("🔍 Box 3 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box3_setup()

if verification_passed:
    print("🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    print("✅ ALL SYSTEMS VERIFIED AND READY!")
    print("🚀 LAUNCH OPTIONS:")
    print(" • Jupyter Interface: launch_system('jupyter')")
    print(" • Gradio Interface: launch_system('gradio')")
    print(" • API Server Only: launch_system('api')")
    print(" • ALL Interfaces: launch_system('all')")
    print(f"🌍 URLs:")
    print(f" • Main API: {public_url}")
    print(f" • Dashboard: {dashboard_url}")
    print(f" • Web Interface: {public_url}/site/")
    print(f" • API Docs: {public_url}/docs")
    print(f"📁 System Directories:")
    print(f" • Base: {BASE_DIR}")
    print(f" • Workspace: {WORKSPACE_DIR}")
    print(f" • Logs: {BASE_DIR / 'logs'}")
    print(f" • Site: {BASE_DIR / 'site'}")
    print(f"🔧 System Status:")
    print(f" • Environment: {'Google Colab' if IS_COLAB else 'Local'}")
    print(f" • Box 2 Status: {'Integrated' if box2_running_in_session else ('Proxying' if box2_api_accessible else 'Unavailable')}")
    print(f" • Gradio Ready: {'Yes' if GRADIO_AVAILABLE else 'No'}")
    print(f" • Jupyter Ready: {'Yes' if JUPYTER_AVAILABLE else 'No'}")
    print("=" * 80)
    print("🔄 Box 3 is ready for launch!")
else:
    print("❌ BOX 3 SETUP HAD ISSUES!")
    print("Please check the errors above.")

# --- Launch Functions ---
def launch_system(mode: str = "jupyter"):
    """
    Launch the system in different modes.
    Modes: 'jupyter', 'gradio', 'api', 'all'
    """
    global integrated_app # Use the app created earlier

    if mode == "jupyter":
        if JUPYTER_AVAILABLE:
            print("📓 Launching Jupyter interface...")
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_ui()
            else:
                print("❌ Failed to setup Jupyter interface.")
        else:
            print("❌ Jupyter is not available in this environment.")

    elif mode == "gradio":
        if GRADIO_AVAILABLE:
            print("🎨 Launching Gradio interface...")
            demo = setup_gradio_interface()
            if demo:
                demo.launch(share=True, server_name="0.0.0.0", server_port=7860) # Share for public URL
                print(f"✅ Gradio launched. Access at: http://localhost:7860 (or the public Gradio link if shared)")
            else:
                print("❌ Failed to setup Gradio interface.")
        else:
            print("❌ Gradio is not available.")

    elif mode == "api":
        print("🌐 Launching FastAPI server (Box 3)...")
        # Run the integrated/proxy app
        uvicorn.run(integrated_app, host="0.0.0.0", port=8000)
        print(f"✅ FastAPI server launched. Access at: {public_url}")

    elif mode == "all":
        print("🔄 Launching all interfaces...")
        # Start API server in a thread
        def run_api():
             uvicorn.run(integrated_app, host="0.0.0.0", port=8000)

        api_thread = threading.Thread(target=run_api)
        api_thread.daemon = True
        api_thread.start()
        print(f"✅ API Server started in background on port 8000")

        time.sleep(2) # Give API a moment to start

        # Launch Jupyter
        if JUPYTER_AVAILABLE:
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                # Run Jupyter UI in a separate thread to avoid blocking
                jupyter_thread = threading.Thread(target=jupyter_ui)
                jupyter_thread.start()
                print("✅ Jupyter interface launched")
            else:
                 print("⚠️ Jupyter interface setup failed")

        # Launch Gradio
        if GRADIO_AVAILABLE:
            demo = setup_gradio_interface()
            if demo:
                # Run Gradio in a separate thread (Gradio handles this internally usually, but launching in thread is safer)
                def run_gradio():
                    demo.launch(share=True, server_name="0.0.0.0", server_port=7860, prevent_thread_lock=True)
                    demo.block_thread() # This keeps Gradio running

                gradio_thread = threading.Thread(target=run_gradio)
                gradio_thread.daemon = True # Allow main program to exit
                gradio_thread.start()
                print("✅ Gradio interface launched")
                time.sleep(3) # Give Gradio time to print its URL
            else:
                 print("⚠️ Gradio interface setup failed")

        print("🎉 All requested interfaces started!")
        print(f"🌍 API: {public_url}")
        print(f"📊 Dashboard: {dashboard_url} (if applicable)")
        if GRADIO_AVAILABLE:
            print(f"🎨 Gradio: Check output above or http://localhost:7860")
        print("ℹ️ Main thread will now idle. Stop with KeyboardInterrupt (Ctrl+C).")

        # Keep main thread alive
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("🛑 Shutting down all services...")
            # Note: Graceful shutdown of threads/web servers in Jupyter can be tricky.
            # Usually, restarting the kernel is the cleanest way.

    else:
        print(f"❌ Unknown mode: {mode}")
        print("Available modes: jupyter, gradio, api, all")


# Auto-launch Jupyter interface if in a Jupyter environment and script run directly
if IS_COLAB or ('ipykernel' in sys.modules):
    print("📓 Auto-launching Jupyter interface...")
    launch_system('jupyter')

print("✅ Box 3 initialization complete!")
print("🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)")

# Export launch function
__all__ = ['launch_system']


✅ Box 3 initialization complete!
🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 4: Live Stream & History Observer GUI - v7.0.x                                                  ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - Displays static history (agent memory, recent logs) on load/refresh                                   ║
# ║ - Shows live stream of agent thoughts and tool executions                                               ║
# ║ - Integrates with existing ManusAgent and logging system                                                ║
# ║ - Jupyter widget based for notebook integration                                                         ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 4: Initializing Live Stream & History Observer GUI...")

import os
import sys
import json
import time
import threading
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

# --- STEP 1: Import Required Modules ---
print("📦 Step 1: Importing required modules...")
try:
    from IPython.display import display, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    JUPYTER_AVAILABLE = False
    print("❌ Jupyter widgets not available. This GUI requires a Jupyter environment.")

if not JUPYTER_AVAILABLE:
    print("🛑 Cannot initialize GUI without Jupyter widgets.")
    # Define a dummy class to prevent errors if accidentally used
    class ManusLiveStreamGUI:
        def __init__(self, *args, **kwargs): pass
        def display(self): print("❌ GUI not available: Jupyter widgets missing.")
    __all__ = ['ManusLiveStreamGUI']
else:

    # --- STEP 2: Locate Configuration and Essential Paths ---
    print("📥 Step 2: Locating system configuration...")
    try:
        # Standardized path from updated Box 1
        config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
        # Fallback for local runs
        if not config_file.exists():
            config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

        if config_file.exists():
            with open(config_file, "r") as f:
                box1_config = json.load(f)

            BASE_DIR = Path(box1_config["BASE_DIR"])
            WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
            LOG_FILE = Path(box1_config["LOG_FILE"])
            print("✅ Box 1 configuration loaded successfully")
        else:
            raise FileNotFoundError("Box 1 config not found. Run Box 1 first.")

    except Exception as e:
        print(f"❌ Error loading Box 1 config: {e}")
        print("🔄 Using fallback paths (may not work if system structure differs)...")
        BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
        WORKSPACE_DIR = BASE_DIR / "workspace"
        LOG_FILE = BASE_DIR / "logs" / "manus_log.json"

    # --- STEP 3: Define the GUI Class ---
    print("🎨 Step 3: Defining the Live Stream & History GUI...")
    class ManusLiveStreamGUI:
        """
        A Jupyter widget GUI to observe the Manus Agent's history and live stream.
        """
        def __init__(self, agent_instance=None, log_file_path: Optional[Path] = None):
            """
            Initializes the GUI components.
            :param agent_instance: The ManusAgent instance to observe. If None, tries to find it.
            :param log_file_path: Path to the log file. If None, uses the one from config.
            """
            self.log_file_path = log_file_path if log_file_path else LOG_FILE

            # --- Get Agent Instance ---
            if agent_instance is None:
                if 'agent' in globals():
                    self.agent = globals()['agent']
                else:
                    try:
                        from __main__ import agent as imported_agent
                        self.agent = imported_agent
                    except ImportError:
                        print("❌ Agent instance not found. GUI will not display agent memory.")
                        self.agent = None
            else:
                self.agent = agent_instance

            # --- GUI Widgets ---
            self.output_history_area = widgets.Output(
                layout=widgets.Layout(height='300px', overflow='auto', border='1px solid #ccc', padding='10px')
            )
            self.log_history_area = widgets.Output(
                layout=widgets.Layout(height='300px', overflow='auto', border='1px solid #ccc', padding='10px')
            )
            self.live_stream_area = widgets.Output(
                layout=widgets.Layout(height='400px', overflow='auto', border='2px solid #4CAF50', padding='10px', background_color='#f9f9f9')
            )
            self.refresh_button = widgets.Button(
                description="🔄 Refresh History",
                button_style='info',
                tooltip='Reload agent memory and recent logs',
                icon='sync'
            )
            self.clear_button = widgets.Button(
                description="🗑️ Clear Live Stream",
                button_style='warning',
                tooltip='Clear the live stream display',
                icon='trash'
            )

            # --- Event Handlers ---
            self.refresh_button.on_click(self._on_refresh)
            self.clear_button.on_click(self._on_clear)

            # --- Setup Initial Display ---
            self._setup_layout()
            print("✅ Live Stream & History GUI components created")

        def _setup_layout(self):
            """Sets up the overall layout of the GUI."""
            header = widgets.HTML(
                value="<h1>🤖 Manus Agent Live Stream & History Observer</h1><p>Monitor agent thoughts, actions, and logs in real-time.</p>",
                layout=widgets.Layout(padding='10px')
            )

            history_tabs = widgets.Tab([self.output_history_area, self.log_history_area])
            history_tabs.set_title(0, '🧠 Agent Memory (Thoughts & Actions)')
            history_tabs.set_title(1, '📝 Recent System Logs')

            stream_header = widgets.HTML("<h2 style='margin-top: 20px;'>📡 Live Stream</h2>")
            button_box = widgets.HBox([self.refresh_button, self.clear_button], layout=widgets.Layout(margin='10px 0'))

            self.main_layout = widgets.VBox([
                header,
                history_tabs,
                button_box,
                stream_header,
                self.live_stream_area
            ])

        def _on_refresh(self, button):
            """Handler for the Refresh button."""
            self.refresh_history()

        def _on_clear(self, button):
            """Handler for the Clear button."""
            self.live_stream_area.clear_output()

        def refresh_history(self):
            """Refreshes the static history panels."""
            self._display_agent_memory()
            self._display_recent_logs()

        def _display_agent_memory(self):
            """Displays the current agent memory in the history area."""
            with self.output_history_area:
                clear_output(wait=True)
                if self.agent is None or not hasattr(self.agent, 'memory'):
                    print("❌ Agent memory not available.")
                    return

                memory = self.agent.memory
                session_id = getattr(self.agent, 'session_id', 'Unknown')

                print(f"🧠 Agent Memory (Session: {session_id})")
                print("=" * 50)
                if not memory:
                    print("📭 Memory is empty.")
                    return

                for i, entry in enumerate(memory):
                    timestamp = entry.get("timestamp", "N/A")
                    entry_type = entry.get("type", "N/A")
                    role = entry.get("role", "N/A")
                    content = entry.get("content", "N/A")

                    print(f"\n--- Entry #{i+1} ({timestamp}) ---")
                    print(f"  Type: {entry_type}")
                    if role != "N/A":
                        print(f"  Role: {role}")
                    # Truncate very long content for display
                    if isinstance(content, str) and len(content) > 500:
                        print(f"  Content: {content[:500]}... (truncated)")
                    else:
                        print(f"  Content: {content}")
                print("=" * 50)

        def _display_recent_logs(self, num_entries: int = 30):
            """Displays recent entries from the system log file."""
            with self.log_history_area:
                clear_output(wait=True)
                if not self.log_file_path.exists():
                    print(f"❌ Log file not found at {self.log_file_path}")
                    return

                try:
                    with open(self.log_file_path, "r") as f:
                        # Log file is JSON lines or a single JSON array
                        try:
                            logs = [json.loads(line) for line in f if line.strip()]
                        except json.JSONDecodeError:
                            # Try loading as a single JSON array
                            f.seek(0)
                            logs = json.load(f)
                except Exception as e:
                    print(f"❌ Error reading log file: {e}")
                    return

                print(f"📝 Recent System Logs (Last {num_entries})")
                print("=" * 50)
                if not logs:
                    print("📭 Log file is empty.")
                    return

                entries_to_show = logs[-num_entries:]

                for i, entry in enumerate(entries_to_show):
                    idx = len(logs) - len(entries_to_show) + i + 1
                    timestamp = entry.get("timestamp", "N/A")
                    category = entry.get("category", "N/A")
                    message = entry.get("message", "N/A")
                    data = entry.get("data", {})
                    box = entry.get("box", "N/A")

                    print(f"\n--- Log Entry #{idx} ({timestamp}) [Box {box}] ---")
                    print(f"  Category: {category}")
                    print(f"  Message: {message}")
                    if data:
                        # Truncate data for display
                        data_str = json.dumps(data)
                        if len(data_str) > 300:
                            print(f"  Data: {data_str[:300]}... (truncated)")
                        else:
                            print(f"  Data: {data_str}")
                print("=" * 50)

        def log_to_stream(self, message: str, source: str = "System"):
            """
            Logs a message to the live stream area.
            This can be called by external functions or patched logging.
            """
            timestamp = datetime.now().strftime("%H:%M:%S")
            formatted_message = f"[{timestamp}] ({source}) {message}"
            with self.live_stream_area:
                # Using print inside Output widget adds a new line automatically
                print(formatted_message)

        def display(self):
            """Displays the GUI in the Jupyter notebook."""
            # Initial population of history
            self.refresh_history()
            display(self.main_layout)
            print("✅ Live Stream & History GUI displayed. Use `gui.log_to_stream('message')` to add to live stream.")

    print("✅ BOX 4: Live Stream & History Observer GUI Defined!")

    # Export the class
    __all__ = ['ManusLiveStreamGUI']

    print("\n🚀 How to Use:")
    print("1. Create an instance: `gui = ManusLiveStreamGUI()`")
    print("2. Display it: `gui.display()`")
    print("3. To add to live stream from code: `gui.log_to_stream('Your message', 'Source')`")
    print("4. Use the 'Refresh History' button to update the static panels.")
    print("5. Use the 'Clear Live Stream' button to clear the live feed.")

# --- Example Usage (Commented out, run manually in cells) ---
# print("\n--- Example: Launching GUI ---")
# # Ensure the agent has been initialized (e.g., run Box 2)
gui = ManusLiveStreamGUI() # Uses global 'agent'
gui.display()
# print("💡 Run the example lines above in separate cells after initializing the agent.")

print("\n✅ BOX 4 is ready for use!")


🔧 BOX 4: Initializing Live Stream & History Observer GUI...
📦 Step 1: Importing required modules...
✅ Jupyter widgets available
📥 Step 2: Locating system configuration...
✅ Box 1 configuration loaded successfully
🎨 Step 3: Defining the Live Stream & History GUI...
✅ BOX 4: Live Stream & History Observer GUI Defined!

🚀 How to Use:
1. Create an instance: `gui = ManusLiveStreamGUI()`
2. Display it: `gui.display()`
3. To add to live stream from code: `gui.log_to_stream('Your message', 'Source')`
4. Use the 'Refresh History' button to update the static panels.
5. Use the 'Clear Live Stream' button to clear the live feed.
✅ Live Stream & History GUI components created


✅ Live Stream & History GUI displayed. Use `gui.log_to_stream('message')` to add to live stream.

✅ BOX 4 is ready for use!


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 2: Agent Core and Tools with Ollama Integration - v7.0.x                                        ║
# ║ Integrates Ollama-powered ManusAgent with FastAPI tool endpoints                                        ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import psutil
import requests
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable

print("📥 Step 1: Loading Box 1 configuration (Updated Path)...")

# --- Load configuration from Box 1 ---
try:
    # Standardized path from updated Box 1
    config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
    # Fallback for local runs
    if not config_file.exists():
         config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

    if config_file.exists():
        with open(config_file, "r") as f:
            box1_config = json.load(f)

        BASE_DIR = Path(box1_config["BASE_DIR"])
        WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
        LOG_FILE = Path(box1_config["LOG_FILE"])
        public_url = box1_config["public_url"]
        dashboard_url = box1_config["dashboard_url"]
        IS_COLAB = box1_config["IS_COLAB"]
        print("✅ Box 1 configuration loaded successfully")
        print(f"📁 Base Directory: {BASE_DIR}")
        print(f"🌍 Public URL: {public_url}")
    else:
        raise FileNotFoundError("Box 1 config not found at expected location")

except Exception as e:
    print(f"❌ Failed to load Box 1 config: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True # Adjust based on environment

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core FastAPI imports
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from pydantic import BaseModel
import requests # For proxying calls if needed

print("✅ All Box 2 modules imported successfully")

print("📝 Step 3: Setting up logging...")

def log_activity(category: str, message: str, data: Optional[Dict[str, Any]] = None):
    """Log activity to the JSON file."""
    try:
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "category": category,
            "message": message,
            "data": data or {},
            "box": "2"
        }

        # Read existing logs
        logs = []
        if LOG_FILE.exists():
            try:
                with open(LOG_FILE, "r") as f:
                    content = f.read()
                    if content.strip():
                         logs = json.loads(content)
            except json.JSONDecodeError:
                print("⚠️ Log file corrupted, starting fresh.")
                logs = []
        else:
            # Create parent directories if needed
            LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
            # Create an empty log file
            LOG_FILE.write_text("[]")

        logs.append(log_entry)

        # Keep last 1000 entries
        if len(logs) > 1000:
            logs = logs[-1000:]

        with open(LOG_FILE, "w") as f:
            json.dump(logs, f, indent=2)

        print(f"[LOG] {timestamp} [{category}] {message}")
    except Exception as e:
        print(f"❌ Logging failed: {e}")

log_activity("system", "Box 2 initialization started")

print("🛡️ Step 4: Setting up safety and path validation...")

def safe_path(file_path: str) -> Path:
    """Ensure file operations stay within safe directory"""
    if not file_path:
        raise ValueError("File path cannot be empty")

    base_path = WORKSPACE_DIR.resolve()
    full_path = (base_path / file_path).resolve()

    try:
        full_path.relative_to(base_path) # Raises ValueError if not relative
        return full_path
    except ValueError:
        raise PermissionError(f"Access denied: {file_path} is outside the workspace")

print("🦙 Step 5: Setting up Ollama Integration...")

# --- Ollama Configuration and Setup ---
OLLAMA_PORT = 11434
OLLAMA_PID_FILE = BASE_DIR / ".ollama_pid"
DEFAULT_MODEL = "llama3:8b"
MODEL_NAME = DEFAULT_MODEL # Can be overridden

def is_ollama_installed() -> bool:
    """Check if Ollama binary is installed"""
    return os.path.exists("/usr/local/bin/ollama") or os.path.exists("/usr/bin/ollama")

def is_ollama_running() -> bool:
    """Check if Ollama service is already running"""
    # Method 1: Check via API
    try:
        response = requests.get(f"http://localhost:{OLLAMA_PORT}", timeout=5)
        if response.status_code == 200:
            print("✅ Ollama API is accessible.")
            return True
    except requests.exceptions.RequestException:
        pass

    # Method 2: Check via PID file
    if OLLAMA_PID_FILE.exists():
        try:
            with open(OLLAMA_PID_FILE, 'r') as f:
                pid = int(f.read().strip())
            proc = psutil.Process(pid)
            if 'ollama' in proc.name().lower():
                print(f"✅ Ollama running from PID file (PID: {pid}).")
                return True
        except (ValueError, psutil.NoSuchProcess, psutil.AccessDenied, FileNotFoundError):
            pass
        # Clean up stale PID file
        OLLAMA_PID_FILE.unlink(missing_ok=True)

    # Method 3: Check via psutil for any ollama process
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            # Check if the process name contains 'ollama'
            if 'ollama' in proc.info['name'].lower():
                print(f"✅ Found Ollama process (PID: {proc.info['pid']}).")
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

    return False

def setup_ollama(model_name: str = DEFAULT_MODEL):
    """Setup Ollama: install, start server, and pull model."""
    global MODEL_NAME
    MODEL_NAME = model_name
    print(f"🔧 Setting up Ollama for model: {MODEL_NAME}")

    # 1. Install Ollama (if not present)
    if not is_ollama_installed():
        print("🔽 Installing Ollama...")
        try:
            install_script_url = "https://ollama.com/install.sh"
            result = subprocess.run(f"curl -fsSL {install_script_url} | sh", shell=True, capture_output=True, text=True, check=True)
            log_activity("setup", "Ollama installation", {"stdout": result.stdout, "stderr": result.stderr})
            print("✅ Ollama installation complete.")
        except subprocess.CalledProcessError as e:
            print(f"❌ Ollama installation failed: {e.stderr}")
            log_activity("setup", "Ollama installation failed", {"error": e.stderr})
            return
    else:
        print("✅ Ollama is already installed.")

    # 2. Start Ollama Serve
    if not is_ollama_running():
        print("🚀 Starting Ollama serve process...")
        try:
            process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # Save PID for future reference
            with open(OLLAMA_PID_FILE, 'w') as f:
                f.write(str(process.pid))
            time.sleep(5) # Give Ollama time to start
            if is_ollama_running():
                print("✅ Ollama serve started.")
            else:
                print("⚠️ Ollama serve process started, but API not immediately responsive.")
        except Exception as e:
            print(f"❌ Failed to start Ollama serve: {e}")
            log_activity("setup", "Ollama start failed", {"error": str(e)})
            return
    else:
        print("✅ Ollama is already running.")

    # 3. Pull Model
    print(f"🔽 Checking if model '{MODEL_NAME}' is available...")
    try:
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        if MODEL_NAME in result.stdout:
            print(f"✅ Model '{MODEL_NAME}' is already available.")
        else:
            print(f"🔽 Model '{MODEL_NAME}' not found, pulling...")
            print("   ⚠️ This may take several minutes for large models. Please wait...")
            pull_result = subprocess.run(["ollama", "pull", MODEL_NAME], capture_output=True, text=True)
            log_activity("setup", f"Model {MODEL_NAME} pulled", {"stdout": pull_result.stdout, "stderr": pull_result.stderr})
            if pull_result.returncode == 0:
                print(f"✅ Model {MODEL_NAME} pulled successfully.")
            else:
                print(f"❌ Failed to pull model {MODEL_NAME}: {pull_result.stderr}")
                return # Stop if model pull fails
    except subprocess.CalledProcessError as e:
        print(f"❌ Error checking model list: {e}")
        # Try pulling anyway
        print(f"🔽 Attempting to pull model '{MODEL_NAME}'...")
        pull_result = subprocess.run(["ollama", "pull", MODEL_NAME], capture_output=True, text=True)
        log_activity("setup", f"Model {MODEL_NAME} pulled (fallback)", {"stdout": pull_result.stdout, "stderr": pull_result.stderr})


# --- Ollama Query Function ---
def query_ollama_stream(prompt: str, system_prompt: str = "", role: str = "assistant", stream_callback=None) -> str:
    """
    Query Ollama model with streaming response.
    """
    full_prompt = f"{system_prompt}\nUser: {prompt}\nAssistant:" if system_prompt else prompt
    full_response = ""
    try:
        response = requests.post(
            f"http://localhost:{OLLAMA_PORT}/api/generate",
            json={"model": MODEL_NAME, "prompt": full_prompt, "stream": True},
            stream=True,
            timeout=300 # 5 minute timeout
        )
        response.raise_for_status()
        for line in response.iter_lines():
            if line:
                chunk = json.loads(line)
                content = chunk.get("response", "")
                full_response += content
                if stream_callback:
                    # Pass content and role to the callback
                    stream_callback(content, role)
                if chunk.get("done"):
                    break
    except Exception as e:
        error_msg = f"Ollama query failed: {str(e)}"
        print(f"❌ {error_msg}")
        log_activity("ollama", "Query failed", {"error": str(e), "prompt": prompt[:100]})
        if stream_callback:
            stream_callback(f"\n❌ {error_msg}\n", "system")
        full_response = error_msg
    return full_response


print("🎭 Step 6: Setting up Ollama-powered Agent Role system...")
# --- Agent Role System (Ollama-powered) ---
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        """Process task using Ollama LLM."""
        log_activity("agent", f"Role '{self.name}' processing task", {"task": task_description})
        full_response = query_ollama_stream(task_description, self.system_prompt, self.name, stream_callback)
        log_activity("agent", f"Role '{self.name}' finished processing")
        return full_response

# Define roles using Ollama
ROLES = {
    "planner": ManusRole(
        "Planner",
        "Creates plans",
        "You are an expert software architect. Break down user requests into clear, actionable implementation steps. Respond with a numbered list of steps."
    ),
    "researcher": ManusRole(
        "Researcher",
        "Gathers information",
        "You are a research assistant. Find relevant information and summarize it clearly. Only provide the summary, not the search process itself."
    ),
    "coder": ManusRole(
        "Coder",
        "Writes code",
        "You are a skilled software engineer. Write clean, efficient, well-documented code based on the plan provided. Only output the code without explanations."
    ),
    "reviewer": ManusRole(
        "Reviewer",
        "Reviews code",
        "You are a senior engineer reviewing code. Check for correctness, efficiency, security, and adherence to best practices. Provide specific suggestions for improvement."
    ),
    "debugger": ManusRole(
        "Debugger",
        "Fixes code",
        "You are an expert debugger. Identify the root cause of the error and provide fixed code. Explain your reasoning clearly."
    ),
}

print("📁 Step 7: Setting up File System Manager...")
# --- File System Manager (from previous snippets) ---
class FileSystemManager:
    def __init__(self, base_path: Path = WORKSPACE_DIR):
        self.base_path = base_path.resolve()
        self.base_path.mkdir(parents=True, exist_ok=True)

    def safe_join(self, *paths) -> Path:
        """Join paths and ensure the result is within the base path."""
        full_path = Path(self.base_path, *paths).resolve()
        try:
            full_path.relative_to(self.base_path) # Raises ValueError if not relative
            return full_path
        except ValueError:
            raise PermissionError(f"Path traversal attempt: {full_path}")

    def read_file(self, file_path: str) -> str:
        """Read a file safely."""
        full_path = self.safe_join(file_path)
        with open(full_path, 'r', encoding='utf-8') as f:
            return f.read()

    def write_file(self, file_path: str, content: str) -> str:
        """Write content to a file safely."""
        full_path = self.safe_join(file_path)
        full_path.parent.mkdir(parents=True, exist_ok=True) # Ensure parent dirs exist
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ File written to {full_path}"

    def list_files(self, dir_path: str = ".") -> List[str]:
        """List files in a directory safely."""
        full_path = self.safe_join(dir_path)
        if full_path.is_dir():
            return [str(p.relative_to(self.base_path)) for p in full_path.iterdir() if p.is_file()]
        else:
            return [str(full_path.relative_to(self.base_path))] if full_path.is_file() else []

print("🤖 Step 8: Setting up Ollama-powered Core Manus Agent...")
# --- Core Agent Class (Ollama-powered) ---
class ManusAgent:
    def __init__(self):
        self.roles = ROLES
        self.fs = FileSystemManager()
        self.memory: List[Dict[str, Any]] = []
        self.session_id = f"session_{int(time.time())}"
        log_activity("system", "Ollama-powered Manus Agent initialized")

    def solve_task(self, task_description: str, stream_callback=None):
        """Main task solving logic using Ollama-powered roles."""
        log_activity("agent", "Task started", {"task": task_description})
        self.memory.append({"type": "task_start", "content": task_description, "timestamp": datetime.now().isoformat()})
        if stream_callback:
            stream_callback(f"🧠 Starting task: {task_description}\n", "system")

        try:
            # Planner Role
            if stream_callback:
                stream_callback(f"\n📝 [Planner] Generating plan...\n", "planner")
            plan_output = self.roles["planner"].process(f"Create a plan for: {task_description}", stream_callback)
            self.memory.append({"type": "thought", "role": "planner", "content": plan_output, "timestamp": datetime.now().isoformat()})

            # Coder Role
            if stream_callback:
                 stream_callback(f"\n💻 [Coder] Writing code based on plan...\n", "coder")
            code_task = f"Plan:\n{plan_output}\n\nTask:\n{task_description}"
            code_output = self.roles["coder"].process(code_task, stream_callback)
            self.memory.append({"type": "action", "role": "coder", "content": code_output, "timestamp": datetime.now().isoformat()})

            # Reviewer Role
            if stream_callback:
                 stream_callback(f"\n🔍 [Reviewer] Reviewing code...\n", "reviewer")
            review_task = f"Code:\n{code_output}\n\nOriginal Plan:\n{plan_output}\n\nTask:\n{task_description}"
            review_output = self.roles["reviewer"].process(review_task, stream_callback)
            self.memory.append({"type": "thought", "role": "reviewer", "content": review_output, "timestamp": datetime.now().isoformat()})

            final_result = f"✅ Task '{task_description}' completed successfully!\n\n📝 Plan:\n{plan_output}\n\n💻 Generated Code:\n```python\n{code_output}\n```\n\n🔍 Review:\n{review_output}"
            self.memory.append({"type": "task_end", "content": final_result, "timestamp": datetime.now().isoformat()})
            if stream_callback:
                 stream_callback(f"\n✅ Task completed.\n", "system")
            log_activity("agent", "Task completed", {"task": task_description})
            return {"status": "success", "plan": plan_output, "code": code_output, "review": review_output, "final_output": final_result}

        except Exception as e:
            error_msg = f"❌ Agent task failed: {str(e)}\n{traceback.format_exc()}"
            self.memory.append({"type": "task_error", "content": error_msg, "timestamp": datetime.now().isoformat()})
            if stream_callback:
                 stream_callback(f"\n{error_msg}\n", "system")
            log_activity("agent", "Task failed", {"task": task_description, "error": str(e)})
            return {"status": "error", "message": error_msg}


    def run_task(self, goal: str, context: Optional[str] = None) -> Dict[str, Any]:
        """Wrapper for solve_task, potentially adding context."""
        task_with_context = f"{goal}\nContext: {context}" if context else goal
        # For direct tool call, we don't stream to a return value easily here.
        # The streaming is handled by the endpoint or GUI calling with a callback.
        result = self.solve_task(task_with_context) # Pass stream_callback if needed from caller
        return result

# Initialize the global agent instance
# Run Ollama setup first
setup_ollama(DEFAULT_MODEL)
# Now initialize the agent
agent = ManusAgent()
print("✅ Ollama-powered ManusAgent system ready")

print("🔧 Step 9: Registering all core tools...")

# --- Tool Registry ---
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str):
    """Decorator to register tools"""
    def decorator(func: Callable):
        TOOL_REGISTRY[name] = func
        print(f"🔧 Registered tool: {name}")
        log_activity("system", f"Tool registered: {name}")
        return func
    return decorator

# --- Registering Tools ---
@register_tool("write_file")
def write_file(file_path: str, content: str) -> str:
    """Write content to a file"""
    try:
        safe_file_path = safe_path(file_path)
        safe_file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(safe_file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        log_activity("tool", "File written", {"file": file_path})
        return f"✅ Successfully wrote to {safe_file_path}"
    except Exception as e:
        error_msg = f"❌ Failed to write file {file_path}: {e}"
        log_activity("tool", "File write failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("read_file")
def read_file(file_path: str) -> str:
    """Read content from a file"""
    try:
        safe_file_path = safe_path(file_path)
        with open(safe_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        log_activity("tool", "File read", {"file": file_path})
        return content
    except Exception as e:
        error_msg = f"❌ Failed to read file {file_path}: {e}"
        log_activity("tool", "File read failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("list_files")
def list_files(path: str = ".") -> List[str]:
    """List files in a directory"""
    try:
        safe_dir_path = safe_path(path)
        if safe_dir_path.is_dir():
            files = [str(p.relative_to(WORKSPACE_DIR)) for p in safe_dir_path.iterdir() if p.is_file()]
            log_activity("tool", "Directory listed", {"path": path})
            return files
        elif safe_dir_path.is_file():
            log_activity("tool", "File listed", {"path": path})
            return [str(safe_dir_path.relative_to(WORKSPACE_DIR))]
        else:
            return []
    except Exception as e:
        log_activity("tool", "Directory listing failed", {"path": path, "error": str(e)})
        return [f"❌ Error listing {path}: {e}"]

@register_tool("install_package")
def install_package(package_name: str) -> str:
    """Install a Python package using pip"""
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package_name],
                                capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            log_activity("tool", "Package installed", {"package": package_name})
            return f"✅ Successfully installed {package_name}\n{result.stdout}"
        else:
            log_activity("tool", "Package installation failed", {"package": package_name, "error": result.stderr})
            return f"❌ Failed to install {package_name}\n{result.stderr}"
    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Installation of {package_name} timed out"
        log_activity("tool", "Package installation timed out", {"package": package_name})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error installing {package_name}: {e}"
        log_activity("tool", "Package installation error", {"package": package_name, "error": str(e)})
        return error_msg

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30) -> str:
    """Execute Python code in a subprocess"""
    try:
        start_time = time.time()
        # Write code to a temporary file
        temp_file = WORKSPACE_DIR / f"temp_exec_{int(time.time())}.py"
        with open(temp_file, 'w') as f:
            f.write(code)

        # Run the code
        result = subprocess.run([sys.executable, str(temp_file)],
                                capture_output=True, text=True, timeout=timeout,
                                cwd=str(WORKSPACE_DIR))

        # Clean up
        temp_file.unlink(missing_ok=True)

        execution_time = time.time() - start_time
        if result.returncode == 0:
            output = f"✅ Code executed successfully (in {execution_time:.2f}s):\n{result.stdout}"
            if result.stderr:
                output += f"\n⚠️ Stderr:\n{result.stderr}"
            log_activity("tool", "Python code executed", {"execution_time": execution_time})
        else:
            output = f"❌ Code execution failed (in {execution_time:.2f}s):\n{result.stderr}"
            if result.stdout:
                output += f"\n_stdout:\n{result.stdout}"
            log_activity("tool", "Python code execution failed", {"execution_time": execution_time, "error": result.stderr})

        return output

    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Code execution timed out after {timeout} seconds"
        log_activity("tool", "Python code timeout", {"timeout": timeout})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error executing code: {e}\n{traceback.format_exc()}"
        log_activity("tool", "Python code execution error", {"error": str(e)})
        return error_msg

# --- Register the Action Agent as a Tool ---
@register_tool("action_agent")
def action_agent(goal: str, context: Optional[str] = None) -> Dict[str, Any]:
    """
    Execute a complex task using the internal Ollama-powered ManusAgent.
    This is the core reasoning and multi-step execution tool.
    Returns the full result dictionary.
    """
    try:
        log_activity("tool", "Action Agent invoked", {"goal": goal})
        result = agent.run_task(goal, context)
        log_activity("tool", "Action Agent task completed", {"goal": goal, "status": result.get('status')})
        return result # Return the full dict
    except Exception as e:
        error_result = {"status": "error", "message": f"❌ Action Agent failed: {e}\n{traceback.format_exc()}"}
        log_activity("tool", "Action Agent error", {"goal": goal, "error": str(e)})
        return error_result


# --- Other Agent-related Tools ---
@register_tool("get_agent_memory")
def get_agent_memory() -> Dict[str, Any]:
    """Get the agent's memory"""
    return {
        "memory": agent.memory,
        "session_id": agent.session_id,
        "memory_count": len(agent.memory)
    }

@register_tool("clear_agent_memory")
def clear_agent_memory() -> str:
    """Clear the agent's memory"""
    old_count = len(agent.memory)
    agent.memory.clear()
    log_activity("tool", "Agent memory cleared", {"old_count": old_count})
    return f"🧹 Cleared {old_count} memory entries"


print(f"✅ Registered {len(TOOL_REGISTRY)} tools, including Ollama-powered 'action_agent'")

print("🌐 Step 10: Setting up FastAPI application...")

# --- Pydantic models ---
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# --- Initialize FastAPI app ---
app = FastAPI(
    title="Unified Manus MCP Server with Ollama",
    description="Multi-agent coding assistant powered by Ollama LLM and tool API",
    version="7.0.x"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Adjust for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- API Endpoints ---
@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "status": "Unified Manus MCP System v7.0.x online (Ollama-powered)",
        "box": "2 - Agent Core",
        "docs": "/docs",
        "tool_call": "/mcp/tools/call",
        "tool_list": "/mcp/tools/list",
        "agent_status": f"Active - Session {agent.session_id}",
        "tools_available": len(TOOL_REGISTRY),
        "ollama_model": MODEL_NAME,
        "public_url": public_url,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    ollama_ok = is_ollama_running()
    return {
        "status": "healthy" if ollama_ok else "degraded",
        "box": 2,
        "agent_memory_size": len(agent.memory),
        "tools_registered": len(TOOL_REGISTRY),
        "ollama_status": "running" if ollama_ok else "not running",
        "ollama_model": MODEL_NAME,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/tools/call")
async def call_tool(tool_call: ToolCall):
    """Execute a registered tool"""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found"})

    try:
        # Execute the tool
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("tool_error", f"Tool '{tool_name}' failed", {"error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Tool execution failed: {str(e)}", "details": tb_str})

@app.get("/mcp/tools/list")
async def list_tools():
    """List all available tools"""
    tools_info = []
    for name, func in TOOL_REGISTRY.items():
        description = func.__doc__.strip() if func.__doc__ else "No description provided."
        # Get function signature
        try:
            import inspect
            sig = inspect.signature(func)
            parameters = {}
            for param_name, param in sig.parameters.items():
                param_info = {
                    "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any",
                    "required": param.default == inspect.Parameter.empty
                }
                if param.default != inspect.Parameter.empty:
                    param_info["default"] = param.default
                parameters[param_name] = param_info
        except Exception:
            parameters = {"error": "Could not parse parameters"}

        tools_info.append({
            "name": name,
            "description": description,
            "parameters": parameters
        })
    return {
        "tools": tools_info,
        "count": len(tools_info),
        "timestamp": datetime.now().isoformat()
    }

# --- Endpoint specifically for the Action Agent ---
@app.post("/mcp/agent/action")
async def run_action_agent_endpoint(request: TaskRequest):
    """Run a task through the internal Ollama-powered ManusAgent"""
    try:
        # The agent's solve_task doesn't natively stream to HTTP yet,
        # but the result dict contains the full output.
        result = agent.run_task(request.task, request.context)
        return result # This returns the dict with status, plan, code, review, final_output
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("agent_error", "Action Agent task failed", {"task": request.task, "error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"status": "error", "message": f"Action Agent task failed: {str(e)}", "details": tb_str})


@app.get("/mcp/agent/memory")
async def get_agent_memory_endpoint():
    """Get the agent's memory"""
    return get_agent_memory()

@app.post("/mcp/agent/memory/clear")
async def clear_agent_memory_endpoint():
    """Clear the agent's memory"""
    return {"result": clear_agent_memory()}

@app.get("/mcp/system/info")
async def get_system_info():
    """Get system information"""
    return {
        "box": 2,
        "name": "Agent Core and Tools (Ollama)",
        "version": "7.0.x",
        "base_dir": str(BASE_DIR),
        "workspace_dir": str(WORKSPACE_DIR),
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "is_colab": IS_COLAB,
        "agent_session": agent.session_id,
        "tools_count": len(TOOL_REGISTRY),
        "memory_entries": len(agent.memory),
        "ollama_model": MODEL_NAME,
        "ollama_status": "running" if is_ollama_running() else "not running",
        "uptime": datetime.now().isoformat(),
        "python_version": sys.version,
        "working_directory": os.getcwd()
    }


print("💾 Step 11: Setting up data persistence...")

def save_box2_state():
    """Save Box 2 state for other boxes"""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "memory_count": len(agent.memory),
        "api_endpoints": [
            "/mcp/tools/call",
            "/mcp/tools/list",
            "/mcp/agent/action",
            "/mcp/agent/memory"
        ],
        "ollama_model": MODEL_NAME,
        "timestamp": datetime.now().isoformat()
    }
    config_file = BASE_DIR / "config" / "box2_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    with open(config_file, "w") as f:
        json.dump(state, f, indent=2)
    print(f"✅ Box 2 state saved to {config_file}")
    log_activity("system", "Box 2 state saved", {"path": str(config_file)})

save_box2_state()

print("🔍 Step 12: Verification and testing...")

def verify_box2_setup():
    """Verify Box 2 setup"""
    checks = {
        "Agent Initialized": agent is not None,
        "Tools Registered": len(TOOL_REGISTRY) > 0,
        "Action Agent Tool Available": "action_agent" in TOOL_REGISTRY,
        "FastAPI App": app is not None,
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
        "Ollama Running": is_ollama_running(),
        "Default Model Available": MODEL_NAME in (subprocess.run(["ollama", "list"], capture_output=True, text=True).stdout if is_ollama_running() else "")
    }
    print("🔍 Box 2 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box2_setup()

if verification_passed:
    print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"🤖 Agent Session: {agent.session_id}")
    print(f"🦙 Ollama Model: {MODEL_NAME}")
    print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)} (including 'action_agent')")
    print(f"🧠 Memory Entries: {len(agent.memory)}")
    print(f"🌐 API Ready at: {public_url}")
    print("=" * 60)
    print("🔄 Ready for Box 3: Server Launch and GUI")
    log_activity("system", "Box 2 setup completed successfully", {"tools_count": len(TOOL_REGISTRY), "agent_session": agent.session_id, "ollama_model": MODEL_NAME})

    # Test the action_agent tool (non-streaming)
    try:
        test_result = TOOL_REGISTRY["action_agent"]("Say hello in 5 different languages.")
        print("✅ Action Agent test completed successfully")
        print(f"   Test Result Status: {test_result.get('status', 'N/A')}")
        # print(f"   Test Result Output: {test_result.get('final_output', 'N/A')[:200]}...") # Preview
        log_activity("system", "Action Agent test completed successfully")
    except Exception as e:
        print(f"⚠️ Action Agent test failed: {e}")
        log_activity("system", "Action Agent test failed", {"error": str(e)})

else:
    print("❌ BOX 2 SETUP HAD ISSUES!")
    print("Please check the errors above before proceeding to Box 3")
    log_activity("system", "Box 2 setup failed", {"errors": "See output above"})

print("📤 Box 2 ready for integration with Box 3!")
print("🚀 PROCEED TO BOX 3 to launch the complete system!")

# Make the app and agent available for Box 3 to use
__all__ = ['app', 'agent', 'TOOL_REGISTRY']


🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration...
📥 Step 1: Loading Box 1 configuration (Updated Path)...
✅ Box 1 configuration loaded successfully
📁 Base Directory: /content/drive/MyDrive/UnifiedManusSystem
🌍 Public URL: http://localhost:8000
📦 Step 2: Importing required modules...
🔄 nest_asyncio applied
✅ All Box 2 modules imported successfully
📝 Step 3: Setting up logging...
[LOG] 2025-07-27T14:29:20.942133 [system] Box 2 initialization started
🛡️ Step 4: Setting up safety and path validation...
🦙 Step 5: Setting up Ollama Integration...
🎭 Step 6: Setting up Ollama-powered Agent Role system...
📁 Step 7: Setting up File System Manager...
🤖 Step 8: Setting up Ollama-powered Core Manus Agent...
🔧 Setting up Ollama for model: llama3:8b
🔽 Installing Ollama...
[LOG] 2025-07-27T14:29:47.859502 [setup] Ollama installation
✅ Ollama installation complete.
🚀 Starting Ollama serve process...
✅ Ollama API is accessible.
✅ Ollama serve started.
🔽 Checking if model 'llama3:8b' 

In [ ]:
# ---
print("🧪 Running a direct test of the Ollama-powered Action Agent...")

# 1. Access the global agent and tool registry (should be available after running Box 2)
try:
    # These should be available if Box 2 ran in the same notebook session
    from __main__ import agent, TOOL_REGISTRY
    print("✅ Successfully accessed agent and tools from Box 2.")
except ImportError:
    print("❌ Could not import agent/TOOL_REGISTRY. Ensure Box 2 ran successfully in this session.")
    # Stop the test if we can't access the core components
    raise

# 2. Define a test task
test_task = "Write a short Python function to calculate the factorial of a number."
print(f"📝 Test Task: {test_task}")

# 3. Run the task using the action_agent tool directly
# We will capture the structured output it returns
print("🚀 Invoking the 'action_agent' tool...")
action_agent_tool = TOOL_REGISTRY.get("action_agent")

if action_agent_tool:
    try:
        # Execute the tool. It returns a dictionary with plan, code, review, etc.
        result_dict = action_agent_tool(test_task)

        print("\n✅ Tool execution completed.")

        # 4. Inspect the structured result from the agent
        status = result_dict.get("status", "unknown")
        print(f"📊 Agent Task Status: {status}")

        if status == "success":
            print("\n--- Agent's Final Output ---")
            final_output = result_dict.get("final_output", "No final output found.")
            print(final_output)

        elif status == "error":
            print("\n❌ Agent reported an error:")
            error_message = result_dict.get("message", "No error message provided.")
            print(error_message)

    except Exception as e:
        print(f"💥 An error occurred while running the tool: {e}")
        import traceback
        traceback.print_exc()

    # 5. Inspect the Agent's Internal Memory (Thoughts)
    print("\n🧠 Inspecting Agent's Internal Memory (Thoughts & Actions):")
    if hasattr(agent, 'memory') and agent.memory:
        for i, entry in enumerate(agent.memory):
            entry_type = entry.get("type", "N/A")
            role = entry.get("role", "N/A")
            content = entry.get("content", "N/A")
            timestamp = entry.get("timestamp", "N/A")

            print(f"\n--- Memory Entry #{i+1} ({timestamp}) ---")
            print(f"  Type: {entry_type}")
            if role != "N/A":
                print(f"  Role: {role}")
            # Truncate very long content for display
            if isinstance(content, str) and len(content) > 800:
                print(f"  Content (truncated): {content[:800]}...")
            else:
                print(f"  Content: {content}")
    else:
        print("  📭 Agent memory is empty.")

else:
    print("❌ 'action_agent' tool not found in TOOL_REGISTRY.")

print("\n🏁 Test finished.")

🧪 Running a direct test of the Ollama-powered Action Agent...
✅ Successfully accessed agent and tools from Box 2.
📝 Test Task: Write a short Python function to calculate the factorial of a number.
🚀 Invoking the 'action_agent' tool...
[OLLAMA] time=2025-07-27T14:31:14.539Z level=INFO source=sched.go:788 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa gpu=GPU-41dac8a3-7f95-8e4d-7ba7-70f2913fd1ba parallel=2 available=15720382464 required="6.2 GiB"
[OLLAMA] time=2025-07-27T14:31:14.625Z level=INFO source=server.go:135 msg="system memory" total="51.0 GiB" free="49.0 GiB" free_swap="0 B"
[OLLAMA] time=2025-07-27T14:31:14.626Z level=INFO source=server.go:175 msg=offload library=cuda layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[14.6 GiB]" memory.gpu_overhead="0 B" memory.required.full="6.2 GiB" memory.required.partial="6.2 GiB" 

In [ ]:
# BOX 2 PATCHED: Agent Core and Tools with Persistent Ollama Integration
# Version: v7.0.1 (Persistent Serve Fix)

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import psutil
import requests
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable

# Load Box 1 config
config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
if not config_file.exists():
    config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

if config_file.exists():
    with open(config_file, "r") as f:
        box1_config = json.load(f)
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]
else:
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True

if BASE_DIR.exists():
    os.chdir(BASE_DIR)

try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception: pass

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel

# Log Function
def log_activity(category: str, message: str, data: Optional[Dict[str, Any]] = None):
    try:
        LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
        if not LOG_FILE.exists():
            LOG_FILE.write_text("[]")
        logs = []
        with open(LOG_FILE, "r") as f:
            content = f.read()
            if content.strip():
                logs = json.loads(content)
        logs.append({"timestamp": datetime.now().isoformat(), "category": category, "message": message, "data": data or {}, "box": "2"})
        with open(LOG_FILE, "w") as f:
            json.dump(logs[-1000:], f, indent=2)
    except: pass

# Ollama Integration
OLLAMA_PORT = 11434
OLLAMA_PID_FILE = BASE_DIR / ".ollama_pid"
DEFAULT_MODEL = "llama3:8b"
MODEL_NAME = DEFAULT_MODEL

def is_ollama_installed():
    return os.path.exists("/usr/local/bin/ollama") or os.path.exists("/usr/bin/ollama")

def is_ollama_running():
    try:
        response = requests.get(f"http://localhost:{OLLAMA_PORT}", timeout=2)
        return response.status_code == 200
    except:
        return False

def launch_ollama_background():
    def serve_loop():
        try:
            process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            with open(OLLAMA_PID_FILE, 'w') as f:
                f.write(str(process.pid))
            for line in iter(process.stdout.readline, b''):
                if not line: break
                print(f"[OLLAMA] {line.decode(errors='ignore').strip()}")
        except Exception as e:
            print(f"❌ Ollama background serve failed: {e}")
    t = threading.Thread(target=serve_loop, daemon=True)
    t.start()
    time.sleep(3)

def setup_ollama(model_name: str = DEFAULT_MODEL):
    global MODEL_NAME
    MODEL_NAME = model_name
    if not is_ollama_installed():
        subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True)
    if not is_ollama_running():
        launch_ollama_background()
        for i in range(10):
            if is_ollama_running(): break
            time.sleep(1)
    try:
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        if MODEL_NAME.lower().split(":")[0] not in result.stdout.lower():
            subprocess.run(["ollama", "pull", MODEL_NAME], check=True)
    except Exception as e:
        subprocess.run(["ollama", "pull", MODEL_NAME])

# Ollama Stream Query
def query_ollama_stream(prompt: str, system_prompt: str = "", role: str = "assistant", stream_callback=None) -> str:
    full_prompt = f"{system_prompt}\nUser: {prompt}\nAssistant:" if system_prompt else prompt
    full_response = ""
    try:
        response = requests.post(
            f"http://localhost:{OLLAMA_PORT}/api/generate",
            json={"model": MODEL_NAME, "prompt": full_prompt, "stream": True},
            stream=True, timeout=300
        )
        for line in response.iter_lines():
            if line:
                chunk = json.loads(line)
                content = chunk.get("response", "")
                full_response += content
                if stream_callback:
                    stream_callback(content, role)
    except Exception as e:
        full_response = f"❌ Ollama query failed: {e}"
    return full_response

# Roles
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        return query_ollama_stream(task_description, self.system_prompt, self.name, stream_callback)

ROLES = {
    "planner": ManusRole("Planner", "Creates plans", "Break down user requests into steps."),
    "coder": ManusRole("Coder", "Writes code", "Write clean Python code only."),
    "reviewer": ManusRole("Reviewer", "Reviews code", "Critique the code and suggest fixes."),
}

# Agent
class ManusAgent:
    def __init__(self):
        self.roles = ROLES
        self.memory = []
        self.session_id = f"session_{int(time.time())}"

    def run_task(self, goal: str, context: Optional[str] = None):
        full_task = f"{goal}\nContext: {context}" if context else goal
        plan = self.roles["planner"].process(full_task)
        code = self.roles["coder"].process(f"{plan}\n\nTask: {goal}")
        review = self.roles["reviewer"].process(f"Code:\n{code}")
        return {"status": "success", "plan": plan, "code": code, "review": review}

# Initialize agent
setup_ollama(DEFAULT_MODEL)
agent = ManusAgent()

# FastAPI app
app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

@app.get("/")
async def root():
    return {"status": "Box 2 running", "model": MODEL_NAME, "agent_session": agent.session_id}

@app.post("/mcp/agent/action")
async def run_agent(request: TaskRequest):
    return agent.run_task(request.task, request.context)

[OLLAMA] [GIN] 2025/07/27 - 14:31:53 | 200 |      27.157µs |       127.0.0.1 | GET      "/"
[OLLAMA] [GIN] 2025/07/27 - 14:31:53 | 200 |      29.513µs |       127.0.0.1 | HEAD     "/"
[OLLAMA] [GIN] 2025/07/27 - 14:31:53 | 200 |     315.076µs |       127.0.0.1 | GET      "/api/tags"


In [ ]:
!ollama list

[OLLAMA] [GIN] 2025/07/26 - 22:08:38 | 200 |      32.948µs |       127.0.0.1 | HEAD     "/"
[OLLAMA] [GIN] 2025/07/26 - 22:08:38 | 200 |     329.336µs |       127.0.0.1 | GET      "/api/tags"
NAME         ID              SIZE      MODIFIED      
llama3:8b    365c0bd3c000    4.7 GB    6 minutes ago    


In [ ]:
# BOX 2 PATCHED: Agent Core and Tools with Persistent Ollama Integration
# Version: v7.0.2 (Persistent Serve Fix + Verification)

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import psutil
import requests
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable

# Load Box 1 config
config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
if not config_file.exists():
    config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

if config_file.exists():
    with open(config_file, "r") as f:
        box1_config = json.load(f)
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]
else:
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True

if BASE_DIR.exists():
    os.chdir(BASE_DIR)

try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception: pass

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel

# Log Function
def log_activity(category: str, message: str, data: Optional[Dict[str, Any]] = None):
    try:
        LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
        if not LOG_FILE.exists():
            LOG_FILE.write_text("[]")
        logs = []
        with open(LOG_FILE, "r") as f:
            content = f.read()
            if content.strip():
                logs = json.loads(content)
        logs.append({"timestamp": datetime.now().isoformat(), "category": category, "message": message, "data": data or {}, "box": "2"})
        with open(LOG_FILE, "w") as f:
            json.dump(logs[-1000:], f, indent=2)
    except: pass

# Ollama Integration
OLLAMA_PORT = 11434
OLLAMA_PID_FILE = BASE_DIR / ".ollama_pid"
DEFAULT_MODEL = "llama3:8b"
MODEL_NAME = DEFAULT_MODEL

def is_ollama_installed():
    return os.path.exists("/usr/local/bin/ollama") or os.path.exists("/usr/bin/ollama")

def is_ollama_running():
    try:
        response = requests.get(f"http://localhost:{OLLAMA_PORT}/api/tags", timeout=2)
        return response.status_code == 200 and "models" in response.text.lower()
    except:
        return False

def launch_ollama_background():
    def serve_loop():
        try:
            process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            with open(OLLAMA_PID_FILE, 'w') as f:
                f.write(str(process.pid))
            for line in iter(process.stdout.readline, b''):
                if not line: break
                print(f"[OLLAMA] {line.decode(errors='ignore').strip()}")
        except Exception as e:
            print(f"❌ Ollama background serve failed: {e}")
    t = threading.Thread(target=serve_loop, daemon=True)
    t.start()
    time.sleep(3)

def setup_ollama(model_name: str = DEFAULT_MODEL):
    global MODEL_NAME
    MODEL_NAME = model_name
    if not is_ollama_installed():
        subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True)
    if not is_ollama_running():
        launch_ollama_background()
        for i in range(10):
            if is_ollama_running(): break
            time.sleep(1)
    try:
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        if MODEL_NAME.lower().split(":")[0] not in result.stdout.lower():
            subprocess.run(["ollama", "pull", MODEL_NAME], check=True)
    except Exception as e:
        subprocess.run(["ollama", "pull", MODEL_NAME])

# Ollama Stream Query
def query_ollama_stream(prompt: str, system_prompt: str = "", role: str = "assistant", stream_callback=None) -> str:
    full_prompt = f"{system_prompt}\nUser: {prompt}\nAssistant:" if system_prompt else prompt
    full_response = ""
    try:
        response = requests.post(
            f"http://localhost:{OLLAMA_PORT}/api/generate",
            json={"model": MODEL_NAME, "prompt": full_prompt, "stream": True},
            stream=True, timeout=300
        )
        for line in response.iter_lines():
            if line:
                chunk = json.loads(line)
                content = chunk.get("response", "")
                full_response += content
                if stream_callback:
                    stream_callback(content, role)
    except Exception as e:
        full_response = f"❌ Ollama query failed: {e}"
    return full_response

# Roles
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        return query_ollama_stream(task_description, self.system_prompt, self.name, stream_callback)

ROLES = {
    "planner": ManusRole("Planner", "Creates plans", "Break down user requests into steps."),
    "coder": ManusRole("Coder", "Writes code", "Write clean Python code only."),
    "reviewer": ManusRole("Reviewer", "Reviews code", "Critique the code and suggest fixes."),
}

# Agent
class ManusAgent:
    def __init__(self):
        self.roles = ROLES
        self.memory = []
        self.session_id = f"session_{int(time.time())}"

    def run_task(self, goal: str, context: Optional[str] = None):
        full_task = f"{goal}\nContext: {context}" if context else goal
        plan = self.roles["planner"].process(full_task)
        code = self.roles["coder"].process(f"{plan}\n\nTask: {goal}")
        review = self.roles["reviewer"].process(f"Code:\n{code}")
        return {"status": "success", "plan": plan, "code": code, "review": review, "final_output": f"Plan:\n{plan}\n\nCode:\n{code}\n\nReview:\n{review}"}

# Initialize agent
setup_ollama(DEFAULT_MODEL)
agent = ManusAgent()

# FastAPI app
app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

@app.get("/")
async def root():
    return {"status": "Box 2 running", "model": MODEL_NAME, "agent_session": agent.session_id}

@app.post("/mcp/agent/action")
async def run_agent(request: TaskRequest):
    return agent.run_task(request.task, request.context)

# Verification
TOOL_REGISTRY = {"action_agent": agent.run_task}

def verify_box2_setup():
    try:
        model_list_result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        available_models = model_list_result.stdout.lower()
        model_base = MODEL_NAME.split(":")[0].lower()
        model_ok = model_base in available_models
    except Exception:
        model_ok = False

    checks = {
        "Agent Initialized": agent is not None,
        "Action Agent Tool Available": "action_agent" in TOOL_REGISTRY,
        "FastAPI App": app is not None,
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
        "Ollama Running": is_ollama_running(),
        "Default Model Available": model_ok
    }

    print("🔍 Box 2 verification results:")
    all_passed = True
    for name, status in checks.items():
        print(f"{'✅' if status else '❌'} {name}")
        if not status:
            all_passed = False
    return all_passed

# Run verification
if verify_box2_setup():
    print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
    log_activity("system", "Box 2 setup verified successfully")
    try:
        print("🧪 Running test: Say hello in 5 different languages...")
        test = agent.run_task("Say hello in 5 different languages")
        print("✅ Test Result:\n" + test["final_output"][:500] + "\n...")
    except Exception as e:
        print(f"⚠️ Action Agent test failed: {e}")
else:
    print("❌ BOX 2 SETUP FAILED VERIFICATION.")
    log_activity("system", "Box 2 setup failed verification")

[OLLAMA] [GIN] 2025/07/27 - 14:32:01 | 200 |     516.905µs |       127.0.0.1 | GET      "/api/tags"
[OLLAMA] [GIN] 2025/07/27 - 14:32:01 | 200 |      31.016µs |       127.0.0.1 | HEAD     "/"
[OLLAMA] [GIN] 2025/07/27 - 14:32:01 | 200 |     254.951µs |       127.0.0.1 | GET      "/api/tags"
[OLLAMA] [GIN] 2025/07/27 - 14:32:01 | 200 |      20.521µs |       127.0.0.1 | HEAD     "/"
[OLLAMA] [GIN] 2025/07/27 - 14:32:01 | 200 |     246.081µs |       127.0.0.1 | GET      "/api/tags"
[OLLAMA] [GIN] 2025/07/27 - 14:32:01 | 200 |     243.422µs |       127.0.0.1 | GET      "/api/tags"
🔍 Box 2 verification results:
✅ Agent Initialized
✅ Action Agent Tool Available
✅ FastAPI App
✅ Base Directory
✅ Workspace Directory
✅ Log File Parent
✅ Ollama Running
✅ Default Model Available
🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!
🧪 Running test: Say hello in 5 different languages...
[OLLAMA] [GIN] 2025/07/27 - 14:32:11 | 200 |  9.637641387s |       127.0.0.1 | POST     "/api/generate"
[OLLAMA] [GIN] 2025/07/27 

In [ ]:
# BOX 3: Unified Jupyter GUI for Ollama-Powered ManusAgent

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime

def launch_jupyter_gui():
    clear_output(wait=True)
    print("🧠 Launching Jupyter GUI for ManusAgent...")

    task_input = widgets.Textarea(
        value='',
        placeholder='Describe your coding or planning task here...',
        description='Task:',
        layout=widgets.Layout(width='100%', height='80px')
    )

    context_input = widgets.Textarea(
        value='',
        placeholder='Optional context, constraints, or notes...',
        description='Context:',
        layout=widgets.Layout(width='100%', height='60px')
    )

    run_button = widgets.Button(
        description='Run Agent 🧠',
        button_style='success',
        layout=widgets.Layout(width='150px')
    )

    clear_mem_button = widgets.Button(
        description='🧹 Clear Memory',
        button_style='warning',
        layout=widgets.Layout(width='150px')
    )

    output_area = widgets.Output(layout={'border': '1px solid gray', 'height': '400px', 'overflow_y': 'scroll'})
    mem_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '5px'})

    def stream_callback(text, role):
        with output_area:
            print(f"[{role.upper()}] {text}", flush=True)

    def on_run_clicked(_):
        output_area.clear_output()
        goal = task_input.value.strip()
        context = context_input.value.strip()
        if not goal:
            with output_area:
                print("❌ Please enter a task.")
                return
        print(f"⚡ Running Agent at {datetime.now().strftime('%H:%M:%S')}...")
        result = agent.run_task(goal, context=context if context else None)
        with output_area:
            print("\n✅ FINAL OUTPUT:\n")
            print(result.get("final_output", "No output generated."))

    def on_clear_memory(_):
        agent.memory.clear()
        with mem_output:
            mem_output.clear_output()
            print("🧹 Agent memory cleared.")

    run_button.on_click(on_run_clicked)
    clear_mem_button.on_click(on_clear_memory)

    controls = widgets.VBox([
        widgets.HBox([run_button, clear_mem_button]),
        task_input,
        context_input
    ])

    mem_title = widgets.HTML("<b>🧠 Agent Memory Log</b>")
    mem_refresh_button = widgets.Button(description="🔄 Refresh Memory", layout=widgets.Layout(width='150px'))

    def refresh_memory(_):
        mem_output.clear_output()
        with mem_output:
            for entry in agent.memory[-10:]:
                ts = entry.get("timestamp", "")
                role = entry.get("role", entry.get("type", ""))
                content = entry.get("content", "")
                print(f"[{ts}] ({role})\n{content}\n")

    mem_refresh_button.on_click(refresh_memory)

    display(HTML("<h3 style='color:darkblue'>🤖 ManusAgent Jupyter GUI (Box 3)</h3>"))
    display(controls)
    display(HTML("<h4>📝 Agent Output</h4>"))
    display(output_area)
    display(HTML("<h4>📘 Agent Memory</h4>"))
    display(widgets.HBox([mem_title, mem_refresh_button]))
    display(mem_output)
    # --- Direct Chat Interface ---
    chat_input = widgets.Text(
        value='',
        placeholder='Ask the agent anything...',
        description='Chat:',
        layout=widgets.Layout(width='100%')
    )

    send_chat_button = widgets.Button(
        description='Send Message',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )

    chat_output = widgets.Output(layout={'border': '1px solid #aaa', 'height': '200px', 'overflow_y': 'auto'})

    def on_chat_send(_):
        message = chat_input.value.strip()
        if not message:
            return
        with chat_output:
            print(f"You: {message}")
        try:
            response = agent.run_task(goal=message, context=None)
            with chat_output:
                print(f"Agent: {response.get('final_output', '[No reply]')}")
        except Exception as e:
            with chat_output:
                print(f"Error: {str(e)}")
        chat_input.value = ''

    send_chat_button.on_click(on_chat_send)

    display(HTML("<h4>Direct Agent Chat</h4>"))
    display(widgets.HBox([chat_input, send_chat_button]))
    display(chat_output)
    print("✅ GUI Ready. Enter a task and click 'Run Agent 🧠'.")

# 🚀 Launch the GUI
launch_jupyter_gui()

IndentationError: unexpected indent (ipython-input-31-760952179.py, line 150)

In [ ]:
# ╔════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 3: Unified Jupyter GUI – Agent Writes, Executes, and Debugs Python Tasks       ║
# ╚════════════════════════════════════════════════════════════════════════════════════════╝

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import subprocess
import os
import traceback

# ✅ Launch the GUI
def launch_jupyter_gui():
    clear_output(wait=True)
    print("🧠 Launching Jupyter GUI for ManusAgent...")

    task_input = widgets.Textarea(
        value='',
        placeholder='Describe your coding or planning task here...',
        description='Task:',
        layout=widgets.Layout(width='100%', height='80px')
    )

    context_input = widgets.Textarea(
        value='',
        placeholder='Optional context, constraints, or notes...',
        description='Context:',
        layout=widgets.Layout(width='100%', height='60px')
    )

    run_button = widgets.Button(
        description='Run Agent 🧠',
        button_style='success',
        layout=widgets.Layout(width='150px')
    )

    clear_mem_button = widgets.Button(
        description='🧹 Clear Memory',
        button_style='warning',
        layout=widgets.Layout(width='150px')
    )

    output_area = widgets.Output(layout={'border': '1px solid gray', 'height': '400px', 'overflow_y': 'scroll'})
    mem_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '5px'})

    def stream_callback(text, role):
        with output_area:
            print(f"[{role.upper()}] {text}", flush=True)

    def on_run_clicked(_):
        output_area.clear_output()
        goal = task_input.value.strip()
        context = context_input.value.strip()
        if not goal:
            with output_area:
                print("❌ Please enter a task.")
                return

        with output_area:
            print(f"⚡ Running Agent at {datetime.now().strftime('%H:%M:%S')}...")
            print(f"🎯 Goal: {goal}")
            if context: print(f"🧩 Context: {context}")

        result = agent.run_task(
            goal=f"Write a Python script to do the following: {goal}",
            context=context if context else None
        )

        code = result.get("final_output", "")
        with output_area:
            print("\n📝 Agent-Generated Code:\n")
            print(code)

        try:
            file_path = "agent_task.py"
            with open(file_path, "w") as f:
                f.write(code)

            with output_area:
                print(f"\n💾 Code written to {file_path}")
                print(f"\n▶️ Executing {file_path}...\n")

            result = subprocess.run(["python3", file_path], capture_output=True, text=True, timeout=10)
            stdout, stderr, returncode = result.stdout, result.stderr, result.returncode

            with output_area:
                print("📤 STDOUT:\n", stdout.strip())
                if stderr.strip():
                    print("\n❌ STDERR:\n", stderr.strip())
                print("\n📟 Return Code:", returncode)

            debug_output = agent.run_task(
                goal="Analyze this output from a Python script and identify any bugs or confirmations of success.",
                context=f"STDOUT:\n{stdout}\n\nSTDERR:\n{stderr}\n\nReturn Code: {returncode}"
            )

            with output_area:
                print("\n🧠 Debugger Analysis:\n", debug_output.get("final_output", "[No output]"))

        except Exception as e:
            tb = traceback.format_exc()
            with output_area:
                print("💥 Exception while running or analyzing script:\n", tb)

    def on_clear_memory(_):
        agent.memory.clear()
        with mem_output:
            mem_output.clear_output()
            print("🧹 Agent memory cleared.")

    run_button.on_click(on_run_clicked)
    clear_mem_button.on_click(on_clear_memory)

    controls = widgets.VBox([
        widgets.HBox([run_button, clear_mem_button]),
        task_input,
        context_input
    ])

    mem_title = widgets.HTML("<b>🧠 Agent Memory Log</b>")
    mem_refresh_button = widgets.Button(description="🔄 Refresh Memory", layout=widgets.Layout(width='150px'))

    def refresh_memory(_):
        mem_output.clear_output()
        with mem_output:
            for entry in agent.memory[-10:]:
                ts = entry.get("timestamp", "")
                role = entry.get("role", entry.get("type", ""))
                content = entry.get("content", "")
                print(f"[{ts}] ({role})\n{content}\n")

    mem_refresh_button.on_click(refresh_memory)

    display(HTML("<h3 style='color:darkblue'>🤖 ManusAgent Jupyter GUI (Box 3)</h3>"))
    display(controls)
    display(HTML("<h4>📝 Agent Output</h4>"))
    display(output_area)
    display(HTML("<h4>📘 Agent Memory</h4>"))
    display(widgets.HBox([mem_title, mem_refresh_button]))
    display(mem_output)

    print("✅ GUI Ready. Enter a task and click 'Run Agent 🧠'.")

# 🚀 Run GUI
launch_jupyter_gui()

🧠 Launching Jupyter GUI for ManusAgent...


Output(layout=Layout(border='1px solid gray', height='400px', overflow_y='scroll'))

Output(layout=Layout(border='1px solid #ccc', padding='5px'))

✅ GUI Ready. Enter a task and click 'Run Agent 🧠'.
[OLLAMA] time=2025-07-26T22:19:05.273Z level=INFO source=sched.go:788 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa gpu=GPU-41b26855-b961-3b12-ff40-38a659beb61b parallel=2 available=15720382464 required="6.2 GiB"
[OLLAMA] time=2025-07-26T22:19:05.359Z level=INFO source=server.go:135 msg="system memory" total="51.0 GiB" free="48.8 GiB" free_swap="0 B"
[OLLAMA] time=2025-07-26T22:19:05.360Z level=INFO source=server.go:175 msg=offload library=cuda layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[14.6 GiB]" memory.gpu_overhead="0 B" memory.required.full="6.2 GiB" memory.required.partial="6.2 GiB" memory.required.kv="1.0 GiB" memory.required.allocations="[6.2 GiB]" memory.weights.total="4.1 GiB" memory.weights.repeating="3.7 GiB" memory.weights.nonrepeating="411.0 MiB" memory.gr

In [ ]:
!cat /content/drive/MyDrive/UnifiedManusSystem/workspace/output.txt

cat: /content/drive/MyDrive/UnifiedManusSystem/workspace/output.txt: No such file or directory


In [ ]:
!ls -lh /content/drive/MyDrive/UnifiedManusSystem/workspace/

total 0


In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import subprocess
import os
import traceback

# This updated GUI orchestrates a team of agents to handle coding tasks.
# 1. Supervisor Agent: Manages the overall workflow.
# 2. Coder Agent: Writes the initial Python script.
# 3. Executor Agent: MUST run the code to complete its task.
# 4. Debugger Agent: Helps fix errors if the code execution fails.

def launch_jupyter_gui():
    """Launches the Jupyter GUI for multi-agent task processing."""
    clear_output(wait=True)
    print("Launching Multi-Agent Jupyter GUI...")

    task_input = widgets.Textarea(
        value='',
        placeholder='Describe your high-level coding or automation goal here...',
        description='Task:',
        layout=widgets.Layout(width='100%', height='80px')
    )

    context_input = widgets.Textarea(
        value='',
        placeholder='Optional: Provide any context, constraints, or data...',
        description='Context:',
        layout=widgets.Layout(width='100%', height='60px')
    )

    run_button = widgets.Button(
        description='Run Agents',
        button_style='success',
        layout=widgets.Layout(width='150px')
    )

    clear_mem_button = widgets.Button(
        description='Clear Memory',
        button_style='warning',
        layout=widgets.Layout(width='150px')
    )

    output_area = widgets.Output(layout={'border': '1px solid gray', 'height': '500px', 'overflow_y': 'scroll'})
    mem_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '5px'})

    def on_run_clicked(_):
        """
        This function implements the multi-agent workflow.
        The Supervisor delegates tasks to the Coder, Executor, and Debugger agents.
        """
        output_area.clear_output()
        goal = task_input.value.strip()
        context = context_input.value.strip()
        if not goal:
            with output_area:
                print("Please enter a task goal.")
                return

        executor_has_run_code = False

        try:
            # --- SUPERVISOR AGENT ---
            with output_area:
                print(f"--- [Supervisor Agent at {datetime.now().strftime('%H:%M:%S')}] ---")
                print(f"Received Goal: {goal}")
                if context: print(f"Context: {context}")
                print("Delegating task to Coder Agent...")

            # --- DELEGATION TO CODER AGENT ---
            coder_result = agent.run_task(
                goal=f"You are a Coder Agent. Write a complete Python script to achieve the following goal: {goal}",
                context=context if context else None
            )
            generated_code = coder_result.get("final_output", "")

            with output_area:
                print("\n--- [Coder Agent] ---")
                print("Generated Code:\n")
                print(generated_code)
                print("\n--- [Supervisor Agent] ---")
                print("Code received. Assigning to Executor Agent for mandatory execution.")

            # --- EXECUTOR AGENT'S MANDATORY TASK ---
            # This agent MUST run the code before it can exit its task.
            with output_area:
                print("\n--- [Executor Agent] ---")
                print("My task is to run the provided code. Executing now...")

            file_path = "agent_task.py"
            with open(file_path, "w") as f:
                f.write(generated_code)

            # Execute the script
            result = subprocess.run(["python3", file_path], capture_output=True, text=True, timeout=20)
            executor_has_run_code = True  # Mark that the mandatory action was completed.
            stdout, stderr, returncode = result.stdout, result.stderr, result.returncode

            with output_area:
                print("Execution Finished.")
                print("\n--- Execution STDOUT ---")
                print(stdout.strip())
                if stderr.strip():
                    print("\n--- Execution STDERR ---")
                    print(stderr.strip())
                print("\nReturn Code:", returncode)

            # --- EXECUTOR AGENT: ANALYSIS & HELP REQUEST ---
            if returncode != 0 or stderr.strip():
                with output_area:
                    print("\n--- [Executor Agent] ---")
                    print("Execution failed or produced an error. I must request help from the Debugger Agent.")

                # --- DELEGATION TO DEBUGGER AGENT ---
                debugger_context = f"The following script failed.\n\n--- CODE ---\n{generated_code}\n\n--- STDOUT ---\n{stdout}\n\n--- STDERR ---\n{stderr}"
                debug_output = agent.run_task(
                    goal="You are a Debugger Agent. Analyze the error and provide a corrected version of the code.",
                    context=debugger_context
                )

                with output_area:
                    print("\n--- [Debugger Agent] ---")
                    print("Analysis and suggested fix:\n")
                    print(debug_output.get("final_output", "[No output from Debugger]"))
            else:
                with output_area:
                    print("\n--- [Executor Agent] ---")
                    print("Script executed successfully.")

        except Exception as e:
            tb = traceback.format_exc()
            with output_area:
                print("\n--- [SYSTEM ERROR] ---")
                print("A critical error occurred during the agent workflow:\n", tb)
        finally:
            # The Executor Agent confirms if it fulfilled its primary directive.
            with output_area:
                print("\n--- [Executor Agent's Final Status] ---")
                if executor_has_run_code:
                    print("I have successfully completed my directive to run the code.")
                else:
                    print("I failed to complete my directive to run the code due to a system error.")


    def on_clear_memory(_):
        agent.memory.clear()
        with mem_output:
            mem_output.clear_output()
            print("Agent memory cleared.")

    run_button.on_click(on_run_clicked)
    clear_mem_button.on_click(on_clear_memory)

    controls = widgets.VBox([
        widgets.HBox([run_button, clear_mem_button]),
        task_input,
        context_input
    ])

    mem_title = widgets.HTML("<b>Agent Memory Log</b>")
    mem_refresh_button = widgets.Button(description="Refresh Memory", layout=widgets.Layout(width='150px'))

    def refresh_memory(_):
        mem_output.clear_output()
        with mem_output:
            # Display the last 10 memory entries
            for entry in agent.memory[-10:]:
                ts = entry.get("timestamp", "")
                role = entry.get("role", entry.get("type", ""))
                content = entry.get("content", "")
                print(f"[{ts}] ({role})\n{content}\n")

    mem_refresh_button.on_click(refresh_memory)

    # --- Display GUI ---
    display(HTML("<h3 style='color:darkblue'>Multi-Agent Jupyter GUI (Box 3)</h3>"))
    display(controls)
    display(HTML("<h4>Agent Workflow Output</h4>"))
    display(output_area)
    display(HTML("<h4>Agent Memory</h4>"))
    display(widgets.HBox([mem_title, mem_refresh_button]))
    display(mem_output)

    print("GUI Ready. Enter a task and click 'Run Agents'.")

# To run this code, you need to have an 'agent' object available in the scope
# that has a 'run_task' method and a 'memory' attribute.
#
# Example placeholder for the 'agent' object:
#
# class PlaceholderAgent:
#     def __init__(self):
#         self.memory = []
#     def run_task(self, goal, context=None):
#         # In a real scenario, this would call an LLM
#         self.memory.append({"timestamp": datetime.now().strftime('%H:%M:%S'), "role": "system", "content": f"Goal: {goal}"})
#         if "Coder Agent" in goal:
#             return {"final_output": "print('Hello from the agent-generated script!')"}
#         if "Debugger Agent" in goal:
#             return {"final_output": "# Corrected Code:\nprint('Hello from the corrected script!')"}
#         return {"final_output": "Task analysis complete."}
#
# agent = PlaceholderAgent()
#
# launch_jupyter_gui()

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import subprocess
import os
import traceback

# This script is specifically designed to be the frontend for the ManusAgent defined in Box 2.
# It respects the agent's internal workflow (Plan -> Code -> Review) and adds an execution layer.

def launch_jupyter_gui():
    """
    Launches the Jupyter GUI, now correctly aligned with the Box 2 agent's capabilities.
    """
    clear_output(wait=True)
    print("🧠 Launching GUI Frontend for Box 2 Agent...")

    # --- GUI Component Definitions ---
    task_input = widgets.Textarea(
        value='',
        placeholder='Describe your high-level goal here...',
        description='Task:',
        layout=widgets.Layout(width='100%', height='80px')
    )

    context_input = widgets.Textarea(
        value='',
        placeholder='Optional: Provide any context, constraints, or starting data...',
        description='Context:',
        layout=widgets.Layout(width='100%', height='60px')
    )

    run_button = widgets.Button(
        description='🚀 Run Task',
        button_style='success',
        tooltip='Send the task to the Box 2 Agent',
        icon='cogs',
        layout=widgets.Layout(width='180px')
    )

    clear_mem_button = widgets.Button(
        description='🧹 Clear Memory',
        button_style='warning',
        tooltip='Clear the memory of the agent',
        icon='trash',
        layout=widgets.Layout(width='180px')
    )

    output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '10px', 'height': '500px', 'overflow_y': 'scroll'})
    mem_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px', 'height': '200px', 'overflow_y': 'scroll'})
    # The "Chat Room" is now a "Workflow Log" to accurately reflect its new purpose.
    workflow_log_output = widgets.Output(layout={'border': '1px solid darkblue', 'padding': '10px', 'height': '500px', 'overflow_y': 'scroll'})

    progress_bar = widgets.IntProgress(
        value=0, min=0, max=4, # Reduced steps: Generate, Execute, Analyze, Complete
        description='Idle',
        bar_style='info',
        orientation='horizontal',
        layout=widgets.Layout(width='99%')
    )

    # --- Core Functions ---
    def log_workflow(step_name, message, icon="➡️"):
        """Logs a message to the workflow log panel."""
        with workflow_log_output:
            print(f"[{datetime.now().strftime('%H:%M:%S')}] {icon} {step_name}: {message}")

    def on_run_clicked(_):
        """
        Orchestrates the new, simplified workflow that is compatible with Box 2.
        """
        # 1. Initialization
        output_area.clear_output()
        workflow_log_output.clear_output()
        goal = task_input.value.strip()
        context = context_input.value.strip()

        if not goal:
            with output_area:
                print("❌ Please enter a task.")
            return

        progress_bar.value = 0
        progress_bar.description = 'Starting...'
        progress_bar.bar_style = 'info'
        log_workflow("GUI", "New task received. Engaging Box 2 agent.")

        try:
            # === Step 1: Call Box 2 Agent to Generate Plan/Code/Review ===
            progress_bar.value = 1
            progress_bar.description = 'Agent Thinking...'
            log_workflow("GUI", f"Sending goal ('{goal}') to agent's Plan->Code->Review pipeline.")

            # This is the single, primary call to the Box 2 agent.
            result_dict = agent.run_task(
                goal=goal,
                context=context
            )

            # Extract the specific parts from the agent's response
            plan = result_dict.get("plan", "[No plan provided by agent]")
            code = result_dict.get("code", "# [No code provided by agent]")
            review = result_dict.get("review", "[No review provided by agent]")

            with output_area:
                print("--- 🤖 Agent Response (from Box 2) ---")
                print("\n📋 PLAN:\n", "="*20)
                print(plan)
                print("\n💻 CODE:\n", "="*20)
                print(code)
                print("\n🧐 REVIEW:\n", "="*20)
                print(review)

            log_workflow("Agent", "Plan, Code, and Review received successfully.")

            # === Step 2: GUI Executes the Received Code ===
            progress_bar.value = 2
            progress_bar.description = 'Executing Code...'
            log_workflow("Executor", "Saving and executing the code provided by the agent.")

            file_path = os.path.join(WORKSPACE_DIR, "agent_task.py")
            with open(file_path, "w") as f:
                f.write(code)

            proc = subprocess.run(["python3", file_path], capture_output=True, text=True, timeout=30)
            stdout, stderr, returncode = proc.stdout, proc.stderr, proc.returncode

            with output_area:
                print("\n--- ▶️ GUI Execution Results ---\n")
                print("STDOUT:\n", stdout.strip())
                if stderr.strip():
                    print("\n❌ STDERR:\n", stderr.strip())
                print("\n📟 Return Code:", returncode)

            log_workflow("Executor", f"Execution finished with return code {returncode}.")

            # === Step 3: Analyze Results & Optional Debugging ===
            progress_bar.value = 3
            progress_bar.description = 'Analyzing...'

            if returncode != 0 or stderr.strip():
                # FAILURE: Send failure context back to the agent for another analysis round
                log_workflow("GUI", "Execution failed. Sending error details back to agent for analysis.", icon="⚠️")

                debug_goal = "The previously generated code failed to execute. Analyze the code and the error to identify the problem and suggest a fix."
                debug_context = f"--- Original Goal ---\n{goal}\n\n--- Failed Code ---\n{code}\n\n--- Execution STDOUT ---\n{stdout}\n\n--- Execution STDERR ---\n{stderr}"

                # Make a second call for debugging analysis
                debug_result_dict = agent.run_task(goal=debug_goal, context=debug_context)

                with output_area:
                    print("\n--- 🐞 Agent Debugging Analysis ---\n")
                    print(debug_result_dict.get("final_output", "[No debug analysis provided by agent]"))

                log_workflow("Agent", "Debugging analysis received.")
                progress_bar.bar_style = 'danger'
                progress_bar.description = 'Finished with Errors'
            else:
                # SUCCESS
                log_workflow("GUI", "Script executed successfully. Task complete.", icon="✅")
                progress_bar.bar_style = 'success'
                progress_bar.description = '✅ Success!'

            progress_bar.value = 4

        except Exception as e:
            tb = traceback.format_exc()
            progress_bar.bar_style = 'danger'
            progress_bar.description = '💥 CRITICAL ERROR'
            with output_area:
                print(f"\n💥💥💥 A CRITICAL EXCEPTION occurred in the GUI workflow 💥💥💥\n{tb}")
            log_workflow("GUI", f"A critical error disrupted the workflow: {e}", icon="💥")

    def on_clear_memory(_):
        """Clears the agent's memory via its method."""
        agent.memory.clear()
        mem_output.clear_output()
        workflow_log_output.clear_output()
        with mem_output:
            print("🧹 Agent memory cleared.")
        log_workflow("GUI", "Agent memory has been cleared.", icon="🧹")

    def refresh_memory(_):
        """Refreshes the memory log from the agent.memory attribute."""
        mem_output.clear_output()
        with mem_output:
            if not agent.memory:
                print("Agent memory is currently empty.")
                return

            print(f"Displaying last 20 of {len(agent.memory)} memory entries...")
            for entry in agent.memory[-20:]:
                ts = entry.get("timestamp", "N/A")
                role = entry.get("role", entry.get("type", "N/A"))
                content = entry.get("content", "")
                print("-" * 30)
                print(f"[{ts}] ({role})\n{content}\n")

    # --- Event Handler Wiring ---
    run_button.on_click(on_run_clicked)
    clear_mem_button.on_click(on_clear_memory)

    # --- Final GUI Assembly and Display ---
    controls = widgets.VBox([
        widgets.HBox([run_button, clear_mem_button]),
        task_input,
        context_input,
        progress_bar
    ])

    mem_controls = widgets.HBox([widgets.HTML("<b>🧠 Agent Memory Log</b>"), widgets.Button(description="🔄 Refresh Memory", on_click=refresh_memory, layout=widgets.Layout(width='180px'))])

    left_panel = widgets.VBox([
        widgets.HTML("<h3 style='color:darkblue'>📋 Workflow Log</h3>"),
        workflow_log_output,
        widgets.HTML("<h4 style='margin-top: 20px;'>📘 Agent Memory</h4>"),
        mem_controls,
        mem_output
    ])

    right_panel = widgets.VBox([
        widgets.HTML("<h3 style='color:darkgreen'>🛠️ Agent Output & Execution</h3>"),
        output_area
    ])

    main_layout = widgets.HBox([left_panel, right_panel])

    display(HTML("<h2 style='color:black'>🤖 GUI for Box 2 ManusAgent 🤖</h2>"))
    display(controls)
    display(main_layout)

    print("✅ GUI Ready. It is now properly configured to work with your Box 2 agent.")


# To use this script:
# 1. Ensure your 'agent' object from Box 2 is loaded and available.
# 2. Run this cell to define the function.
# 3. Call launch_jupyter_gui() in a new cell to start the interface.
launch_jupyter_gui()

🧠 Launching GUI Frontend for Box 2 Agent...


✅ GUI Ready. It is now properly configured to work with your Box 2 agent.
[OLLAMA] [GIN] 2025/07/27 - 14:34:21 | 200 | 30.045878251s |       127.0.0.1 | POST     "/api/generate"
[OLLAMA] [GIN] 2025/07/27 - 14:34:34 | 200 | 13.134193349s |       127.0.0.1 | POST     "/api/generate"
[OLLAMA] [GIN] 2025/07/27 - 14:34:53 | 200 | 18.986044882s |       127.0.0.1 | POST     "/api/generate"[OLLAMA] [GIN] 2025/07/27 - 14:35:01 | 200 |  7.314328564s |       127.0.0.1 | POST     "/api/generate"
[OLLAMA] [GIN] 2025/07/27 - 14:35:05 | 200 |  4.482865276s |       127.0.0.1 | POST     "/api/generate"
[OLLAMA] [GIN] 2025/07/27 - 14:35:20 | 200 | 14.484292167s |       127.0.0.1 | POST     "/api/generate"


In [ ]:
pip install sse-starlette

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 2: Agent Core and Tools with Ollama Integration & Chat Endpoint - v7.0.x                        ║
# ║ Integrates Ollama-powered ManusAgent with FastAPI tool endpoints and Human-in-the-Loop Chat             ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration & Chat...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import psutil
import requests
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable

print("📥 Step 1: Loading Box 1 configuration (Updated Path)...")

# --- Load configuration from Box 1 ---
try:
    # Standardized path from updated Box 1
    config_file = Path("/content/drive/MyDrive/UnifiedManusSystem/config/box1_exports.json")
    # Fallback for local runs
    if not config_file.exists():
         config_file = Path("./UnifiedManusSystem/config/box1_exports.json")

    if config_file.exists():
        with open(config_file, "r") as f:
            box1_config = json.load(f)

        BASE_DIR = Path(box1_config["BASE_DIR"])
        WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
        LOG_FILE = Path(box1_config["LOG_FILE"])
        public_url = box1_config["public_url"]
        dashboard_url = box1_config["dashboard_url"]
        IS_COLAB = box1_config["IS_COLAB"]
        print("✅ Box 1 configuration loaded successfully")
        print(f"📁 Base Directory: {BASE_DIR}")
        print(f"🌍 Public URL: {public_url}")
    else:
        raise FileNotFoundError("Box 1 config not found at expected location")

except Exception as e:
    print(f"❌ Failed to load Box 1 config: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True # Adjust based on environment

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core FastAPI imports
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from pydantic import BaseModel
# Import for SSE streaming
from sse_starlette.sse import EventSourceResponse
import asyncio

print("✅ All Box 2 modules imported successfully")

print("📝 Step 3: Setting up logging...")

def log_activity(category: str, message: str, data: Optional[Dict[str, Any]] = None):
    """Log activity to the JSON file."""
    try:
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "category": category,
            "message": message,
            "data": data or {},
            "box": "2"
        }

        # Read existing logs
        logs = []
        if LOG_FILE.exists():
            try:
                with open(LOG_FILE, "r") as f:
                    content = f.read()
                    if content.strip():
                         logs = json.loads(content)
            except json.JSONDecodeError:
                print("⚠️ Log file corrupted, starting fresh.")
                logs = []
        else:
            # Create parent directories if needed
            LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
            # Create an empty log file
            LOG_FILE.write_text("[]")

        logs.append(log_entry)

        # Keep last 1000 entries
        if len(logs) > 1000:
            logs = logs[-1000:]

        with open(LOG_FILE, "w") as f:
            json.dump(logs, f, indent=2)

        print(f"[LOG] {timestamp} [{category}] {message}")
    except Exception as e:
        print(f"❌ Logging failed: {e}")

log_activity("system", "Box 2 initialization started")

print("🛡️ Step 4: Setting up safety and path validation...")

def safe_path(file_path: str) -> Path:
    """Ensure file operations stay within safe directory"""
    if not file_path:
        raise ValueError("File path cannot be empty")

    base_path = WORKSPACE_DIR.resolve()
    full_path = (base_path / file_path).resolve()

    try:
        full_path.relative_to(base_path) # Raises ValueError if not relative
        return full_path
    except ValueError:
        raise PermissionError(f"Access denied: {file_path} is outside the workspace")

print("🦙 Step 5: Setting up Ollama Integration...")

# --- Ollama Configuration and Setup ---
OLLAMA_PORT = 11434
OLLAMA_PID_FILE = BASE_DIR / ".ollama_pid"
DEFAULT_MODEL = "llama3:8b"
MODEL_NAME = DEFAULT_MODEL # Can be overridden

def is_ollama_installed() -> bool:
    """Check if Ollama binary is installed"""
    return os.path.exists("/usr/local/bin/ollama") or os.path.exists("/usr/bin/ollama")

def is_ollama_running() -> bool:
    """Check if Ollama service is already running"""
    # Method 1: Check via API
    try:
        response = requests.get(f"http://localhost:{OLLAMA_PORT}", timeout=5)
        if response.status_code == 200:
            print("✅ Ollama API is accessible.")
            return True
    except requests.exceptions.RequestException:
        pass

    # Method 2: Check via PID file
    if OLLAMA_PID_FILE.exists():
        try:
            with open(OLLAMA_PID_FILE, 'r') as f:
                pid = int(f.read().strip())
            proc = psutil.Process(pid)
            if 'ollama' in proc.name().lower():
                print(f"✅ Ollama running from PID file (PID: {pid}).")
                return True
        except (ValueError, psutil.NoSuchProcess, psutil.AccessDenied, FileNotFoundError):
            pass
        # Clean up stale PID file
        OLLAMA_PID_FILE.unlink(missing_ok=True)

    # Method 3: Check via psutil for any ollama process
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            # Check if the process name contains 'ollama'
            if 'ollama' in proc.info['name'].lower():
                print(f"✅ Found Ollama process (PID: {proc.info['pid']}).")
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

    return False

def setup_ollama(model_name: str = DEFAULT_MODEL):
    """Setup Ollama: install, start server, and pull model."""
    global MODEL_NAME
    MODEL_NAME = model_name
    print(f"🔧 Setting up Ollama for model: {MODEL_NAME}")

    # 1. Install Ollama (if not present)
    if not is_ollama_installed():
        print("🔽 Installing Ollama...")
        try:
            install_script_url = "https://ollama.com/install.sh"
            result = subprocess.run(f"curl -fsSL {install_script_url} | sh", shell=True, capture_output=True, text=True, check=True)
            log_activity("setup", "Ollama installation", {"stdout": result.stdout, "stderr": result.stderr})
            print("✅ Ollama installation complete.")
        except subprocess.CalledProcessError as e:
            print(f"❌ Ollama installation failed: {e.stderr}")
            log_activity("setup", "Ollama installation failed", {"error": e.stderr})
            return
    else:
        print("✅ Ollama is already installed.")

    # 2. Start Ollama Serve
    if not is_ollama_running():
        print("🚀 Starting Ollama serve process...")
        try:
            process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # Save PID for future reference
            with open(OLLAMA_PID_FILE, 'w') as f:
                f.write(str(process.pid))
            time.sleep(5) # Give Ollama time to start
            if is_ollama_running():
                print("✅ Ollama serve started.")
            else:
                print("⚠️ Ollama serve process started, but API not immediately responsive.")
        except Exception as e:
            print(f"❌ Failed to start Ollama serve: {e}")
            log_activity("setup", "Ollama start failed", {"error": str(e)})
            return
    else:
        print("✅ Ollama is already running.")

    # 3. Pull Model
    print(f"🔽 Checking if model '{MODEL_NAME}' is available...")
    try:
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True, check=True)
        if MODEL_NAME in result.stdout:
            print(f"✅ Model '{MODEL_NAME}' is already available.")
        else:
            print(f"🔽 Model '{MODEL_NAME}' not found, pulling...")
            print("   ⚠️ This may take several minutes for large models. Please wait...")
            pull_result = subprocess.run(["ollama", "pull", MODEL_NAME], capture_output=True, text=True)
            log_activity("setup", f"Model {MODEL_NAME} pulled", {"stdout": pull_result.stdout, "stderr": pull_result.stderr})
            if pull_result.returncode == 0:
                print(f"✅ Model {MODEL_NAME} pulled successfully.")
            else:
                print(f"❌ Failed to pull model {MODEL_NAME}: {pull_result.stderr}")
                return # Stop if model pull fails
    except subprocess.CalledProcessError as e:
        print(f"❌ Error checking model list: {e}")
        # Try pulling anyway
        print(f"🔽 Attempting to pull model '{MODEL_NAME}'...")
        pull_result = subprocess.run(["ollama", "pull", MODEL_NAME], capture_output=True, text=True)
        log_activity("setup", f"Model {MODEL_NAME} pulled (fallback)", {"stdout": pull_result.stdout, "stderr": pull_result.stderr})


# --- Ollama Query Function (for Agent Roles) ---
def query_ollama_stream(prompt: str, system_prompt: str = "", role: str = "assistant", stream_callback=None) -> str:
    """
    Query Ollama model with streaming response (used by agent roles).
    """
    full_prompt = f"{system_prompt}\nUser: {prompt}\nAssistant:" if system_prompt else prompt
    full_response = ""
    try:
        response = requests.post(
            f"http://localhost:{OLLAMA_PORT}/api/generate",
            json={"model": MODEL_NAME, "prompt": full_prompt, "stream": True},
            stream=True,
            timeout=300 # 5 minute timeout
        )
        response.raise_for_status()
        for line in response.iter_lines():
            if line:
                chunk = json.loads(line)
                content = chunk.get("response", "")
                full_response += content
                if stream_callback:
                    # Pass content and role to the callback
                    stream_callback(content, role)
                if chunk.get("done"):
                    break
    except Exception as e:
        error_msg = f"Ollama query failed: {str(e)}"
        print(f"❌ {error_msg}")
        log_activity("ollama", "Query failed", {"error": str(e), "prompt": prompt[:100]})
        if stream_callback:
            stream_callback(f"\n❌ {error_msg}\n", "system")
        full_response = error_msg
    return full_response

# --- Ollama Chat Function (for Human-in-the-Loop) ---
def query_ollama_chat_stream(messages: List[Dict[str, str]], stream_callback=None) -> str:
    """
    Query Ollama model via the /api/chat endpoint for conversational interactions.
    Messages format: [{"role": "user/system/assistant", "content": "..."}, ...]
    """
    full_response = ""
    try:
        response = requests.post(
            f"http://localhost:{OLLAMA_PORT}/api/chat",
            json={"model": MODEL_NAME, "messages": messages, "stream": True},
            stream=True,
            timeout=300 # 5 minute timeout
        )
        response.raise_for_status()
        for line in response.iter_lines():
            if line:
                chunk = json.loads(line)
                # The /api/chat endpoint returns 'message' object with 'role' and 'content'
                message_content = chunk.get("message", {}).get("content", "")
                full_response += message_content
                if stream_callback:
                    # Pass content. Role is usually 'assistant' for chat.
                    stream_callback(message_content, "assistant")
                if chunk.get("done"):
                    break
    except Exception as e:
        error_msg = f"Ollama chat query failed: {str(e)}"
        print(f"❌ {error_msg}")
        log_activity("ollama", "Chat Query failed", {"error": str(e), "messages_preview": str(messages[:1])})
        if stream_callback:
            stream_callback(f"\n❌ {error_msg}\n", "system")
        full_response = error_msg
    return full_response



print("🎭 Step 6: Setting up Ollama-powered Agent Role system...")
# --- Agent Role System (Ollama-powered) ---
class ManusRole:
    def __init__(self, name: str, description: str, system_prompt: str):
        self.name = name
        self.description = description
        self.system_prompt = system_prompt

    def process(self, task_description: str, stream_callback=None):
        """Process task using Ollama LLM."""
        log_activity("agent", f"Role '{self.name}' processing task", {"task": task_description})
        full_response = query_ollama_stream(task_description, self.system_prompt, self.name, stream_callback)
        log_activity("agent", f"Role '{self.name}' finished processing")
        return full_response

# Define roles using Ollama
ROLES = {
    "planner": ManusRole(
        "Planner",
        "Creates plans",
        "You are an expert software architect. Break down user requests into clear, actionable implementation steps. Respond with a numbered list of steps."
    ),
    "researcher": ManusRole(
        "Researcher",
        "Gathers information",
        "You are a research assistant. Find relevant information and summarize it clearly. Only provide the summary, not the search process itself."
    ),
    "coder": ManusRole(
        "Coder",
        "Writes code",
        "You are a skilled software engineer. Write clean, efficient, well-documented code based on the plan provided. Only output the code without explanations."
    ),
    "reviewer": ManusRole(
        "Reviewer",
        "Reviews code",
        "You are a senior engineer reviewing code. Check for correctness, efficiency, security, and adherence to best practices. Provide specific suggestions for improvement."
    ),
    "debugger": ManusRole(
        "Debugger",
        "Fixes code",
        "You are an expert debugger. Identify the root cause of the error and provide fixed code. Explain your reasoning clearly."
    ),
}

print("📁 Step 7: Setting up File System Manager...")
# --- File System Manager (from previous snippets) ---
class FileSystemManager:
    def __init__(self, base_path: Path = WORKSPACE_DIR):
        self.base_path = base_path.resolve()
        self.base_path.mkdir(parents=True, exist_ok=True)

    def safe_join(self, *paths) -> Path:
        """Join paths and ensure the result is within the base path."""
        full_path = Path(self.base_path, *paths).resolve()
        try:
            full_path.relative_to(self.base_path) # Raises ValueError if not relative
            return full_path
        except ValueError:
            raise PermissionError(f"Path traversal attempt: {full_path}")

    def read_file(self, file_path: str) -> str:
        """Read a file safely."""
        full_path = self.safe_join(file_path)
        with open(full_path, 'r', encoding='utf-8') as f:
            return f.read()

    def write_file(self, file_path: str, content: str) -> str:
        """Write content to a file safely."""
        full_path = self.safe_join(file_path)
        full_path.parent.mkdir(parents=True, exist_ok=True) # Ensure parent dirs exist
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ File written to {full_path}"

    def list_files(self, dir_path: str = ".") -> List[str]:
        """List files in a directory safely."""
        full_path = self.safe_join(dir_path)
        if full_path.is_dir():
            return [str(p.relative_to(self.base_path)) for p in full_path.iterdir() if p.is_file()]
        else:
            return [str(full_path.relative_to(self.base_path))] if full_path.is_file() else []

print("🤖 Step 8: Setting up Ollama-powered Core Manus Agent...")
# --- Core Agent Class (Ollama-powered) ---
class ManusAgent:
    def __init__(self):
        self.roles = ROLES
        self.fs = FileSystemManager()
        self.memory: List[Dict[str, Any]] = []
        self.session_id = f"session_{int(time.time())}"
        log_activity("system", "Ollama-powered Manus Agent initialized")

    def solve_task(self, task_description: str, stream_callback=None):
        """Main task solving logic using Ollama-powered roles."""
        log_activity("agent", "Task started", {"task": task_description})
        self.memory.append({"type": "task_start", "content": task_description, "timestamp": datetime.now().isoformat()})
        if stream_callback:
            stream_callback(f"🧠 Starting task: {task_description}\n", "system")

        try:
            # Planner Role
            if stream_callback:
                stream_callback(f"\n📝 [Planner] Generating plan...\n", "planner")
            plan_output = self.roles["planner"].process(f"Create a plan for: {task_description}", stream_callback)
            self.memory.append({"type": "thought", "role": "planner", "content": plan_output, "timestamp": datetime.now().isoformat()})

            # Coder Role
            if stream_callback:
                 stream_callback(f"\n💻 [Coder] Writing code based on plan...\n", "coder")
            code_task = f"Plan:\n{plan_output}\n\nTask:\n{task_description}"
            code_output = self.roles["coder"].process(code_task, stream_callback)
            self.memory.append({"type": "action", "role": "coder", "content": code_output, "timestamp": datetime.now().isoformat()})

            # Reviewer Role
            if stream_callback:
                 stream_callback(f"\n🔍 [Reviewer] Reviewing code...\n", "reviewer")
            review_task = f"Code:\n{code_output}\n\nOriginal Plan:\n{plan_output}\n\nTask:\n{task_description}"
            review_output = self.roles["reviewer"].process(review_task, stream_callback)
            self.memory.append({"type": "thought", "role": "reviewer", "content": review_output, "timestamp": datetime.now().isoformat()})

            final_result = f"✅ Task '{task_description}' completed successfully!\n\n📝 Plan:\n{plan_output}\n\n💻 Generated Code:\n```python\n{code_output}\n```\n\n🔍 Review:\n{review_output}"
            self.memory.append({"type": "task_end", "content": final_result, "timestamp": datetime.now().isoformat()})
            if stream_callback:
                 stream_callback(f"\n✅ Task completed.\n", "system")
            log_activity("agent", "Task completed", {"task": task_description})
            return {"status": "success", "plan": plan_output, "code": code_output, "review": review_output, "final_output": final_result}

        except Exception as e:
            error_msg = f"❌ Agent task failed: {str(e)}\n{traceback.format_exc()}"
            self.memory.append({"type": "task_error", "content": error_msg, "timestamp": datetime.now().isoformat()})
            if stream_callback:
                 stream_callback(f"\n{error_msg}\n", "system")
            log_activity("agent", "Task failed", {"task": task_description, "error": str(e)})
            return {"status": "error", "message": error_msg}


    def run_task(self, goal: str, context: Optional[str] = None) -> Dict[str, Any]:
        """Wrapper for solve_task, potentially adding context."""
        task_with_context = f"{goal}\nContext: {context}" if context else goal
        # For direct tool call, we don't stream to a return value easily here.
        # The streaming is handled by the endpoint or GUI calling with a callback.
        result = self.solve_task(task_with_context) # Pass stream_callback if needed from caller
        return result

# Initialize the global agent instance
# Run Ollama setup first
setup_ollama(DEFAULT_MODEL)
# Now initialize the agent
agent = ManusAgent()
print("✅ Ollama-powered ManusAgent system ready")

print("🔧 Step 9: Registering all core tools...")

# --- Tool Registry ---
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str):
    """Decorator to register tools"""
    def decorator(func: Callable):
        TOOL_REGISTRY[name] = func
        print(f"🔧 Registered tool: {name}")
        log_activity("system", f"Tool registered: {name}")
        return func
    return decorator

# --- Registering Tools ---
@register_tool("write_file")
def write_file(file_path: str, content: str) -> str:
    """Write content to a file"""
    try:
        safe_file_path = safe_path(file_path)
        safe_file_path.parent.mkdir(parents=True, exist_ok=True)
        with open(safe_file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        log_activity("tool", "File written", {"file": file_path})
        return f"✅ Successfully wrote to {safe_file_path}"
    except Exception as e:
        error_msg = f"❌ Failed to write file {file_path}: {e}"
        log_activity("tool", "File write failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("read_file")
def read_file(file_path: str) -> str:
    """Read content from a file"""
    try:
        safe_file_path = safe_path(file_path)
        with open(safe_file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        log_activity("tool", "File read", {"file": file_path})
        return content
    except Exception as e:
        error_msg = f"❌ Failed to read file {file_path}: {e}"
        log_activity("tool", "File read failed", {"file": file_path, "error": str(e)})
        return error_msg

@register_tool("list_files")
def list_files(path: str = ".") -> List[str]:
    """List files in a directory"""
    try:
        safe_dir_path = safe_path(path)
        if safe_dir_path.is_dir():
            files = [str(p.relative_to(WORKSPACE_DIR)) for p in safe_dir_path.iterdir() if p.is_file()]
            log_activity("tool", "Directory listed", {"path": path})
            return files
        elif safe_dir_path.is_file():
            log_activity("tool", "File listed", {"path": path})
            return [str(safe_dir_path.relative_to(WORKSPACE_DIR))]
        else:
            return []
    except Exception as e:
        log_activity("tool", "Directory listing failed", {"path": path, "error": str(e)})
        return [f"❌ Error listing {path}: {e}"]

@register_tool("install_package")
def install_package(package_name: str) -> str:
    """Install a Python package using pip"""
    try:
        result = subprocess.run([sys.executable, "-m", "pip", "install", package_name],
                                capture_output=True, text=True, timeout=300)
        if result.returncode == 0:
            log_activity("tool", "Package installed", {"package": package_name})
            return f"✅ Successfully installed {package_name}\n{result.stdout}"
        else:
            log_activity("tool", "Package installation failed", {"package": package_name, "error": result.stderr})
            return f"❌ Failed to install {package_name}\n{result.stderr}"
    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Installation of {package_name} timed out"
        log_activity("tool", "Package installation timed out", {"package": package_name})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error installing {package_name}: {e}"
        log_activity("tool", "Package installation error", {"package": package_name, "error": str(e)})
        return error_msg

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30) -> str:
    """Execute Python code in a subprocess"""
    try:
        start_time = time.time()
        # Write code to a temporary file
        temp_file = WORKSPACE_DIR / f"temp_exec_{int(time.time())}.py"
        with open(temp_file, 'w') as f:
            f.write(code)

        # Run the code
        result = subprocess.run([sys.executable, str(temp_file)],
                                capture_output=True, text=True, timeout=timeout,
                                cwd=str(WORKSPACE_DIR))

        # Clean up
        temp_file.unlink(missing_ok=True)

        execution_time = time.time() - start_time
        if result.returncode == 0:
            output = f"✅ Code executed successfully (in {execution_time:.2f}s):\n{result.stdout}"
            if result.stderr:
                output += f"\n⚠️ Stderr:\n{result.stderr}"
            log_activity("tool", "Python code executed", {"execution_time": execution_time})
        else:
            output = f"❌ Code execution failed (in {execution_time:.2f}s):\n{result.stderr}"
            if result.stdout:
                output += f"\n_stdout:\n{result.stdout}"
            log_activity("tool", "Python code execution failed", {"execution_time": execution_time, "error": result.stderr})

        return output

    except subprocess.TimeoutExpired:
        error_msg = f"⏰ Code execution timed out after {timeout} seconds"
        log_activity("tool", "Python code timeout", {"timeout": timeout})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error executing code: {e}\n{traceback.format_exc()}"
        log_activity("tool", "Python code execution error", {"error": str(e)})
        return error_msg

# --- Register the Action Agent as a Tool ---
@register_tool("action_agent")
def action_agent(goal: str, context: Optional[str] = None) -> Dict[str, Any]:
    """
    Execute a complex task using the internal Ollama-powered ManusAgent.
    This is the core reasoning and multi-step execution tool.
    Returns the full result dictionary.
    """
    try:
        log_activity("tool", "Action Agent invoked", {"goal": goal})
        result = agent.run_task(goal, context)
        log_activity("tool", "Action Agent task completed", {"goal": goal, "status": result.get('status')})
        return result # Return the full dict
    except Exception as e:
        error_result = {"status": "error", "message": f"❌ Action Agent failed: {e}\n{traceback.format_exc()}"}
        log_activity("tool", "Action Agent error", {"goal": goal, "error": str(e)})
        return error_result


# --- Other Agent-related Tools ---
@register_tool("get_agent_memory")
def get_agent_memory() -> Dict[str, Any]:
    """Get the agent's memory"""
    return {
        "memory": agent.memory,
        "session_id": agent.session_id,
        "memory_count": len(agent.memory)
    }

@register_tool("clear_agent_memory")
def clear_agent_memory() -> str:
    """Clear the agent's memory"""
    old_count = len(agent.memory)
    agent.memory.clear()
    log_activity("tool", "Agent memory cleared", {"old_count": old_count})
    return f"🧹 Cleared {old_count} memory entries"


print(f"✅ Registered {len(TOOL_REGISTRY)} tools, including Ollama-powered 'action_agent'")

print("🌐 Step 10: Setting up FastAPI application...")

# --- Pydantic models ---
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# --- New Pydantic model for Chat ---
class ChatRequest(BaseModel):
    messages: List[Dict[str, str]] # [{"role": "user", "content": "Hello"}, ...]

# --- Initialize FastAPI app ---
app = FastAPI(
    title="Unified Manus MCP Server with Ollama & Chat",
    description="Multi-agent coding assistant powered by Ollama LLM, tool API, and Human-in-the-Loop Chat",
    version="7.0.x"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Adjust for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- API Endpoints ---
@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "status": "Unified Manus MCP System v7.0.x online (Ollama-powered with Chat)",
        "box": "2 - Agent Core",
        "docs": "/docs",
        "tool_call": "/mcp/tools/call",
        "tool_list": "/mcp/tools/list",
        "chat_endpoint": "/mcp/chat", # New chat endpoint
        "agent_status": f"Active - Session {agent.session_id}",
        "tools_available": len(TOOL_REGISTRY),
        "ollama_model": MODEL_NAME,
        "public_url": public_url,
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    ollama_ok = is_ollama_running()
    return {
        "status": "healthy" if ollama_ok else "degraded",
        "box": 2,
        "agent_memory_size": len(agent.memory),
        "tools_registered": len(TOOL_REGISTRY),
        "ollama_status": "running" if ollama_ok else "not running",
        "ollama_model": MODEL_NAME,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/tools/call")
async def call_tool(tool_call: ToolCall):
    """Execute a registered tool"""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found"})

    try:
        # Execute the tool
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return {"result": result}
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("tool_error", f"Tool '{tool_name}' failed", {"error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"error": f"Tool execution failed: {str(e)}", "details": tb_str})

@app.get("/mcp/tools/list")
async def list_tools():
    """List all available tools"""
    tools_info = []
    for name, func in TOOL_REGISTRY.items():
        description = func.__doc__.strip() if func.__doc__ else "No description provided."
        # Get function signature
        try:
            import inspect
            sig = inspect.signature(func)
            parameters = {}
            for param_name, param in sig.parameters.items():
                param_info = {
                    "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any",
                    "required": param.default == inspect.Parameter.empty
                }
                if param.default != inspect.Parameter.empty:
                    param_info["default"] = param.default
                parameters[param_name] = param_info
        except Exception:
            parameters = {"error": "Could not parse parameters"}

        tools_info.append({
            "name": name,
            "description": description,
            "parameters": parameters
        })
    return {
        "tools": tools_info,
        "count": len(tools_info),
        "timestamp": datetime.now().isoformat()
    }

# --- Endpoint specifically for the Action Agent ---
@app.post("/mcp/agent/action")
async def run_action_agent_endpoint(request: TaskRequest):
    """Run a task through the internal Ollama-powered ManusAgent"""
    try:
        # The agent's solve_task doesn't natively stream to HTTP yet,
        # but the result dict contains the full output.
        result = agent.run_task(request.task, request.context)
        return result # This returns the dict with status, plan, code, review, final_output
    except Exception as e:
        tb_str = traceback.format_exc()
        log_activity("agent_error", "Action Agent task failed", {"task": request.task, "error": str(e), "traceback": tb_str})
        return JSONResponse(status_code=500, content={"status": "error", "message": f"Action Agent task failed: {str(e)}", "details": tb_str})


@app.get("/mcp/agent/memory")
async def get_agent_memory_endpoint():
    """Get the agent's memory"""
    return get_agent_memory()

@app.post("/mcp/agent/memory/clear")
async def clear_agent_memory_endpoint():
    """Clear the agent's memory"""
    return {"result": clear_agent_memory()}

# --- NEW: Human-in-the-Loop Chat Endpoint ---
@app.post("/mcp/chat")
async def chat_with_ollama(chat_request: ChatRequest):
    """
    Chat directly with the Ollama model using the /api/chat endpoint.
    Supports streaming responses via Server-Sent Events (SSE).
    """
    messages = chat_request.messages

    async def event_generator():
        """Generator function for streaming SSE events."""
        full_response = ""
        try:
            # Prepare the request to Ollama
            ollama_url = f"http://localhost:{OLLAMA_PORT}/api/chat"
            payload = {"model": MODEL_NAME, "messages": messages, "stream": True}

            # Use requests in a thread to avoid blocking the async event loop
            loop = asyncio.get_event_loop()
            response = await loop.run_in_executor(None, lambda: requests.post(ollama_url, json=payload, stream=True, timeout=300))
            response.raise_for_status()

            # Stream the response chunks
            for line in response.iter_lines():
                if line:
                    try:
                        chunk_data = json.loads(line)
                        message_content = chunk_data.get("message", {}).get("content", "")
                        full_response += message_content

                        # Send the content chunk as an SSE event
                        yield {"event": "message", "data": json.dumps({"content": message_content})}

                        # Check if the stream is done
                        if chunk_data.get("done", False):
                            # Log the full interaction
                            log_activity("chat", "Chat interaction completed", {"message_count": len(messages)})
                            yield {"event": "end", "data": json.dumps({"final_message": full_response})}
                            break
                    except json.JSONDecodeError:
                        # Handle potential malformed JSON lines
                        yield {"event": "error", "data": json.dumps({"error": "Malformed response chunk from Ollama"})}
                        break
                    except Exception as e:
                        yield {"event": "error", "data": json.dumps({"error": f"Error processing stream: {str(e)}"})}
                        break

        except requests.exceptions.RequestException as e:
            error_msg = f"Ollama chat request failed: {str(e)}"
            log_activity("chat_error", error_msg, {"messages_preview": str(messages[:1])})
            yield {"event": "error", "data": json.dumps({"error": error_msg})}
        except Exception as e:
            error_msg = f"Unexpected error in chat stream: {str(e)}"
            log_activity("chat_error", error_msg)
            yield {"event": "error", "data": json.dumps({"error": error_msg})}

    # Return the Server-Sent Events response
    return EventSourceResponse(event_generator())


@app.get("/mcp/system/info")
async def get_system_info():
    """Get system information"""
    return {
        "box": 2,
        "name": "Agent Core and Tools (Ollama)",
        "version": "7.0.x",
        "base_dir": str(BASE_DIR),
        "workspace_dir": str(WORKSPACE_DIR),
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "is_colab": IS_COLAB,
        "agent_session": agent.session_id,
        "tools_count": len(TOOL_REGISTRY),
        "memory_entries": len(agent.memory),
        "ollama_model": MODEL_NAME,
        "ollama_status": "running" if is_ollama_running() else "not running",
        "uptime": datetime.now().isoformat(),
        "python_version": sys.version,
        "working_directory": os.getcwd()
    }


print("💾 Step 11: Setting up data persistence...")

def save_box2_state():
    """Save Box 2 state for other boxes"""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "memory_count": len(agent.memory),
        "api_endpoints": [
            "/mcp/tools/call",
            "/mcp/tools/list",
            "/mcp/agent/action",
            "/mcp/agent/memory",
            "/mcp/chat" # Include new chat endpoint
        ],
        "ollama_model": MODEL_NAME,
        "timestamp": datetime.now().isoformat()
    }
    config_file = BASE_DIR / "config" / "box2_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    with open(config_file, "w") as f:
        json.dump(state, f, indent=2)
    print(f"✅ Box 2 state saved to {config_file}")
    log_activity("system", "Box 2 state saved", {"path": str(config_file)})

save_box2_state()

print("🔍 Step 12: Verification and testing...")

def verify_box2_setup():
    """Verify Box 2 setup"""
    checks = {
        "Agent Initialized": agent is not None,
        "Tools Registered": len(TOOL_REGISTRY) > 0,
        "Action Agent Tool Available": "action_agent" in TOOL_REGISTRY,
        "FastAPI App": app is not None,
        "Base Directory": BASE_DIR.exists(),
        "Workspace Directory": WORKSPACE_DIR.exists(),
        "Log File Parent": LOG_FILE.parent.exists(),
        "Ollama Running": is_ollama_running(),
        "Default Model Available": MODEL_NAME in (subprocess.run(["ollama", "list"], capture_output=True, text=True).stdout if is_ollama_running() else "")
    }
    print("🔍 Box 2 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box2_setup()

if verification_passed:
    print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"🤖 Agent Session: {agent.session_id}")
    print(f"🦙 Ollama Model: {MODEL_NAME}")
    print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)} (including 'action_agent')")
    print(f"🧠 Memory Entries: {len(agent.memory)}")
    print(f"🌐 API Ready at: {public_url}")
    print(f"🗨️  New Chat Endpoint: {public_url}/mcp/chat (Supports SSE streaming)")
    print("=" * 60)
    print("🔄 Ready for Box 3: Server Launch and GUI")
    log_activity("system", "Box 2 setup completed successfully", {"tools_count": len(TOOL_REGISTRY), "agent_session": agent.session_id, "ollama_model": MODEL_NAME})

    # Test the action_agent tool (non-streaming)
    try:
        test_result = TOOL_REGISTRY["action_agent"]("Say hello in 3 different languages.")
        print("✅ Action Agent test completed successfully")
        print(f"   Test Result Status: {test_result.get('status', 'N/A')}")
        # print(f"   Test Result Output: {test_result.get('final_output', 'N/A')[:200]}...") # Preview
        log_activity("system", "Action Agent test completed successfully")
    except Exception as e:
        print(f"⚠️ Action Agent test failed: {e}")
        log_activity("system", "Action Agent test failed", {"error": str(e)})

else:
    print("❌ BOX 2 SETUP HAD ISSUES!")
    print("Please check the errors above before proceeding to Box 3")
    log_activity("system", "Box 2 setup failed", {"errors": "See output above"})

print("📤 Box 2 ready for integration with Box 3!")
print("🚀 PROCEED TO BOX 3 to launch the complete system!")

# Make the app and agent available for Box 3 to use
__all__ = ['app', 'agent', 'TOOL_REGISTRY']


🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration & Chat...
📥 Step 1: Loading Box 1 configuration (Updated Path)...
✅ Box 1 configuration loaded successfully
📁 Base Directory: /content/drive/MyDrive/UnifiedManusSystem
🌍 Public URL: http://localhost:8000
📦 Step 2: Importing required modules...
🔄 nest_asyncio applied
✅ All Box 2 modules imported successfully
📝 Step 3: Setting up logging...
[LOG] 2025-07-26T22:46:11.087849 [system] Box 2 initialization started
🛡️ Step 4: Setting up safety and path validation...
🦙 Step 5: Setting up Ollama Integration...
🎭 Step 6: Setting up Ollama-powered Agent Role system...
📁 Step 7: Setting up File System Manager...
🤖 Step 8: Setting up Ollama-powered Core Manus Agent...
🔧 Setting up Ollama for model: llama3:8b
✅ Ollama is already installed.
[OLLAMA] [GIN] 2025/07/26 - 22:46:11 | 200 |      27.495µs |       127.0.0.1 | GET      "/"
✅ Ollama API is accessible.
✅ Ollama is already running.
🔽 Checking if model 'llama3:8b' is available...


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 3: Server Launch and GUI - v7.0.x (Updated for Ollama-powered Box 2 with Chat)                  ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - FastAPI server launch with Uvicorn (Integrates Box 2 or runs proxy)                                   ║
# ║ - Multi-interface support: Jupyter, Gradio                                                          ║
# ║ - Plugin manifests for AI integration (Claude, OpenAI)                                                  ║
# ║ - System integration and monitoring                                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 3: Initializing Server Launch and GUI Systems (Updated for Ollama-powered Box 2 with Chat)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📥 Step 1: Loading configurations from previous boxes (Updated Paths)...")

# --- Load Box 1 and Box 2 configurations ---
try:
    # Standardized config directory path from updated Box 1
    config_dir = Path("/content/drive/MyDrive/UnifiedManusSystem/config")
    # Fallback for local runs
    if not config_dir.exists():
        config_dir = Path("./UnifiedManusSystem/config")

    if not config_dir.exists():
        raise FileNotFoundError("Config directory not found")

    # Load Box 1 config
    box1_config_file = config_dir / "box1_exports.json"
    if box1_config_file.exists():
        with open(box1_config_file, "r") as f:
            box1_config = json.load(f)
        print("✅ Box 1 configuration loaded")
    else:
        raise FileNotFoundError("Box 1 config not found")

    # Load Box 2 config (now includes Ollama details)
    box2_config_file = config_dir / "box2_exports.json"
    if box2_config_file.exists():
        with open(box2_config_file, "r") as f:
            box2_config = json.load(f)
        print("✅ Box 2 configuration loaded")
    else:
        print("⚠️ Box 2 config not found, will use defaults/fallbacks")
        box2_config = {"tools_registered": [], "agent_session": "unknown", "api_endpoints": [], "ollama_model": "unknown"}

    # Extract configuration
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]

    tools_available = box2_config.get("tools_registered", [])
    agent_session = box2_config.get("agent_session", "unknown")
    box2_api_endpoints = box2_config.get("api_endpoints", [])
    ollama_model = box2_config.get("ollama_model", "unknown")

    print(f"📁 Base Directory: {BASE_DIR}")
    print(f"🌍 Public URL: {public_url}")
    print(f"🤖 Agent Session: {agent_session}")
    print(f"🦙 Ollama Model: {ollama_model}")
    print(f"🛠️ Tools Available: {len(tools_available)}")

except Exception as e:
    print(f"❌ Error loading configurations: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True
    tools_available = []
    agent_session = "unknown"
    box2_api_endpoints = []
    ollama_model = "unknown"

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules for Box 3...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core web framework
import uvicorn
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse
import requests # For proxying calls to Box 2 if needed

# GUI frameworks
GRADIO_AVAILABLE = False
JUPYTER_AVAILABLE = False
try:
    import gradio as gr
    GRADIO_AVAILABLE = True
    print("✅ Gradio available")
except ImportError:
    print("⚠️ Gradio not available")

try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️ Jupyter widgets not available")

print("✅ All Box 3 modules imported successfully")

print("🔄 Step 3: Re-establishing connection to Box 2 components...")

# --- Determine if Box 2 is running ---
# Check if Box 2's app and agent are available in the current session (e.g., if this is a unified script)
box2_running_in_session = 'app' in globals() and 'agent' in globals() and 'TOOL_REGISTRY' in globals()
box2_api_accessible = False
box2_api_url = "http://localhost:8000" # Default assumption for internal calls

if box2_running_in_session:
    print("✅ Box 2 components found in current session (unified script mode)")
    box2_running = True
    # Use the in-session components
    try:
        from __main__ import app as box2_app, agent as box2_agent, TOOL_REGISTRY as box2_tool_registry
        print("🔗 Linked to in-session Box 2 components")
    except ImportError:
        print("⚠️ Could not import Box 2 components directly, using global references if available")
        # They are already in globals if the check passed
        box2_app = globals().get('app')
        box2_agent = globals().get('agent')
        box2_tool_registry = globals().get('TOOL_REGISTRY')
else:
    # Check if Box 2 server is running externally
    try:
        response = requests.get(f"{box2_api_url}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Box 2 server is accessible externally")
            box2_running = True
            box2_api_accessible = True
        else:
            print(f"⚠️ Box 2 server health check failed (Status: {response.status_code})")
            box2_running = False
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not connect to Box 2 server at {box2_api_url}: {e}")
        box2_running = False

if not box2_running:
    print("⚠️ Box 2 components not found/runnable. GUI will use fallback methods or fail gracefully.")


print("📄 Step 4: Creating plugin manifest files...")

def create_plugin_manifests():
    """Create plugin manifest files for AI integration"""
    print("📄 Creating plugin manifest files...")
    site_dir = BASE_DIR / "site"
    site_dir.mkdir(exist_ok=True)
    static_dir = site_dir / "static"
    static_dir.mkdir(exist_ok=True)

    # AI Plugin manifest (OpenAI/Claude compatible structure)
    ai_plugin_manifest = {
        "schema_version": "v1",
        "name_for_human": "Unified Manus MCP",
        "name_for_model": "unified_manus",
        "description_for_human": "Multi-agent coding assistant with comprehensive tool support, powered by Ollama LLM.",
        "description_for_model": f"A unified agent system with file operations, Python execution, package management, and multi-role thinking capabilities via the 'action_agent' tool, using the {ollama_model} model. Also supports direct chat via /mcp/chat.",
        "auth": {"type": "none"},
        "api": {"type": "openapi", "url": f"{public_url}/openapi.json"}, # Points to Box 2's OpenAPI spec
        "logo_url": f"{public_url}/site/static/logo.png", # Placeholder
        "contact_email": "support@example.com",
        "legal_info_url": f"{public_url}/site/legal.html" # Placeholder
    }
    try:
        with open(site_dir / "ai-plugin.json", "w") as f:
            json.dump(ai_plugin_manifest, f, indent=2)
        print("✅ AI Plugin manifest created")
    except Exception as e:
        print(f"❌ Failed to create AI Plugin manifest: {e}")

    # Claude-compatible manifest (YAML)
    claude_manifest = {
        "name": "unified_manus",
        "description": f"Multi-agent coding assistant with comprehensive tool support, including an 'action_agent' for complex tasks and a '/mcp/chat' endpoint for direct conversation, powered by Ollama ({ollama_model}).",
        "version": "7.0.x",
        "endpoints": {
            "tool_call": f"{public_url}/mcp/tools/call",
            "tool_list": f"{public_url}/mcp/tools/list",
            "chat": f"{public_url}/mcp/chat" # Include the new chat endpoint
        },
        "capabilities": ["file_operations", "python_execution", "package_management", "agent_thinking", "memory_management", "chat"]
    }
    try:
        import yaml # Should be available as installed by updated Box 1
        with open(site_dir / "claude.yaml", "w") as f:
            yaml.dump(claude_manifest, f, default_flow_style=False)
        print("✅ Claude manifest (YAML) created")
    except ImportError:
        print("⚠️ PyYAML not found, skipping Claude manifest YAML creation.")
    except Exception as e:
        print(f"❌ Failed to create Claude manifest: {e}")

    # Simple index.html for /site/
    index_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Unified Manus System (Ollama)</title>
</head>
<body>
    <h1>🤖 Unified Manus MCP System v7.0.x (Ollama-powered)</h1>
    <p>Multi-Agent Coding Assistant</p>
    <ul>
        <li><a href="/docs">API Documentation (FastAPI)</a></li>
        <li><a href="/redoc">API Documentation (ReDoc)</a></li>
        <li>Agent Session: {agent_session}</li>
        <li>Ollama Model: {ollama_model}</li>
        <li>Tools Available: {len(tools_available)}</li>
        <li>Public URL: <a href="{public_url}">{public_url}</a></li>
    </ul>
</body>
</html>
"""
    try:
        with open(site_dir / "index.html", "w") as f:
            f.write(index_content)
        print("✅ Basic site index.html created")
    except Exception as e:
        print(f"❌ Failed to create site index.html: {e}")

create_plugin_manifests()


print("🎨 Step 5: Setting up Gradio and Jupyter interfaces...")

# --- Gradio Interface ---
def setup_gradio_interface():
    """Setup Gradio interface"""
    if not GRADIO_AVAILABLE:
        print("⚠️ Gradio not available, skipping Gradio interface")
        return None

    print("🎨 Setting up Gradio interface...")

    def run_agent_task(task, context):
        """Run a task through the agent"""
        try:
            if box2_running_in_session:
                # Direct call to in-session agent
                result_dict = box2_agent.run_task(task, context)
                if result_dict.get("status") == "success":
                     return result_dict.get("final_output", "No final output found.")
                else:
                     return f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}"
            elif box2_api_accessible:
                 payload = {"task": task, "context": context}
                 response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                 if response.status_code == 200:
                     result_dict = response.json()
                     if result_dict.get("status") == "success":
                          return result_dict.get("final_output", "No final output found in API response.")
                     else:
                          return f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}"
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Agent Core) is not available."
        except Exception as e:
            return f"❌ Failed to run agent task: {str(e)}"

    def list_tools():
        """List available tools"""
        try:
             if box2_running_in_session:
                 from __main__ import TOOL_REGISTRY
                 tools_text = f"🛠️ Available Tools ({len(TOOL_REGISTRY)}):\n"
                 for name, func in TOOL_REGISTRY.items():
                     desc = func.__doc__.split('\n')[0] if func.__doc__ else "No description"
                     tools_text += f"• {name}: {desc}\n"
                 return tools_text
             elif box2_api_accessible:
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", timeout=10)
                 if response.status_code == 200:
                     result = response.json()
                     tools_text = f"🛠️ Available Tools ({result['count']}):\n"
                     for tool in result['tools']:
                         tools_text += f"• {tool['name']}: {tool['description']}\n"
                     return tools_text
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
             else:
                  return "❌ Box 2 (Tool Registry) is not available."
        except Exception as e:
            return f"❌ Failed to fetch tools: {str(e)}"

    def execute_tool(tool_name, tool_input):
        """Execute a specific tool"""
        try:
            if isinstance(tool_input, str) and tool_input.strip():
                try:
                    input_data = json.loads(tool_input)
                except json.JSONDecodeError:
                    input_data = {"content": tool_input}
            else:
                input_data = {}

            if box2_running_in_session:
                from __main__ import TOOL_REGISTRY
                if tool_name in TOOL_REGISTRY:
                    result = TOOL_REGISTRY[tool_name](**input_data)
                    if isinstance(result, dict):
                        return json.dumps(result, indent=2)
                    return str(result)
                else:
                    return f"❌ Tool '{tool_name}' not found."
            elif box2_api_accessible:
                payload = {"tool_name": tool_name, "tool_input": input_data}
                response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                if response.status_code == 200:
                    res_data = response.json()
                    result_content = res_data.get("result", "No result field")
                    if isinstance(result_content, dict):
                        return json.dumps(result_content, indent=2)
                    return str(result_content)
                else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Tool Execution) is not available."
        except Exception as e:
            return f"❌ Failed to execute tool: {str(e)}"

    # --- New Chat Functionality for Gradio ---
    def chat_with_model(message_history):
        """
        Chat with the model via the new /mcp/chat endpoint.
        Gradio's ChatInterface passes message_history as a list of [user_msg, bot_msg, user_msg, ...]
        We need to convert it to the format expected by /mcp/chat: [{"role": "...", "content": "..."}]
        """
        # Convert Gradio history to Ollama format
        ollama_messages = []
        for i, msg in enumerate(message_history):
            if i % 2 == 0: # User message
                ollama_messages.append({"role": "user", "content": msg})
            else: # Bot message
                ollama_messages.append({"role": "assistant", "content": msg})

        if box2_running_in_session and box2_api_accessible:
            # Prefer direct API call for streaming
            try:
                payload = {"messages": ollama_messages}
                # Note: Streaming responses from external APIs to Gradio chatbot can be complex.
                # A simpler approach is to make a non-streaming call.
                response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120)
                if response.status_code == 200:
                    # The /mcp/chat endpoint returns a stream, but for simplicity in Gradio,
                    # we can aggregate the final response if it's not truly streaming here.
                    # Or, if it returns a final aggregated response, use that.
                    # This depends on the exact implementation of the endpoint.
                    # Let's assume it returns a final message in a standard format for now.
                    # A more robust solution would involve handling the SSE stream.
                    data = response.json()
                    # This part needs adjustment based on how /mcp/chat final response is structured.
                    # Placeholder logic:
                    if "final_message" in data:
                        return data["final_message"]
                    elif "content" in data:
                         return data["content"]
                    else:
                        return "Received response, but format unclear."
                else:
                    return f"❌ Chat API Error ({response.status_code}): {response.text}"
            except Exception as e:
                 return f"❌ Chat API Call Failed: {str(e)}"
        elif box2_running_in_session:
            # If only running in session, we'd need a way to call the Ollama chat function directly
            # and handle streaming. This is more complex in Gradio without async support in this context.
            # Fallback: Use a simple non-streaming direct call (if such a function exists or is adapted).
            # For now, indicate it's not fully supported via direct call in this GUI setup.
            return "⚠️ Direct chat not fully implemented in this GUI mode. Use API or Jupyter GUI for full chat."
        else:
             return "❌ Box 2 Chat API is not accessible."

    with gr.Blocks(title="Unified Manus MCP System (Ollama)", theme=gr.themes.Default()) as demo:
        gr.Markdown("# 🤖 Unified Manus MCP System v7.0.x (Ollama-powered)")
        gr.Markdown("Multi-Agent Coding Assistant with LLM Capabilities")

        with gr.Tab("Agent Tasks"):
            with gr.Row():
                with gr.Column():
                    task_input = gr.Textbox(label="Task", placeholder="Enter a task for the agent...")
                    context_input = gr.Textbox(label="Context (optional)", placeholder="Additional context...", lines=3)
                    run_btn = gr.Button("Run Task with Action Agent", variant="primary")
                with gr.Column():
                    task_output = gr.Textbox(label="Agent Output", lines=20, max_lines=30, show_copy_button=True)
            run_btn.click(fn=run_agent_task, inputs=[task_input, context_input], outputs=task_output)

        with gr.Tab("Tool Execution"):
            with gr.Row():
                with gr.Column():
                    tool_name_input = gr.Dropdown(label="Tool Name", choices=tools_available if tools_available else [], allow_custom_value=True)
                    tool_input_input = gr.Textbox(label="Tool Input (JSON)", placeholder='{"file_path": "test.txt", "content": "Hello"}', lines=5)
                    exec_tool_btn = gr.Button("Execute Tool", variant="secondary")
                with gr.Column():
                    tool_output = gr.Textbox(label="Tool Output", lines=15, max_lines=20, show_copy_button=True)
            exec_tool_btn.click(fn=execute_tool, inputs=[tool_name_input, tool_input_input], outputs=tool_output)

        with gr.Tab("Direct Chat (via /mcp/chat)"):
             # Gradio's ChatInterface is a convenient way to handle chat
             chatbot = gr.Chatbot(label="Conversation")
             msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
             clear_chat = gr.Button("Clear Chat")

             def respond(message, chat_history):
                 # Append user message to history
                 chat_history.append((message, None))
                 # Get response from chat function
                 bot_message = chat_with_model([item for sublist in chat_history for item in sublist if item is not None])
                 # Update the last entry in history with the bot's response
                 chat_history[-1] = (message, bot_message)
                 return "", chat_history

             msg.submit(respond, [msg, chatbot], [msg, chatbot])
             clear_chat.click(fn=lambda: ([], []), inputs=[], outputs=[chatbot, msg], queue=False)


        with gr.Tab("System Info"):
            with gr.Row():
                tools_btn = gr.Button("Refresh Tool List")
                tools_output = gr.Textbox(label="Available Tools", lines=15, max_lines=20)
                tools_btn.click(fn=list_tools, outputs=tools_output)

            info_text = (
                f"**System Information:**\n"
                f"- Public API URL: {public_url}\n"
                f"- Dashboard URL: {dashboard_url}\n"
                f"- Agent Session: {agent_session}\n"
                f"- Ollama Model: {ollama_model}\n"
                f"- Base Directory: {BASE_DIR}\n"
                f"- Workspace Directory: {WORKSPACE_DIR}\n"
                f"- Tools Available: {len(tools_available)}\n"
                f"- Box 2 Status: {'Integrated/Running' if box2_running else 'Not Accessible'}"
            )
            gr.Markdown(info_text)

    return demo

# --- Jupyter Interface ---
def setup_jupyter_interface():
    """Setup Jupyter widget interface"""
    if not JUPYTER_AVAILABLE:
        print("⚠️ Jupyter widgets not available, skipping Jupyter interface")
        return None

    print("📓 Setting up Jupyter interface...")

    # --- Jupyter UI Logic ---
    def create_jupyter_ui():
        clear_output(wait=True)

        # --- UI Elements ---
        task_input = widgets.Text(
            value='',
            placeholder='Enter a task for the agent (e.g., Write a Python script to calculate Fibonacci numbers)',
            description='Task:',
            disabled=False,
            layout=widgets.Layout(width='100%')
        )

        context_input = widgets.Textarea(
            value='',
            placeholder='Optional context for the task',
            description='Context:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        run_button = widgets.Button(
            description="Run Task with Action Agent",
            button_style='success',
            tooltip='Execute the task using the internal Ollama-powered ManusAgent',
            icon='play'
        )

        tool_name_dropdown = widgets.Dropdown(
            options=tools_available if tools_available else ['No tools loaded'],
            value=tools_available[0] if tools_available else 'No tools loaded',
            description='Tool:',
            disabled=not tools_available,
        )

        tool_input_textarea = widgets.Textarea(
            value='{}',
            placeholder='Enter tool input as JSON (e.g., {"file_path": "test.txt", "content": "Hello"})',
            description='Input (JSON):',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        tool_run_button = widgets.Button(
            description="Execute Tool",
            button_style='info',
            tooltip='Run the selected tool with the provided input',
            icon='cogs'
        )

        clear_button = widgets.Button(
            description="Clear Output",
            button_style='warning',
            tooltip='Clear the output areas below',
            icon='eraser'
        )

        output_area = widgets.Output(
            layout=widgets.Layout(height='400px', border='1px solid black', overflow='auto', padding='10px')
        )

        # --- Chat Elements ---
        chat_history_area = widgets.Output(
             layout=widgets.Layout(height='300px', border='1px solid blue', overflow='auto', padding='10px', background_color='#f0f8ff')
        )
        chat_input = widgets.Textarea(
            value='',
            placeholder='Type your message here for direct chat...',
            description='Chat:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='80px')
        )
        chat_send_button = widgets.Button(
            description="Send Message",
            button_style='primary',
            tooltip='Send message to the Ollama model directly',
            icon='paper-plane'
        )
        chat_clear_button = widgets.Button(
            description="Clear Chat",
            button_style='warning',
            tooltip='Clear the chat history',
            icon='trash'
        )

        # --- Event Handlers ---
        def on_run_task(b):
            task = task_input.value
            context = context_input.value
            if not task:
                with output_area:
                    print("⚠️ Please enter a task.")
                return

            with output_area:
                clear_output(wait=True)
                print(f"🚀 Running task: '{task}'")
                if context:
                    print(f"   Context: '{context}'")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        result_dict = box2_agent.run_task(task, context)
                        if result_dict.get("status") == "success":
                             print("✅ Task completed!")
                             print("\n📝 Plan:")
                             print(result_dict.get("plan", "N/A"))
                             print("\n💻 Generated Code:")
                             print(result_dict.get("code", "N/A"))
                             print("\n🔍 Review:")
                             print(result_dict.get("review", "N/A"))
                             print("\n--- Final Output ---")
                             print(result_dict.get("final_output", "N/A"))
                        else:
                             print(f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}")
                    elif box2_api_accessible:
                         payload = {"task": task, "context": context}
                         response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                         if response.status_code == 200:
                             result_dict = response.json()
                             if result_dict.get("status") == "success":
                                  print("✅ Task completed!")
                                  print("\n📝 Plan:")
                                  print(result_dict.get("plan", "N/A"))
                                  print("\n💻 Generated Code:")
                                  print(result_dict.get("code", "N/A"))
                                  print("\n🔍 Review:")
                                  print(result_dict.get("review", "N/A"))
                                  print("\n--- Final Output ---")
                                  print(result_dict.get("final_output", "N/A"))
                             else:
                                  print(f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}")
                         else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Agent Core) is not available.")
                except Exception as e:
                    print(f"💥 ERROR running task: {e}")
                    traceback.print_exc()

        def on_execute_tool(b):
            tool_name = tool_name_dropdown.value
            tool_input_str = tool_input_textarea.value

            if tool_name == 'No tools loaded':
                 with output_area:
                     print("⚠️ No tools are available to execute.")
                 return

            try:
                if tool_input_str.strip():
                    tool_input_data = json.loads(tool_input_str)
                else:
                    tool_input_data = {}
            except json.JSONDecodeError as e:
                 with output_area:
                     print(f"❌ Invalid JSON input for tool: {e}")
                 return

            with output_area:
                clear_output(wait=True)
                print(f"🔧 Executing tool: '{tool_name}'")
                print(f"   Input: {json.dumps(tool_input_data, indent=2)}")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        if tool_name in box2_tool_registry:
                            result = box2_tool_registry[tool_name](**tool_input_data)
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result, indent=2) if isinstance(result, dict) else str(result))
                        else:
                            print(f"❌ Tool '{tool_name}' not found in registry.")
                    elif box2_api_accessible:
                        payload = {"tool_name": tool_name, "tool_input": tool_input_data}
                        response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                        if response.status_code == 200:
                            res_data = response.json()
                            result_content = res_data.get("result", "No result field")
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result_content, indent=2) if isinstance(result_content, dict) else str(result_content))
                        else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Tool Execution) is not available.")
                except Exception as e:
                    print(f"💥 ERROR executing tool: {e}")
                    traceback.print_exc()

        def on_clear(b):
            output_area.clear_output()

        # --- Chat Event Handlers ---
        def on_chat_send(b):
            user_message = chat_input.value.strip()
            if not user_message:
                with chat_history_area:
                    print("⚠️ Please enter a message to send.")
                return

            with chat_history_area:
                print(f"[You]: {user_message}")
                print(f"[Assistant]: ", end="") # Prepare for streaming response

            # Prepare messages for API call (simple two-turn history for demo)
            # In a full implementation, you'd manage a longer history.
            messages = [
                {"role": "user", "content": user_message}
            ]

            try:
                if box2_running_in_session and box2_api_accessible:
                    # Call the new /mcp/chat endpoint
                    payload = {"messages": messages}
                    response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120, stream=True)
                    if response.status_code == 200:
                        full_response = ""
                        # Process streaming response (basic handling)
                        for line in response.iter_lines():
                            if line:
                                try:
                                    chunk_data = json.loads(line)
                                    if chunk_data.get("event") == "message":
                                        content = chunk_data.get("data", {}).get("content", "")
                                        full_response += content
                                        # Update the output area with the new chunk
                                        with chat_history_area:
                                            print(content, end="", flush=True) # Stream print
                                    elif chunk_data.get("event") == "end":
                                        # Final message received
                                        break
                                    elif chunk_data.get("event") == "error":
                                        error_msg = chunk_data.get("data", {}).get("error", "Unknown stream error")
                                        with chat_history_area:
                                            print(f"\n❌ Stream Error: {error_msg}")
                                        break
                                except json.JSONDecodeError:
                                    with chat_history_area:
                                        print(f"\n⚠️ Malformed stream data received.")
                        with chat_history_area:
                             print() # Newline after streaming
                        chat_input.value = '' # Clear input
                    else:
                         with chat_history_area:
                             print(f"\n❌ Chat API Error ({response.status_code}): {response.text}")
                elif box2_running_in_session:
                    # Direct call to Ollama chat function (requires more integration)
                    # This is complex in Jupyter widgets sync context without async handling.
                    with chat_history_area:
                         print("\n⚠️ Direct chat integration not fully implemented in this mode. Use the API endpoint.")
                    chat_input.value = ''
                else:
                     with chat_history_area:
                         print("\n❌ Box 2 Chat API is not accessible.")
            except Exception as e:
                 with chat_history_area:
                     print(f"\n💥 ERROR sending chat message: {e}")
                 # Clear input on error too
                 chat_input.value = ''


        def on_chat_clear(b):
            chat_history_area.clear_output()
            chat_input.value = '' # Also clear the input field

        # Assign event handlers
        run_button.on_click(on_run_task)
        tool_run_button.on_click(on_execute_tool)
        clear_button.on_click(on_clear)
        chat_send_button.on_click(on_chat_send)
        chat_clear_button.on_click(on_chat_clear)

        # --- Display Layout ---
        ui_layout = widgets.VBox([
            widgets.HTML("<h1>🤖 Unified Manus MCP System (Jupyter - Ollama)</h1>"),
            widgets.HTML("<h2>Agent Task Execution</h2>"),
            task_input,
            context_input,
            run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Tool Execution</h2>"),
            tool_name_dropdown,
            tool_input_textarea,
            tool_run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Direct Chat</h2>"),
            chat_history_area,
            chat_input,
            widgets.HBox([chat_send_button, chat_clear_button]),
            widgets.HTML("<h2 style='margin-top: 20px;'>Output</h2>"),
            clear_button,
            output_area
        ])

        display(ui_layout)

    return create_jupyter_ui


print("🌐 Step 6: Setting up FastAPI server (Integrated or Proxy)...")

# --- Integrated or Proxy FastAPI App for Box 3 ---
def create_integrated_or_proxy_server():
    """Create the FastAPI app for Box 3, either integrated or proxying to Box 2"""
    app = FastAPI(
        title="Unified Manus MCP Server - Box 3 (Ollama)",
        description="Integrated server with GUI interfaces and potential proxying to Ollama-powered Box 2",
        version="7.0.x"
    )

    # CORS middleware
    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"], # Adjust for production
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    # Static files
    site_dir = BASE_DIR / "site"
    if site_dir.exists():
        app.mount("/site", StaticFiles(directory=str(site_dir)), name="site")

    # --- Root endpoint ---
    @app.get("/")
    async def root():
        """Root endpoint"""
        index_file = site_dir / "index.html"
        if index_file.exists():
            return FileResponse(str(index_file))
        else:
            return {
                "message": "Unified Manus MCP System - Box 3 (Ollama-powered)",
                "version": "7.0.x",
                "box2_status": "Integrated" if box2_running_in_session else ("Proxying" if box2_api_accessible else "Unavailable"),
                "public_url": public_url,
                "agent_session": agent_session,
                "ollama_model": ollama_model
            }

    # --- Health check ---
    @app.get("/health")
    async def health():
        """Health check endpoint"""
        box2_status = "Unavailable"
        if box2_running_in_session:
            box2_status = "Integrated"
        elif box2_api_accessible:
            try:
                b2_response = requests.get(f"{box2_api_url}/health", timeout=2)
                if b2_response.status_code == 200:
                    b2_data = b2_response.json()
                    if b2_data.get("status") == "healthy":
                        box2_status = "Accessible (Healthy)"
                    else:
                        box2_status = f"Accessible (Degraded: {b2_data.get('status')})"
                else:
                    box2_status = f"Accessible (API Error: {b2_response.status_code})"
            except:
                box2_status = "Accessible (Health Check Failed)"

        return {
            "status": "healthy" if ("Healthy" in box2_status or box2_status == "Integrated") else "degraded",
            "box": 3,
            "version": "7.0.x",
            "timestamp": datetime.now().isoformat(),
            "box2_status": box2_status,
            "agent_session": agent_session,
            "ollama_model": ollama_model
        }

    # --- Proxy endpoints to Box 2 if it's running externally ---
    if not box2_running_in_session and box2_api_accessible:
        print("🔄 Setting up proxy endpoints to external Box 2...")

        @app.api_route("/mcp/tools/call", methods=["POST"])
        async def proxy_tool_call(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/tools/call", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/tools/list", methods=["GET"])
        async def proxy_tool_list(request: Request):
             try:
                 params = dict(request.query_params)
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", params=params)
                 return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
             except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/action", methods=["POST"])
        async def proxy_agent_action(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/agent/action", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/memory", methods=["GET", "POST"])
        async def proxy_agent_memory(request: Request):
            try:
                url = f"{box2_api_url}/mcp/agent/memory"
                if request.method == "POST":
                    url += "/clear"
                body = await request.body() if request.method in ["POST", "PUT"] else None
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                if body:
                    response = requests.request(request.method, url, data=body, headers=headers)
                else:
                     response = requests.request(request.method, url, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        # --- Proxy the new Chat endpoint ---
        @app.api_route("/mcp/chat", methods=["POST"])
        async def proxy_chat(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                # Use requests to forward the stream
                # Note: True proxying of SSE streams is complex with standard requests.
                # A more robust solution might use `httpx` or async libraries.
                # For now, we proxy the request and return the response.
                response = requests.post(f"{box2_api_url}/mcp/chat", data=body, headers=headers, stream=True)
                # Returning a streaming response correctly requires more setup.
                # This is a simplified proxy that might not handle streams perfectly.
                # Consider using `httpx` StreamResponse for better SSE proxying.
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Chat Proxy error: {str(e)}"})


    elif box2_running_in_session:
        print("🔗 Box 2 is integrated, no proxy needed for its endpoints.")
    else:
        print("⚠️ Box 2 is not accessible, proxy endpoints will not function.")

    return app

# Create the integrated/proxy app
integrated_app = create_integrated_or_proxy_server()
print("✅ FastAPI server (Box 3) configured")


print("💾 Step 7: Saving Box 3 configuration...")

def save_box3_state():
    """Save Box 3 state"""
    state = {
        "interfaces_available": ["jupyter", "gradio", "api"],
        "web_interface_ready": True,
        "plugin_manifests_created": True,
        "launch_modes": ["jupyter", "gradio", "api", "all"],
        "timestamp": datetime.now().isoformat(),
        "box2_connection": {
            "in_session": box2_running_in_session,
            "api_accessible": box2_api_accessible,
            "status": "Integrated" if box2_running_in_session else ("Accessible" if box2_api_accessible else "Unavailable")
        },
        "gradio_available": GRADIO_AVAILABLE,
        "jupyter_available": JUPYTER_AVAILABLE,
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "agent_session": agent_session,
        "ollama_model": ollama_model
    }
    config_file = BASE_DIR / "config" / "box3_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 3 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 3 state: {e}")

save_box3_state()

print("🔍 Step 8: Final verification...")

def verify_box3_setup():
    """Verify Box 3 setup"""
    checks = {
        "Web Interface": (BASE_DIR / "site" / "index.html").exists(),
        "Plugin Manifests": (BASE_DIR / "site" / "ai-plugin.json").exists(),
        "Configuration Files": (BASE_DIR / "config" / "box3_exports.json").exists(),
        "Site Directory": (BASE_DIR / "site").exists(),
        "Box 1 Config": (BASE_DIR / "config" / "box1_exports.json").exists(),
        "Box 2 Config": (BASE_DIR / "config" / "box2_exports.json").exists(),
        "Box 2 Connection": box2_running_in_session or box2_api_accessible,
        "Gradio Availability": not GRADIO_AVAILABLE or (GRADIO_AVAILABLE and setup_gradio_interface() is not None),
        "Jupyter Availability": not JUPYTER_AVAILABLE or (JUPYTER_AVAILABLE and setup_jupyter_interface() is not None)
    }
    print("🔍 Box 3 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box3_setup()

if verification_passed:
    print("🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    print("✅ ALL SYSTEMS VERIFIED AND READY!")
    print("🚀 LAUNCH OPTIONS:")
    print(" • Jupyter Interface: launch_system('jupyter')")
    print(" • Gradio Interface: launch_system('gradio')")
    print(" • API Server Only: launch_system('api')")
    print(" • ALL Interfaces: launch_system('all')")
    print(f"🌍 URLs:")
    print(f" • Main API: {public_url}")
    print(f" • Dashboard: {dashboard_url}")
    print(f" • Web Interface: {public_url}/site/")
    print(f" • API Docs: {public_url}/docs")
    print(f"📁 System Directories:")
    print(f" • Base: {BASE_DIR}")
    print(f" • Workspace: {WORKSPACE_DIR}")
    print(f" • Logs: {BASE_DIR / 'logs'}")
    print(f" • Site: {BASE_DIR / 'site'}")
    print(f"🔧 System Status:")
    print(f" • Environment: {'Google Colab' if IS_COLAB else 'Local'}")
    print(f" • Box 2 Status: {'Integrated' if box2_running_in_session else ('Proxying' if box2_api_accessible else 'Unavailable')}")
    print(f" • Ollama Model: {ollama_model}")
    print(f" • Gradio Ready: {'Yes' if GRADIO_AVAILABLE else 'No'}")
    print(f" • Jupyter Ready: {'Yes' if JUPYTER_AVAILABLE else 'No'}")
    print("=" * 80)
    print("🔄 Box 3 is ready for launch!")
else:
    print("❌ BOX 3 SETUP HAD ISSUES!")
    print("Please check the errors above.")

# --- Launch Functions ---
def launch_system(mode: str = "jupyter"):
    """
    Launch the system in different modes.
    Modes: 'jupyter', 'gradio', 'api', 'all'
    """
    global integrated_app # Use the app created earlier

    if mode == "jupyter":
        if JUPYTER_AVAILABLE:
            print("📓 Launching Jupyter interface...")
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_ui()
            else:
                print("❌ Failed to setup Jupyter interface.")
        else:
            print("❌ Jupyter is not available in this environment.")

    elif mode == "gradio":
        if GRADIO_AVAILABLE:
            print("🎨 Launching Gradio interface...")
            demo = setup_gradio_interface()
            if demo:
                demo.launch(share=True, server_name="0.0.0.0", server_port=7860)
                print(f"✅ Gradio launched. Access at: http://localhost:7860 (or the public Gradio link if shared)")
            else:
                print("❌ Failed to setup Gradio interface.")
        else:
            print("❌ Gradio is not available.")

    elif mode == "api":
        print("🌐 Launching FastAPI server (Box 3)...")
        uvicorn.run(integrated_app, host="0.0.0.0", port=8000)
        print(f"✅ FastAPI server launched. Access at: {public_url}")

    elif mode == "all":
        print("🔄 Launching all interfaces...")
        def run_api():
             uvicorn.run(integrated_app, host="0.0.0.0", port=8000)

        api_thread = threading.Thread(target=run_api)
        api_thread.daemon = True
        api_thread.start()
        print(f"✅ API Server started in background on port 8000")

        time.sleep(2)

        if JUPYTER_AVAILABLE:
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_thread = threading.Thread(target=jupyter_ui)
                jupyter_thread.start()
                print("✅ Jupyter interface launched")
            else:
                 print("⚠️ Jupyter interface setup failed")

        if GRADIO_AVAILABLE:
            demo = setup_gradio_interface()
            if demo:
                def run_gradio():
                    demo.launch(share=True, server_name="0.0.0.0", server_port=7860, prevent_thread_lock=True)
                    demo.block_thread()

                gradio_thread = threading.Thread(target=run_gradio)
                gradio_thread.daemon = True
                gradio_thread.start()
                print("✅ Gradio interface launched")
                time.sleep(3)
            else:
                 print("⚠️ Gradio interface setup failed")

        print("🎉 All requested interfaces started!")
        print(f"🌍 API: {public_url}")
        print(f"📊 Dashboard: {dashboard_url} (if applicable)")
        if GRADIO_AVAILABLE:
            print(f"🎨 Gradio: Check output above or http://localhost:7860")
        print("ℹ️ Main thread will now idle. Stop with KeyboardInterrupt (Ctrl+C).")

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("🛑 Shutting down all services...")

    else:
        print(f"❌ Unknown mode: {mode}")
        print("Available modes: jupyter, gradio, api, all")
# --- Direct Chat Interface ---
chat_input = widgets.Text(
    value='',
    placeholder='Ask the agent anything...',
    description='Chat:',
    layout=widgets.Layout(width='100%')
)

send_chat_button = widgets.Button(
    description='Send Message',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

chat_output = widgets.Output(layout={'border': '1px solid #aaa', 'height': '300px', 'overflow_y': 'auto'}) # Slightly taller for better visibility

def on_chat_send(_):
    message = chat_input.value.strip()
    if not message:
        return
    with chat_output:
        # Display user message
        print(f"\n[You]: {message}")
        # Indicate that processing is starting
        print("[Agent]: Thinking... ", end='') # Print without newline

    try:
        # Call the agent's run_task method (which uses the Planner->Coder->Reviewer workflow)
        response_dict = agent.run_task(goal=message, context=None)

        # Extract the final, user-facing output from the agent's response dictionary
        final_reply = response_dict.get('final_output', '[Agent completed task but provided no final summary.]')

        with chat_output:
            # Overwrite the "Thinking..." placeholder with the actual response
            # We need to clear the line and print the full response
            # A simple way in Jupyter is to just print the final part, assuming the previous print handled the user message and initial "Thinking..."
            # Let's refine the output:
            print() # Move to the next line after "Thinking..."
            print(f"[Agent]: {final_reply}")
            print("-" * 20) # Separator

    except Exception as e:
        with chat_output:
            print() # Move to next line if "Thinking..." was printed
            print(f"[System Error]: {str(e)}")
    finally:
        # Clear the input field regardless of success or failure
        chat_input.value = ''

send_chat_button.on_click(on_chat_send)

# Display the chat interface components
display(HTML("<h4 style='margin-top: 20px;'>🧠 Direct Agent Chat (via Action Agent Team)</h4>"))
display(widgets.HBox([chat_input, send_chat_button]))
display(chat_output)

# Auto-launch Jupyter interface if in a Jupyter environment and script run directly
if IS_COLAB or ('ipykernel' in sys.modules):
    print("📓 Auto-launching Jupyter interface...")
    launch_system('jupyter')

print("✅ Box 3 initialization complete!")
print("🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)")

# Export launch function
__all__ = ['launch_system']


✅ Box 3 initialization complete!
🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 3: Server Launch and GUI - v7.0.x (Updated for Ollama-powered Box 2 with Chat & Agent Selector) ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - FastAPI server launch with Uvicorn (Integrates Box 2 or runs proxy)                                   ║
# ║ - Multi-interface support: Jupyter, Gradio                                                          ║
# ║ - Plugin manifests for AI integration (Claude, OpenAI)                                                  ║
# ║ - System integration and monitoring                                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 3: Initializing Server Launch and GUI Systems (Updated for Ollama-powered Box 2 with Chat & Agent Selector)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📥 Step 1: Loading configurations from previous boxes (Updated Paths)...")

# --- Load Box 1 and Box 2 configurations ---
try:
    # Standardized config directory path from updated Box 1
    config_dir = Path("/content/drive/MyDrive/UnifiedManusSystem/config")
    # Fallback for local runs
    if not config_dir.exists():
        config_dir = Path("./UnifiedManusSystem/config")

    if not config_dir.exists():
        raise FileNotFoundError("Config directory not found")

    # Load Box 1 config
    box1_config_file = config_dir / "box1_exports.json"
    if box1_config_file.exists():
        with open(box1_config_file, "r") as f:
            box1_config = json.load(f)
        print("✅ Box 1 configuration loaded")
    else:
        raise FileNotFoundError("Box 1 config not found")

    # Load Box 2 config (now includes Ollama details)
    box2_config_file = config_dir / "box2_exports.json"
    if box2_config_file.exists():
        with open(box2_config_file, "r") as f:
            box2_config = json.load(f)
        print("✅ Box 2 configuration loaded")
    else:
        print("⚠️ Box 2 config not found, will use defaults/fallbacks")
        box2_config = {"tools_registered": [], "agent_session": "unknown", "api_endpoints": [], "ollama_model": "unknown"}

    # Extract configuration
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]

    tools_available = box2_config.get("tools_registered", [])
    agent_session = box2_config.get("agent_session", "unknown")
    box2_api_endpoints = box2_config.get("api_endpoints", [])
    ollama_model = box2_config.get("ollama_model", "unknown")

    print(f"📁 Base Directory: {BASE_DIR}")
    print(f"🌍 Public URL: {public_url}")
    print(f"🤖 Agent Session: {agent_session}")
    print(f"🦙 Ollama Model: {ollama_model}")
    print(f"🛠️ Tools Available: {len(tools_available)}")

except Exception as e:
    print(f"❌ Error loading configurations: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True
    tools_available = []
    agent_session = "unknown"
    box2_api_endpoints = []
    ollama_model = "unknown"

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules for Box 3...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core web framework
import uvicorn
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse
import requests # For proxying calls to Box 2 if needed

# GUI frameworks
GRADIO_AVAILABLE = False
JUPYTER_AVAILABLE = False
try:
    import gradio as gr
    GRADIO_AVAILABLE = True
    print("✅ Gradio available")
except ImportError:
    print("⚠️ Gradio not available")

try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️ Jupyter widgets not available")

print("✅ All Box 3 modules imported successfully")

print("🔄 Step 3: Re-establishing connection to Box 2 components...")

# --- Determine if Box 2 is running ---
# Check if Box 2's app and agent are available in the current session (e.g., if this is a unified script)
box2_running_in_session = 'app' in globals() and 'agent' in globals() and 'TOOL_REGISTRY' in globals()
box2_api_accessible = False
box2_api_url = "http://localhost:8000" # Default assumption for internal calls

if box2_running_in_session:
    print("✅ Box 2 components found in current session (unified script mode)")
    box2_running = True
    # Use the in-session components
    try:
        from __main__ import app as box2_app, agent as box2_agent, TOOL_REGISTRY as box2_tool_registry
        print("🔗 Linked to in-session Box 2 components")
    except ImportError:
        print("⚠️ Could not import Box 2 components directly, using global references if available")
        # They are already in globals if the check passed
        box2_app = globals().get('app')
        box2_agent = globals().get('agent')
        box2_tool_registry = globals().get('TOOL_REGISTRY')
else:
    # Check if Box 2 server is running externally
    try:
        response = requests.get(f"{box2_api_url}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Box 2 server is accessible externally")
            box2_running = True
            box2_api_accessible = True
        else:
            print(f"⚠️ Box 2 server health check failed (Status: {response.status_code})")
            box2_running = False
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not connect to Box 2 server at {box2_api_url}: {e}")
        box2_running = False

if not box2_running:
    print("⚠️ Box 2 components not found/runnable. GUI will use fallback methods or fail gracefully.")


print("📄 Step 4: Creating plugin manifest files...")

def create_plugin_manifests():
    """Create plugin manifest files for AI integration"""
    print("📄 Creating plugin manifest files...")
    site_dir = BASE_DIR / "site"
    site_dir.mkdir(exist_ok=True)
    static_dir = site_dir / "static"
    static_dir.mkdir(exist_ok=True)

    # AI Plugin manifest (OpenAI/Claude compatible structure)
    ai_plugin_manifest = {
        "schema_version": "v1",
        "name_for_human": "Unified Manus MCP",
        "name_for_model": "unified_manus",
        "description_for_human": "Multi-agent coding assistant with comprehensive tool support, powered by Ollama LLM.",
        "description_for_model": f"A unified agent system with file operations, Python execution, package management, and multi-role thinking capabilities via the 'action_agent' tool, using the {ollama_model} model. Also supports direct chat via /mcp/chat.",
        "auth": {"type": "none"},
        "api": {"type": "openapi", "url": f"{public_url}/openapi.json"}, # Points to Box 2's OpenAPI spec
        "logo_url": f"{public_url}/site/static/logo.png", # Placeholder
        "contact_email": "support@example.com",
        "legal_info_url": f"{public_url}/site/legal.html" # Placeholder
    }
    try:
        with open(site_dir / "ai-plugin.json", "w") as f:
            json.dump(ai_plugin_manifest, f, indent=2)
        print("✅ AI Plugin manifest created")
    except Exception as e:
        print(f"❌ Failed to create AI Plugin manifest: {e}")

    # Claude-compatible manifest (YAML)
    claude_manifest = {
        "name": "unified_manus",
        "description": f"Multi-agent coding assistant with comprehensive tool support, including an 'action_agent' for complex tasks and a '/mcp/chat' endpoint for direct conversation, powered by Ollama ({ollama_model}).",
        "version": "7.0.x",
        "endpoints": {
            "tool_call": f"{public_url}/mcp/tools/call",
            "tool_list": f"{public_url}/mcp/tools/list",
            "chat": f"{public_url}/mcp/chat" # Include the new chat endpoint
        },
        "capabilities": ["file_operations", "python_execution", "package_management", "agent_thinking", "memory_management", "chat"]
    }
    try:
        import yaml # Should be available as installed by updated Box 1
        with open(site_dir / "claude.yaml", "w") as f:
            yaml.dump(claude_manifest, f, default_flow_style=False)
        print("✅ Claude manifest (YAML) created")
    except ImportError:
        print("⚠️ PyYAML not found, skipping Claude manifest YAML creation.")
    except Exception as e:
        print(f"❌ Failed to create Claude manifest: {e}")

    # Simple index.html for /site/
    index_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Unified Manus System (Ollama)</title>
</head>
<body>
    <h1>🤖 Unified Manus MCP System v7.0.x (Ollama-powered)</h1>
    <p>Multi-Agent Coding Assistant</p>
    <ul>
        <li><a href="/docs">API Documentation (FastAPI)</a></li>
        <li><a href="/redoc">API Documentation (ReDoc)</a></li>
        <li>Agent Session: {agent_session}</li>
        <li>Ollama Model: {ollama_model}</li>
        <li>Tools Available: {len(tools_available)}</li>
        <li>Public URL: <a href="{public_url}">{public_url}</a></li>
    </ul>
</body>
</html>
"""
    try:
        with open(site_dir / "index.html", "w") as f:
            f.write(index_content)
        print("✅ Basic site index.html created")
    except Exception as e:
        print(f"❌ Failed to create site index.html: {e}")

create_plugin_manifests()


print("🎨 Step 5: Setting up Gradio and Jupyter interfaces...")

# --- Gradio Interface ---
def setup_gradio_interface():
    """Setup Gradio interface"""
    if not GRADIO_AVAILABLE:
        print("⚠️ Gradio not available, skipping Gradio interface")
        return None

    print("🎨 Setting up Gradio interface...")

    def run_agent_task(task, context):
        """Run a task through the agent"""
        try:
            if box2_running_in_session:
                # Direct call to in-session agent
                result_dict = box2_agent.run_task(task, context)
                if result_dict.get("status") == "success":
                     return result_dict.get("final_output", "No final output found.")
                else:
                     return f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}"
            elif box2_api_accessible:
                 payload = {"task": task, "context": context}
                 response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                 if response.status_code == 200:
                     result_dict = response.json()
                     if result_dict.get("status") == "success":
                          return result_dict.get("final_output", "No final output found in API response.")
                     else:
                          return f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}"
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Agent Core) is not available."
        except Exception as e:
            return f"❌ Failed to run agent task: {str(e)}"

    def list_tools():
        """List available tools"""
        try:
             if box2_running_in_session:
                 from __main__ import TOOL_REGISTRY
                 tools_text = f"🛠️ Available Tools ({len(TOOL_REGISTRY)}):\n"
                 for name, func in TOOL_REGISTRY.items():
                     desc = func.__doc__.split('\n')[0] if func.__doc__ else "No description"
                     tools_text += f"• {name}: {desc}\n"
                 return tools_text
             elif box2_api_accessible:
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", timeout=10)
                 if response.status_code == 200:
                     result = response.json()
                     tools_text = f"🛠️ Available Tools ({result['count']}):\n"
                     for tool in result['tools']:
                         tools_text += f"• {tool['name']}: {tool['description']}\n"
                     return tools_text
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
             else:
                  return "❌ Box 2 (Tool Registry) is not available."
        except Exception as e:
            return f"❌ Failed to fetch tools: {str(e)}"

    def execute_tool(tool_name, tool_input):
        """Execute a specific tool"""
        try:
            if isinstance(tool_input, str) and tool_input.strip():
                try:
                    input_data = json.loads(tool_input)
                except json.JSONDecodeError:
                    input_data = {"content": tool_input}
            else:
                input_data = {}

            if box2_running_in_session:
                from __main__ import TOOL_REGISTRY
                if tool_name in TOOL_REGISTRY:
                    result = TOOL_REGISTRY[tool_name](**input_data)
                    if isinstance(result, dict):
                        return json.dumps(result, indent=2)
                    return str(result)
                else:
                    return f"❌ Tool '{tool_name}' not found."
            elif box2_api_accessible:
                payload = {"tool_name": tool_name, "tool_input": input_data}
                response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                if response.status_code == 200:
                    res_data = response.json()
                    result_content = res_data.get("result", "No result field")
                    if isinstance(result_content, dict):
                        return json.dumps(result_content, indent=2)
                    return str(result_content)
                else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Tool Execution) is not available."
        except Exception as e:
            return f"❌ Failed to execute tool: {str(e)}"

    # --- New Chat Functionality for Gradio ---
    def chat_with_model(message_history):
        """
        Chat with the model via the new /mcp/chat endpoint.
        Gradio's ChatInterface passes message_history as a list of [user_msg, bot_msg, user_msg, ...]
        We need to convert it to the format expected by /mcp/chat: [{"role": "...", "content": "..."}]
        """
        # Convert Gradio history to Ollama format
        ollama_messages = []
        for i, msg in enumerate(message_history):
            if i % 2 == 0: # User message
                ollama_messages.append({"role": "user", "content": msg})
            else: # Bot message
                ollama_messages.append({"role": "assistant", "content": msg})

        if box2_running_in_session and box2_api_accessible:
            # Prefer direct API call for streaming
            try:
                payload = {"messages": ollama_messages}
                # Note: Streaming responses from external APIs to Gradio chatbot can be complex.
                # A simpler approach is to make a non-streaming call.
                response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120)
                if response.status_code == 200:
                    # The /mcp/chat endpoint returns a stream, but for simplicity in Gradio,
                    # we can aggregate the final response if it's not truly streaming here.
                    # Or, if it returns a final aggregated response, use that.
                    # This depends on the exact implementation of the endpoint.
                    # Let's assume it returns a final message in a standard format for now.
                    # Placeholder logic:
                    data = response.json()
                    # This part needs adjustment based on how /mcp/chat final response is structured.
                    # Placeholder logic:
                    if "final_message" in
                        return data["final_message"]
                    elif "content" in data:
                         return data["content"]
                    else:
                        return "Received response, but format unclear."
                else:
                    return f"❌ Chat API Error ({response.status_code}): {response.text}"
            except Exception as e:
                 return f"❌ Chat API Call Failed: {str(e)}"
        elif box2_running_in_session:
            # If only running in session, we'd need a way to call the Ollama chat function directly
            # and handle streaming. This is more complex in Gradio without async support in this context.
            # Fallback: Use a simple non-streaming direct call (if such a function exists or is adapted).
            # For now, indicate it's not fully supported via direct call in this GUI setup.
            return "⚠️ Direct chat not fully implemented in this GUI mode. Use API or Jupyter GUI for full chat."
        else:
             return "❌ Box 2 Chat API is not accessible."

    with gr.Blocks(title="Unified Manus MCP System (Ollama)", theme=gr.themes.Default()) as demo:
        gr.Markdown("# 🤖 Unified Manus MCP System v7.0.x (Ollama-powered)")
        gr.Markdown("Multi-Agent Coding Assistant with LLM Capabilities")

        with gr.Tab("Agent Tasks"):
            with gr.Row():
                with gr.Column():
                    task_input = gr.Textbox(label="Task", placeholder="Enter a task for the agent...")
                    context_input = gr.Textbox(label="Context (optional)", placeholder="Additional context...", lines=3)
                    run_btn = gr.Button("Run Task with Action Agent", variant="primary")
                with gr.Column():
                    task_output = gr.Textbox(label="Agent Output", lines=20, max_lines=30, show_copy_button=True)
            run_btn.click(fn=run_agent_task, inputs=[task_input, context_input], outputs=task_output)

        with gr.Tab("Tool Execution"):
            with gr.Row():
                with gr.Column():
                    tool_name_input = gr.Dropdown(label="Tool Name", choices=tools_available if tools_available else [], allow_custom_value=True)
                    tool_input_input = gr.Textbox(label="Tool Input (JSON)", placeholder='{"file_path": "test.txt", "content": "Hello"}', lines=5)
                    exec_tool_btn = gr.Button("Execute Tool", variant="secondary")
                with gr.Column():
                    tool_output = gr.Textbox(label="Tool Output", lines=15, max_lines=20, show_copy_button=True)
            exec_tool_btn.click(fn=execute_tool, inputs=[tool_name_input, tool_input_input], outputs=tool_output)

        with gr.Tab("Direct Chat (via /mcp/chat)"):
             # Gradio's ChatInterface is a convenient way to handle chat
             chatbot = gr.Chatbot(label="Conversation")
             msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
             clear_chat = gr.Button("Clear Chat")

             def respond(message, chat_history):
                 # Append user message to history
                 chat_history.append((message, None))
                 # Get response from chat function
                 bot_message = chat_with_model([item for sublist in chat_history for item in sublist if item is not None])
                 # Update the last entry in history with the bot's response
                 chat_history[-1] = (message, bot_message)
                 return "", chat_history

             msg.submit(respond, [msg, chatbot], [msg, chatbot])
             clear_chat.click(fn=lambda: ([], []), inputs=[], outputs=[chatbot, msg], queue=False)


        with gr.Tab("System Info"):
            with gr.Row():
                tools_btn = gr.Button("Refresh Tool List")
                tools_output = gr.Textbox(label="Available Tools", lines=15, max_lines=20)
                tools_btn.click(fn=list_tools, outputs=tools_output)

            info_text = (
                f"**System Information:**\n"
                f"- Public API URL: {public_url}\n"
                f"- Dashboard URL: {dashboard_url}\n"
                f"- Agent Session: {agent_session}\n"
                f"- Ollama Model: {ollama_model}\n"
                f"- Base Directory: {BASE_DIR}\n"
                f"- Workspace Directory: {WORKSPACE_DIR}\n"
                f"- Tools Available: {len(tools_available)}\n"
                f"- Box 2 Status: {'Integrated/Running' if box2_running else 'Not Accessible'}"
            )
            gr.Markdown(info_text)

    return demo

# --- Jupyter Interface ---
def setup_jupyter_interface():
    """Setup Jupyter widget interface"""
    if not JUPYTER_AVAILABLE:
        print("⚠️ Jupyter widgets not available, skipping Jupyter interface")
        return None

    print("📓 Setting up Jupyter interface...")

    # --- Jupyter UI Logic ---
    def create_jupyter_ui():
        clear_output(wait=True)

        # --- UI Elements ---
        task_input = widgets.Text(
            value='',
            placeholder='Enter a task for the agent (e.g., Write a Python script to calculate Fibonacci numbers)',
            description='Task:',
            disabled=False,
            layout=widgets.Layout(width='100%')
        )

        context_input = widgets.Textarea(
            value='',
            placeholder='Optional context for the task',
            description='Context:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        run_button = widgets.Button(
            description="Run Task with Action Agent",
            button_style='success',
            tooltip='Execute the task using the internal Ollama-powered ManusAgent',
            icon='play'
        )

        tool_name_dropdown = widgets.Dropdown(
            options=tools_available if tools_available else ['No tools loaded'],
            value=tools_available[0] if tools_available else 'No tools loaded',
            description='Tool:',
            disabled=not tools_available,
        )

        tool_input_textarea = widgets.Textarea(
            value='{}',
            placeholder='Enter tool input as JSON (e.g., {"file_path": "test.txt", "content": "Hello"})',
            description='Input (JSON):',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        tool_run_button = widgets.Button(
            description="Execute Tool",
            button_style='info',
            tooltip='Run the selected tool with the provided input',
            icon='cogs'
        )

        clear_button = widgets.Button(
            description="Clear Output",
            button_style='warning',
            tooltip='Clear the output areas below',
            icon='eraser'
        )

        output_area = widgets.Output(
            layout=widgets.Layout(height='400px', border='1px solid black', overflow='auto', padding='10px')
        )

        # --- Chat Elements ---
        chat_history_area = widgets.Output(
             layout=widgets.Layout(height='300px', border='1px solid blue', overflow='auto', padding='10px', background_color='#f0f8ff')
        )
        chat_input = widgets.Textarea(
            value='',
            placeholder='Type your message here for direct chat...',
            description='Chat:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='80px')
        )
        # --- Agent Selection Dropdown for Chat ---
        chat_agent_selector = widgets.Dropdown(
            options=[
                ("Direct Chat with Ollama Model", "direct_chat"),
                ("Chat via Action Agent Team", "action_agent")
            ],
            value="action_agent", # Default to action agent
            description="Chat Agent:",
            disabled=False,
        )
        chat_send_button = widgets.Button(
            description="Send Message",
            button_style='primary',
            tooltip='Send message to the selected agent',
            icon='paper-plane'
        )
        chat_clear_button = widgets.Button(
            description="Clear Chat",
            button_style='warning',
            tooltip='Clear the chat history',
            icon='trash'
        )

        # --- Event Handlers ---
        def on_run_task(b):
            task = task_input.value
            context = context_input.value
            if not task:
                with output_area:
                    print("⚠️ Please enter a task.")
                return

            with output_area:
                clear_output(wait=True)
                print(f"🚀 Running task: '{task}'")
                if context:
                    print(f"   Context: '{context}'")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        result_dict = box2_agent.run_task(task, context)
                        if result_dict.get("status") == "success":
                             print("✅ Task completed!")
                             print("\n📝 Plan:")
                             print(result_dict.get("plan", "N/A"))
                             print("\n💻 Generated Code:")
                             print(result_dict.get("code", "N/A"))
                             print("\n🔍 Review:")
                             print(result_dict.get("review", "N/A"))
                             print("\n--- Final Output ---")
                             print(result_dict.get("final_output", "N/A"))
                        else:
                             print(f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}")
                    elif box2_api_accessible:
                         payload = {"task": task, "context": context}
                         response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                         if response.status_code == 200:
                             result_dict = response.json()
                             if result_dict.get("status") == "success":
                                  print("✅ Task completed!")
                                  print("\n📝 Plan:")
                                  print(result_dict.get("plan", "N/A"))
                                  print("\n💻 Generated Code:")
                                  print(result_dict.get("code", "N/A"))
                                  print("\n🔍 Review:")
                                  print(result_dict.get("review", "N/A"))
                                  print("\n--- Final Output ---")
                                  print(result_dict.get("final_output", "N/A"))
                             else:
                                  print(f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}")
                         else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Agent Core) is not accessible.")
                except Exception as e:
                    print(f"💥 ERROR running task: {e}")
                    traceback.print_exc()

        def on_execute_tool(b):
            tool_name = tool_name_dropdown.value
            tool_input_str = tool_input_textarea.value

            if tool_name == 'No tools loaded':
                 with output_area:
                     print("⚠️ No tools are available to execute.")
                 return

            try:
                if tool_input_str.strip():
                    tool_input_data = json.loads(tool_input_str)
                else:
                    tool_input_data = {}
            except json.JSONDecodeError as e:
                 with output_area:
                     print(f"❌ Invalid JSON input for tool: {e}")
                 return

            with output_area:
                clear_output(wait=True)
                print(f"🔧 Executing tool: '{tool_name}'")
                print(f"   Input: {json.dumps(tool_input_data, indent=2)}")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        if tool_name in box2_tool_registry:
                            result = box2_tool_registry[tool_name](**tool_input_data)
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result, indent=2) if isinstance(result, dict) else str(result))
                        else:
                            print(f"❌ Tool '{tool_name}' not found in registry.")
                    elif box2_api_accessible:
                        payload = {"tool_name": tool_name, "tool_input": tool_input_data}
                        response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                        if response.status_code == 200:
                            res_data = response.json()
                            result_content = res_data.get("result", "No result field")
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result_content, indent=2) if isinstance(result_content, dict) else str(result_content))
                        else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Tool Execution) is not accessible.")
                except Exception as e:
                    print(f"💥 ERROR executing tool: {e}")
                    traceback.print_exc()

        def on_clear(b):
            output_area.clear_output()

        # --- Chat Event Handlers (Revised) ---
        def on_chat_send(b):
            user_message = chat_input.value.strip()
            selected_agent = chat_agent_selector.value
            if not user_message:
                with chat_history_area:
                    print("⚠️ Please enter a message to send.")
                return

            with chat_history_area:
                print(f"[You]: {user_message}")

            try:
                if selected_agent == "direct_chat" and box2_api_accessible:
                    # Call the new /mcp/chat endpoint for direct conversation
                    messages = [{"role": "user", "content": user_message}]
                    payload = {"messages": messages}
                    response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120, stream=True)
                    if response.status_code == 200:
                        full_response = ""
                        with chat_history_area:
                            print(f"[Assistant ({ollama_model})]: ", end="", flush=True)
                        for line in response.iter_lines():
                            if line:
                                try:
                                    chunk_data = json.loads(line)
                                    if chunk_data.get("event") == "message":
                                        content = chunk_data.get("data", {}).get("content", "")
                                        full_response += content
                                        with chat_history_area:
                                            print(content, end="", flush=True)
                                    elif chunk_data.get("event") == "end":
                                        break
                                    elif chunk_data.get("event") == "error":
                                        error_msg = chunk_data.get("data", {}).get("error", "Unknown stream error")
                                        with chat_history_area:
                                            print(f"\n❌ Stream Error: {error_msg}")
                                        break
                                except json.JSONDecodeError:
                                    with chat_history_area:
                                        print(f"\n⚠️ Malformed stream data received.")
                        with chat_history_area:
                             print() # Newline after streaming
                        chat_input.value = ''
                    else:
                         with chat_history_area:
                             print(f"\n❌ Chat API Error ({response.status_code}): {response.text}")
                         chat_input.value = ''

                elif selected_agent == "action_agent":
                    # Send message to the Action Agent Team
                    # Capture context from output_area
                    context_from_output = ""
                    with output_area:
                        # This captures the *textual* representation of what's printed in output_area.
                        # A more robust way would be to store the last result in a variable.
                        # For now, we'll try to get the last printed content.
                        # This is a limitation of capturing output from widgets.Output directly.
                        # Let's pass a simple indicator that context is requested.
                        context_from_output = "See previous output in the 'Output' section above for context."

                    with chat_history_area:
                        print(f"[Assistant (Action Agent Team)]: Thinking... ", end="", flush=True)

                    try:
                        if box2_running_in_session:
                            # Direct call to in-session agent
                            result_dict = box2_agent.run_task(goal=user_message, context=context_from_output)
                            final_reply = result_dict.get('final_output', '[Agent completed task but provided no final summary.]')
                        elif box2_api_accessible:
                            # Call Box 2 API
                            payload = {"task": user_message, "context": context_from_output}
                            api_response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                            if api_response.status_code == 200:
                                result_dict = api_response.json()
                                final_reply = result_dict.get('final_output', '[Agent completed task but provided no final summary via API.]')
                            else:
                                final_reply = f"❌ API Error contacting Action Agent: {api_response.status_code} - {api_response.text}"
                        else:
                            final_reply = "❌ Neither direct session nor API access to Box 2 is available for the Action Agent."

                        with chat_history_area:
                            print() # Newline after "Thinking..."
                            print(f"[Assistant (Action Agent Team)]: {final_reply}")
                            print("-" * 20)

                    except Exception as e:
                         with chat_history_area:
                             print() # Newline after "Thinking..."
                             print(f"[System Error (Action Agent)]: {str(e)}")

                    chat_input.value = '' # Clear input after action agent call

                else:
                    # Fallback if direct chat is selected but API is not accessible
                    with chat_history_area:
                        print(f"\n❌ Selected agent '{selected_agent}' is not available in the current mode.")
                    chat_input.value = ''

            except Exception as e:
                 with chat_history_area:
                     print(f"\n💥 ERROR sending chat message: {e}")
                 chat_input.value = '' # Clear input on general error


        def on_chat_clear(b):
            chat_history_area.clear_output()
            chat_input.value = '' # Also clear the input field

        # Assign event handlers
        run_button.on_click(on_run_task)
        tool_run_button.on_click(on_execute_tool)
        clear_button.on_click(on_clear)
        chat_send_button.on_click(on_chat_send)
        chat_clear_button.on_click(on_chat_clear)

        # --- Display Layout ---
        ui_layout = widgets.VBox([
            widgets.HTML("<h1>🤖 Unified Manus MCP System (Jupyter - Ollama)</h1>"),
            widgets.HTML("<h2>Agent Task Execution</h2>"),
            task_input,
            context_input,
            run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Tool Execution</h2>"),
            tool_name_dropdown,
            tool_input_textarea,
            tool_run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Direct Chat</h2>"),
            chat_agent_selector, # Add the agent selector dropdown
            chat_history_area,
            chat_input,
            widgets.HBox([chat_send_button, chat_clear_button]),
            widgets.HTML("<h2 style='margin-top: 20px;'>Output</h2>"),
            clear_button,
            output_area
        ])

        display(ui_layout)

    return create_jupyter_ui


print("🌐 Step 6: Setting up FastAPI server (Integrated or Proxy)...")

# --- Integrated or Proxy FastAPI App for Box 3 ---
def create_integrated_or_proxy_server():
    """Create the FastAPI app for Box 3, either integrated or proxying to Box 2"""
    app = FastAPI(
        title="Unified Manus MCP Server - Box 3 (Ollama)",
        description="Integrated server with GUI interfaces and potential proxying to Ollama-powered Box 2",
        version="7.0.x"
    )

    # CORS middleware
    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"], # Adjust for production
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    # Static files
    site_dir = BASE_DIR / "site"
    if site_dir.exists():
        app.mount("/site", StaticFiles(directory=str(site_dir)), name="site")

    # --- Root endpoint ---
    @app.get("/")
    async def root():
        """Root endpoint"""
        index_file = site_dir / "index.html"
        if index_file.exists():
            return FileResponse(str(index_file))
        else:
            return {
                "message": "Unified Manus MCP System - Box 3 (Ollama-powered)",
                "version": "7.0.x",
                "box2_status": "Integrated" if box2_running_in_session else ("Proxying" if box2_api_accessible else "Unavailable"),
                "public_url": public_url,
                "agent_session": agent_session,
                "ollama_model": ollama_model
            }

    # --- Health check ---
    @app.get("/health")
    async def health():
        """Health check endpoint"""
        box2_status = "Unavailable"
        if box2_running_in_session:
            box2_status = "Integrated"
        elif box2_api_accessible:
            try:
                b2_response = requests.get(f"{box2_api_url}/health", timeout=2)
                if b2_response.status_code == 200:
                    b2_data = b2_response.json()
                    if b2_data.get("status") == "healthy":
                        box2_status = "Accessible (Healthy)"
                    else:
                        box2_status = f"Accessible (Degraded: {b2_data.get('status')})"
                else:
                    box2_status = f"Accessible (API Error: {b2_response.status_code})"
            except:
                box2_status = "Accessible (Health Check Failed)"

        return {
            "status": "healthy" if ("Healthy" in box2_status or box2_status == "Integrated") else "degraded",
            "box": 3,
            "version": "7.0.x",
            "timestamp": datetime.now().isoformat(),
            "box2_status": box2_status,
            "agent_session": agent_session,
            "ollama_model": ollama_model
        }

    # --- Proxy endpoints to Box 2 if it's running externally ---
    if not box2_running_in_session and box2_api_accessible:
        print("🔄 Setting up proxy endpoints to external Box 2...")

        @app.api_route("/mcp/tools/call", methods=["POST"])
        async def proxy_tool_call(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/tools/call", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/tools/list", methods=["GET"])
        async def proxy_tool_list(request: Request):
             try:
                 params = dict(request.query_params)
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", params=params)
                 return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
             except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/action", methods=["POST"])
        async def proxy_agent_action(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/agent/action", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/memory", methods=["GET", "POST"])
        async def proxy_agent_memory(request: Request):
            try:
                url = f"{box2_api_url}/mcp/agent/memory"
                if request.method == "POST":
                    url += "/clear"
                body = await request.body() if request.method in ["POST", "PUT"] else None
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                if body:
                    response = requests.request(request.method, url, data=body, headers=headers)
                else:
                     response = requests.request(request.method, url, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        # --- Proxy the new Chat endpoint ---
        @app.api_route("/mcp/chat", methods=["POST"])
        async def proxy_chat(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                # Use requests to forward the stream
                # Note: True proxying of SSE streams is complex with standard requests.
                # A more robust solution might use `httpx` or async libraries.
                # For now, we proxy the request and return the response.
                response = requests.post(f"{box2_api_url}/mcp/chat", data=body, headers=headers, stream=True)
                # Returning a streaming response correctly requires more setup.
                # This is a simplified proxy that might not handle streams perfectly.
                # Consider using `httpx` StreamResponse for better SSE proxying.
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Chat Proxy error: {str(e)}"})


    elif box2_running_in_session:
        print("🔗 Box 2 is integrated, no proxy needed for its endpoints.")
    else:
        print("⚠️ Box 2 is not accessible, proxy endpoints will not function.")

    return app

# Create the integrated/proxy app
integrated_app = create_integrated_or_proxy_server()
print("✅ FastAPI server (Box 3) configured")


print("💾 Step 7: Saving Box 3 configuration...")

def save_box3_state():
    """Save Box 3 state"""
    state = {
        "interfaces_available": ["jupyter", "gradio", "api"],
        "web_interface_ready": True,
        "plugin_manifests_created": True,
        "launch_modes": ["jupyter", "gradio", "api", "all"],
        "timestamp": datetime.now().isoformat(),
        "box2_connection": {
            "in_session": box2_running_in_session,
            "api_accessible": box2_api_accessible,
            "status": "Integrated" if box2_running_in_session else ("Accessible" if box2_api_accessible else "Unavailable")
        },
        "gradio_available": GRADIO_AVAILABLE,
        "jupyter_available": JUPYTER_AVAILABLE,
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "agent_session": agent_session,
        "ollama_model": ollama_model
    }
    config_file = BASE_DIR / "config" / "box3_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 3 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 3 state: {e}")

save_box3_state()

print("🔍 Step 8: Final verification...")

def verify_box3_setup():
    """Verify Box 3 setup"""
    checks = {
        "Web Interface": (BASE_DIR / "site" / "index.html").exists(),
        "Plugin Manifests": (BASE_DIR / "site" / "ai-plugin.json").exists(),
        "Configuration Files": (BASE_DIR / "config" / "box3_exports.json").exists(),
        "Site Directory": (BASE_DIR / "site").exists(),
        "Box 1 Config": (BASE_DIR / "config" / "box1_exports.json").exists(),
        "Box 2 Config": (BASE_DIR / "config" / "box2_exports.json").exists(),
        "Box 2 Connection": box2_running_in_session or box2_api_accessible,
        "Gradio Availability": not GRADIO_AVAILABLE or (GRADIO_AVAILABLE and setup_gradio_interface() is not None),
        "Jupyter Availability": not JUPYTER_AVAILABLE or (JUPYTER_AVAILABLE and setup_jupyter_interface() is not None)
    }
    print("🔍 Box 3 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box3_setup()

if verification_passed:
    print("🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    print("✅ ALL SYSTEMS VERIFIED AND READY!")
    print("🚀 LAUNCH OPTIONS:")
    print(" • Jupyter Interface: launch_system('jupyter')")
    print(" • Gradio Interface: launch_system('gradio')")
    print(" • API Server Only: launch_system('api')")
    print(" • ALL Interfaces: launch_system('all')")
    print(f"🌍 URLs:")
    print(f" • Main API: {public_url}")
    print(f" • Dashboard: {dashboard_url}")
    print(f" • Web Interface: {public_url}/site/")
    print(f" • API Docs: {public_url}/docs")
    print(f"📁 System Directories:")
    print(f" • Base: {BASE_DIR}")
    print(f" • Workspace: {WORKSPACE_DIR}")
    print(f" • Logs: {BASE_DIR / 'logs'}")
    print(f" • Site: {BASE_DIR / 'site'}")
    print(f"🔧 System Status:")
    print(f" • Environment: {'Google Colab' if IS_COLAB else 'Local'}")
    print(f" • Box 2 Status: {'Integrated' if box2_running_in_session else ('Proxying' if box2_api_accessible else 'Unavailable')}")
    print(f" • Ollama Model: {ollama_model}")
    print(f" • Gradio Ready: {'Yes' if GRADIO_AVAILABLE else 'No'}")
    print(f" • Jupyter Ready: {'Yes' if JUPYTER_AVAILABLE else 'No'}")
    print("=" * 80)
    print("🔄 Box 3 is ready for launch!")
else:
    print("❌ BOX 3 SETUP HAD ISSUES!")
    print("Please check the errors above.")

# --- Launch Functions ---
def launch_system(mode: str = "jupyter"):
    """
    Launch the system in different modes.
    Modes: 'jupyter', 'gradio', 'api', 'all'
    """
    global integrated_app # Use the app created earlier

    if mode == "jupyter":
        if JUPYTER_AVAILABLE:
            print("📓 Launching Jupyter interface...")
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_ui()
            else:
                print("❌ Failed to setup Jupyter interface.")
        else:
            print("❌ Jupyter is not available in this environment.")

    elif mode == "gradio":
        if GRADIO_AVAILABLE:
            print("🎨 Launching Gradio interface...")
            demo = setup_gradio_interface()
            if demo:
                demo.launch(share=True, server_name="0.0.0.0", server_port=7860)
                print(f"✅ Gradio launched. Access at: http://localhost:7860 (or the public Gradio link if shared)")
            else:
                print("❌ Failed to setup Gradio interface.")
        else:
            print("❌ Gradio is not available.")

    elif mode == "api":
        print("🌐 Launching FastAPI server (Box 3)...")
        uvicorn.run(integrated_app, host="0.0.0.0", port=8000)
        print(f"✅ FastAPI server launched. Access at: {public_url}")

    elif mode == "all":
        print("🔄 Launching all interfaces...")
        def run_api():
             uvicorn.run(integrated_app, host="0.0.0.0", port=8000)

        api_thread = threading.Thread(target=run_api)
        api_thread.daemon = True
        api_thread.start()
        print(f"✅ API Server started in background on port 8000")

        time.sleep(2)

        if JUPYTER_AVAILABLE:
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_thread = threading.Thread(target=jupyter_ui)
                jupyter_thread.start()
                print("✅ Jupyter interface launched")
            else:
                 print("⚠️ Jupyter interface setup failed")

        if GRADIO_AVAILABLE:
            demo = setup_gradio_interface()
            if demo:
                def run_gradio():
                    demo.launch(share=True, server_name="0.0.0.0", server_port=7860, prevent_thread_lock=True)
                    demo.block_thread()

                gradio_thread = threading.Thread(target=run_gradio)
                gradio_thread.daemon = True
                gradio_thread.start()
                print("✅ Gradio interface launched")
                time.sleep(3)
            else:
                 print("⚠️ Gradio interface setup failed")

        print("🎉 All requested interfaces started!")
        print(f"🌍 API: {public_url}")
        print(f"📊 Dashboard: {dashboard_url} (if applicable)")
        if GRADIO_AVAILABLE:
            print(f"🎨 Gradio: Check output above or http://localhost:7860")
        print("ℹ️ Main thread will now idle. Stop with KeyboardInterrupt (Ctrl+C).")

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("🛑 Shutting down all services...")

    else:
        print(f"❌ Unknown mode: {mode}")
        print("Available modes: jupyter, gradio, api, all")


# Auto-launch Jupyter interface if in a Jupyter environment and script run directly
if IS_COLAB or ('ipykernel' in sys.modules):
    print("📓 Auto-launching Jupyter interface...")
    launch_system('jupyter')

print("✅ Box 3 initialization complete!")
print("🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)")

# Export launch function
__all__ = ['launch_system']


SyntaxError: invalid syntax (ipython-input-33-1426524831.py, line 388)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 3: Server Launch and GUI - v7.0.x (Updated for Ollama-powered Box 2 with Chat & Agent Selector) ║
# ║                                                                                                         ║
# ╟────────────────────────────────────── CORE FEATURES ────────────────────────────────────────────────────╢
# ║ - FastAPI server launch with Uvicorn (Integrates Box 2 or runs proxy)                                   ║
# ║ - Multi-interface support: Jupyter, Gradio                                                          ║
# ║ - Plugin manifests for AI integration (Claude, OpenAI)                                                  ║
# ║ - System integration and monitoring                                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 3: Initializing Server Launch and GUI Systems (Updated for Ollama-powered Box 2 with Chat & Agent Selector)...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any

print("📥 Step 1: Loading configurations from previous boxes (Updated Paths)...")

# --- Load Box 1 and Box 2 configurations ---
try:
    # Standardized config directory path from updated Box 1
    config_dir = Path("/content/drive/MyDrive/UnifiedManusSystem/config")
    # Fallback for local runs
    if not config_dir.exists():
        config_dir = Path("./UnifiedManusSystem/config")

    if not config_dir.exists():
        raise FileNotFoundError("Config directory not found")

    # Load Box 1 config
    box1_config_file = config_dir / "box1_exports.json"
    if box1_config_file.exists():
        with open(box1_config_file, "r") as f:
            box1_config = json.load(f)
        print("✅ Box 1 configuration loaded")
    else:
        raise FileNotFoundError("Box 1 config not found")

    # Load Box 2 config (now includes Ollama details)
    box2_config_file = config_dir / "box2_exports.json"
    if box2_config_file.exists():
        with open(box2_config_file, "r") as f:
            box2_config = json.load(f)
        print("✅ Box 2 configuration loaded")
    else:
        print("⚠️ Box 2 config not found, will use defaults/fallbacks")
        box2_config = {"tools_registered": [], "agent_session": "unknown", "api_endpoints": [], "ollama_model": "unknown"}

    # Extract configuration
    BASE_DIR = Path(box1_config["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
    LOG_FILE = Path(box1_config["LOG_FILE"])
    public_url = box1_config["public_url"]
    dashboard_url = box1_config["dashboard_url"]
    IS_COLAB = box1_config["IS_COLAB"]

    tools_available = box2_config.get("tools_registered", [])
    agent_session = box2_config.get("agent_session", "unknown")
    box2_api_endpoints = box2_config.get("api_endpoints", [])
    ollama_model = box2_config.get("ollama_model", "unknown")

    print(f"📁 Base Directory: {BASE_DIR}")
    print(f"🌍 Public URL: {public_url}")
    print(f"🤖 Agent Session: {agent_session}")
    print(f"🦙 Ollama Model: {ollama_model}")
    print(f"🛠️ Tools Available: {len(tools_available)}")

except Exception as e:
    print(f"❌ Error loading configurations: {e}")
    print("🔄 Using fallback configuration...")
    BASE_DIR = Path("/content/drive/MyDrive/UnifiedManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    LOG_FILE = BASE_DIR / "logs" / "manus_log.json"
    public_url = "http://localhost:8000"
    dashboard_url = "http://localhost:5000"
    IS_COLAB = True
    tools_available = []
    agent_session = "unknown"
    box2_api_endpoints = []
    ollama_model = "unknown"

# Ensure we're in the right directory
if BASE_DIR.exists():
    os.chdir(BASE_DIR)

print("📦 Step 2: Importing required modules for Box 3...")

# Apply nest_asyncio (Important for Jupyter environments)
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("🔄 nest_asyncio applied")
except ImportError:
    print("⚠️ nest_asyncio not found (might be needed in Jupyter)")
except Exception as e:
    print(f"⚠️ Error applying nest_asyncio: {e}")

# Core web framework
import uvicorn
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse
import requests # For proxying calls to Box 2 if needed

# GUI frameworks
GRADIO_AVAILABLE = False
JUPYTER_AVAILABLE = False
try:
    import gradio as gr
    GRADIO_AVAILABLE = True
    print("✅ Gradio available")
except ImportError:
    print("⚠️ Gradio not available")

try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    JUPYTER_AVAILABLE = True
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️ Jupyter widgets not available")

print("✅ All Box 3 modules imported successfully")

print("🔄 Step 3: Re-establishing connection to Box 2 components...")

# --- Determine if Box 2 is running ---
# Check if Box 2's app and agent are available in the current session (e.g., if this is a unified script)
box2_running_in_session = 'app' in globals() and 'agent' in globals() and 'TOOL_REGISTRY' in globals()
box2_api_accessible = False
box2_api_url = "http://localhost:8000" # Default assumption for internal calls

if box2_running_in_session:
    print("✅ Box 2 components found in current session (unified script mode)")
    box2_running = True
    # Use the in-session components
    try:
        from __main__ import app as box2_app, agent as box2_agent, TOOL_REGISTRY as box2_tool_registry
        print("🔗 Linked to in-session Box 2 components")
    except ImportError:
        print("⚠️ Could not import Box 2 components directly, using global references if available")
        # They are already in globals if the check passed
        box2_app = globals().get('app')
        box2_agent = globals().get('agent')
        box2_tool_registry = globals().get('TOOL_REGISTRY')
else:
    # Check if Box 2 server is running externally
    try:
        response = requests.get(f"{box2_api_url}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Box 2 server is accessible externally")
            box2_running = True
            box2_api_accessible = True
        else:
            print(f"⚠️ Box 2 server health check failed (Status: {response.status_code})")
            box2_running = False
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Could not connect to Box 2 server at {box2_api_url}: {e}")
        box2_running = False

if not box2_running:
    print("⚠️ Box 2 components not found/runnable. GUI will use fallback methods or fail gracefully.")


print("📄 Step 4: Creating plugin manifest files...")

def create_plugin_manifests():
    """Create plugin manifest files for AI integration"""
    print("📄 Creating plugin manifest files...")
    site_dir = BASE_DIR / "site"
    site_dir.mkdir(exist_ok=True)
    static_dir = site_dir / "static"
    static_dir.mkdir(exist_ok=True)

    # AI Plugin manifest (OpenAI/Claude compatible structure)
    ai_plugin_manifest = {
        "schema_version": "v1",
        "name_for_human": "Unified Manus MCP",
        "name_for_model": "unified_manus",
        "description_for_human": "Multi-agent coding assistant with comprehensive tool support, powered by Ollama LLM.",
        "description_for_model": f"A unified agent system with file operations, Python execution, package management, and multi-role thinking capabilities via the 'action_agent' tool, using the {ollama_model} model. Also supports direct chat via /mcp/chat.",
        "auth": {"type": "none"},
        "api": {"type": "openapi", "url": f"{public_url}/openapi.json"}, # Points to Box 2's OpenAPI spec
        "logo_url": f"{public_url}/site/static/logo.png", # Placeholder
        "contact_email": "support@example.com",
        "legal_info_url": f"{public_url}/site/legal.html" # Placeholder
    }
    try:
        with open(site_dir / "ai-plugin.json", "w") as f:
            json.dump(ai_plugin_manifest, f, indent=2)
        print("✅ AI Plugin manifest created")
    except Exception as e:
        print(f"❌ Failed to create AI Plugin manifest: {e}")

    # Claude-compatible manifest (YAML)
    claude_manifest = {
        "name": "unified_manus",
        "description": f"Multi-agent coding assistant with comprehensive tool support, including an 'action_agent' for complex tasks and a '/mcp/chat' endpoint for direct conversation, powered by Ollama ({ollama_model}).",
        "version": "7.0.x",
        "endpoints": {
            "tool_call": f"{public_url}/mcp/tools/call",
            "tool_list": f"{public_url}/mcp/tools/list",
            "chat": f"{public_url}/mcp/chat" # Include the new chat endpoint
        },
        "capabilities": ["file_operations", "python_execution", "package_management", "agent_thinking", "memory_management", "chat"]
    }
    try:
        import yaml # Should be available as installed by updated Box 1
        with open(site_dir / "claude.yaml", "w") as f:
            yaml.dump(claude_manifest, f, default_flow_style=False)
        print("✅ Claude manifest (YAML) created")
    except ImportError:
        print("⚠️ PyYAML not found, skipping Claude manifest YAML creation.")
    except Exception as e:
        print(f"❌ Failed to create Claude manifest: {e}")

    # Simple index.html for /site/
    index_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Unified Manus System (Ollama)</title>
</head>
<body>
    <h1>🤖 Unified Manus MCP System v7.0.x (Ollama-powered)</h1>
    <p>Multi-Agent Coding Assistant</p>
    <ul>
        <li><a href="/docs">API Documentation (FastAPI)</a></li>
        <li><a href="/redoc">API Documentation (ReDoc)</a></li>
        <li>Agent Session: {agent_session}</li>
        <li>Ollama Model: {ollama_model}</li>
        <li>Tools Available: {len(tools_available)}</li>
        <li>Public URL: <a href="{public_url}">{public_url}</a></li>
    </ul>
</body>
</html>
"""
    try:
        with open(site_dir / "index.html", "w") as f:
            f.write(index_content)
        print("✅ Basic site index.html created")
    except Exception as e:
        print(f"❌ Failed to create site index.html: {e}")

create_plugin_manifests()


print("🎨 Step 5: Setting up Gradio and Jupyter interfaces...")

# --- Gradio Interface ---
def setup_gradio_interface():
    """Setup Gradio interface"""
    if not GRADIO_AVAILABLE:
        print("⚠️ Gradio not available, skipping Gradio interface")
        return None

    print("🎨 Setting up Gradio interface...")

    def run_agent_task(task, context):
        """Run a task through the agent"""
        try:
            if box2_running_in_session:
                # Direct call to in-session agent
                result_dict = box2_agent.run_task(task, context)
                if result_dict.get("status") == "success":
                     return result_dict.get("final_output", "No final output found.")
                else:
                     return f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}"
            elif box2_api_accessible:
                 payload = {"task": task, "context": context}
                 response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                 if response.status_code == 200:
                     result_dict = response.json()
                     if result_dict.get("status") == "success":
                          return result_dict.get("final_output", "No final output found in API response.")
                     else:
                          return f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}"
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Agent Core) is not available."
        except Exception as e:
            return f"❌ Failed to run agent task: {str(e)}"

    def list_tools():
        """List available tools"""
        try:
             if box2_running_in_session:
                 from __main__ import TOOL_REGISTRY
                 tools_text = f"🛠️ Available Tools ({len(TOOL_REGISTRY)}):\n"
                 for name, func in TOOL_REGISTRY.items():
                     desc = func.__doc__.split('\n')[0] if func.__doc__ else "No description"
                     tools_text += f"• {name}: {desc}\n"
                 return tools_text
             elif box2_api_accessible:
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", timeout=10)
                 if response.status_code == 200:
                     result = response.json()
                     tools_text = f"🛠️ Available Tools ({result['count']}):\n"
                     for tool in result['tools']:
                         tools_text += f"• {tool['name']}: {tool['description']}\n"
                     return tools_text
                 else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
             else:
                  return "❌ Box 2 (Tool Registry) is not available."
        except Exception as e:
            return f"❌ Failed to fetch tools: {str(e)}"

    def execute_tool(tool_name, tool_input):
        """Execute a specific tool"""
        try:
            if isinstance(tool_input, str) and tool_input.strip():
                try:
                    input_data = json.loads(tool_input)
                except json.JSONDecodeError:
                    input_data = {"content": tool_input}
            else:
                input_data = {}

            if box2_running_in_session:
                from __main__ import TOOL_REGISTRY
                if tool_name in TOOL_REGISTRY:
                    result = TOOL_REGISTRY[tool_name](**input_data)
                    if isinstance(result, dict):
                        return json.dumps(result, indent=2)
                    return str(result)
                else:
                    return f"❌ Tool '{tool_name}' not found."
            elif box2_api_accessible:
                payload = {"tool_name": tool_name, "tool_input": input_data}
                response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                if response.status_code == 200:
                    res_data = response.json()
                    result_content = res_data.get("result", "No result field")
                    if isinstance(result_content, dict):
                        return json.dumps(result_content, indent=2)
                    return str(result_content)
                else:
                     return f"❌ API Error ({response.status_code}): {response.text}"
            else:
                 return "❌ Box 2 (Tool Execution) is not available."
        except Exception as e:
            return f"❌ Failed to execute tool: {str(e)}"

    # --- New Chat Functionality for Gradio ---
    def chat_with_model(message_history):
        """
        Chat with the model via the new /mcp/chat endpoint.
        Gradio's ChatInterface passes message_history as a list of [user_msg, bot_msg, user_msg, ...]
        We need to convert it to the format expected by /mcp/chat: [{"role": "...", "content": "..."}]
        """
        # Convert Gradio history to Ollama format
        ollama_messages = []
        for i, msg in enumerate(message_history):
            if i % 2 == 0: # User message
                ollama_messages.append({"role": "user", "content": msg})
            else: # Bot message
                ollama_messages.append({"role": "assistant", "content": msg})

        if box2_running_in_session and box2_api_accessible:
            # Prefer direct API call for streaming
            try:
                payload = {"messages": ollama_messages}
                # Note: Streaming responses from external APIs to Gradio chatbot can be complex.
                # A simpler approach is to make a non-streaming call.
                response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120)
                if response.status_code == 200:
                    # The /mcp/chat endpoint returns a stream. If we get a non-stream response here,
                    # it might be an aggregated final message or an error.
                    # Let's try to parse JSON. If it fails, return raw text.
                    try:
                        data = response.json()
                        # Check for common fields in the final aggregated response
                        if "message" in data:
                            # If it's the final message structure from the streaming endpoint
                            return data.get("message", {}).get("content", "Received message, content unclear (format 1).")
                        elif "content" in data:
                             # If it's a simple content response
                             return data["content"]
                        elif "final_output" in data: # Maybe the action_agent format was used somehow
                            return data["final_output"]
                        else:
                            # Return the whole JSON if structure is unknown
                            return f"Received JSON, structure unclear: {data}"
                    except json.JSONDecodeError:
                        # If response isn't JSON, return text content
                        return response.text or "Received response, but it was empty."
                else:
                    return f"❌ Chat API Error ({response.status_code}): {response.text}"
            except Exception as e:
                 return f"❌ Chat API Call Failed: {str(e)}"
        elif box2_running_in_session:
            # If only running in session, we'd need a way to call the Ollama chat function directly
            # and handle streaming. This is more complex in Gradio without async support in this context.
            # Fallback: Use a simple non-streaming direct call (if such a function exists or is adapted).
            # For now, indicate it's not fully supported via direct call in this GUI setup.
            return "⚠️ Direct chat not fully implemented in this GUI mode. Use API or Jupyter GUI for full chat."
        else:
             return "❌ Box 2 Chat API is not accessible."

    with gr.Blocks(title="Unified Manus MCP System (Ollama)", theme=gr.themes.Default()) as demo:
        gr.Markdown("# 🤖 Unified Manus MCP System v7.0.x (Ollama-powered)")
        gr.Markdown("Multi-Agent Coding Assistant with LLM Capabilities")

        with gr.Tab("Agent Tasks"):
            with gr.Row():
                with gr.Column():
                    task_input = gr.Textbox(label="Task", placeholder="Enter a task for the agent...")
                    context_input = gr.Textbox(label="Context (optional)", placeholder="Additional context...", lines=3)
                    run_btn = gr.Button("Run Task with Action Agent", variant="primary")
                with gr.Column():
                    task_output = gr.Textbox(label="Agent Output", lines=20, max_lines=30, show_copy_button=True)
            run_btn.click(fn=run_agent_task, inputs=[task_input, context_input], outputs=task_output)

        with gr.Tab("Tool Execution"):
            with gr.Row():
                with gr.Column():
                    tool_name_input = gr.Dropdown(label="Tool Name", choices=tools_available if tools_available else [], allow_custom_value=True)
                    tool_input_input = gr.Textbox(label="Tool Input (JSON)", placeholder='{"file_path": "test.txt", "content": "Hello"}', lines=5)
                    exec_tool_btn = gr.Button("Execute Tool", variant="secondary")
                with gr.Column():
                    tool_output = gr.Textbox(label="Tool Output", lines=15, max_lines=20, show_copy_button=True)
            exec_tool_btn.click(fn=execute_tool, inputs=[tool_name_input, tool_input_input], outputs=tool_output)

        with gr.Tab("Direct Chat (via /mcp/chat)"):
             # Gradio's ChatInterface is a convenient way to handle chat
             chatbot = gr.Chatbot(label="Conversation")
             msg = gr.Textbox(label="Your Message", placeholder="Type your message here...")
             clear_chat = gr.Button("Clear Chat")

             def respond(message, chat_history):
                 # Append user message to history
                 chat_history.append((message, None))
                 # Get response from chat function
                 bot_message = chat_with_model([item for sublist in chat_history for item in sublist if item is not None])
                 # Update the last entry in history with the bot's response
                 chat_history[-1] = (message, bot_message)
                 return "", chat_history

             msg.submit(respond, [msg, chatbot], [msg, chatbot])
             clear_chat.click(fn=lambda: ([], []), inputs=[], outputs=[chatbot, msg], queue=False)


        with gr.Tab("System Info"):
            with gr.Row():
                tools_btn = gr.Button("Refresh Tool List")
                tools_output = gr.Textbox(label="Available Tools", lines=15, max_lines=20)
                tools_btn.click(fn=list_tools, outputs=tools_output)

            info_text = (
                f"**System Information:**\n"
                f"- Public API URL: {public_url}\n"
                f"- Dashboard URL: {dashboard_url}\n"
                f"- Agent Session: {agent_session}\n"
                f"- Ollama Model: {ollama_model}\n"
                f"- Base Directory: {BASE_DIR}\n"
                f"- Workspace Directory: {WORKSPACE_DIR}\n"
                f"- Tools Available: {len(tools_available)}\n"
                f"- Box 2 Status: {'Integrated/Running' if box2_running else 'Not Accessible'}"
            )
            gr.Markdown(info_text)

    return demo

# --- Jupyter Interface ---
def setup_jupyter_interface():
    """Setup Jupyter widget interface"""
    if not JUPYTER_AVAILABLE:
        print("⚠️ Jupyter widgets not available, skipping Jupyter interface")
        return None

    print("📓 Setting up Jupyter interface...")

    # --- Jupyter UI Logic ---
    def create_jupyter_ui():
        clear_output(wait=True)

        # --- UI Elements ---
        task_input = widgets.Text(
            value='',
            placeholder='Enter a task for the agent (e.g., Write a Python script to calculate Fibonacci numbers)',
            description='Task:',
            disabled=False,
            layout=widgets.Layout(width='100%')
        )

        context_input = widgets.Textarea(
            value='',
            placeholder='Optional context for the task',
            description='Context:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        run_button = widgets.Button(
            description="Run Task with Action Agent",
            button_style='success',
            tooltip='Execute the task using the internal Ollama-powered ManusAgent',
            icon='play'
        )

        tool_name_dropdown = widgets.Dropdown(
            options=tools_available if tools_available else ['No tools loaded'],
            value=tools_available[0] if tools_available else 'No tools loaded',
            description='Tool:',
            disabled=not tools_available,
        )

        tool_input_textarea = widgets.Textarea(
            value='{}',
            placeholder='Enter tool input as JSON (e.g., {"file_path": "test.txt", "content": "Hello"})',
            description='Input (JSON):',
            disabled=False,
            layout=widgets.Layout(width='100%', height='100px')
        )

        tool_run_button = widgets.Button(
            description="Execute Tool",
            button_style='info',
            tooltip='Run the selected tool with the provided input',
            icon='cogs'
        )

        clear_button = widgets.Button(
            description="Clear Output",
            button_style='warning',
            tooltip='Clear the output areas below',
            icon='eraser'
        )

        output_area = widgets.Output(
            layout=widgets.Layout(height='400px', border='1px solid black', overflow='auto', padding='10px')
        )

        # --- Chat Elements ---
        chat_history_area = widgets.Output(
             layout=widgets.Layout(height='300px', border='1px solid blue', overflow='auto', padding='10px', background_color='#f0f8ff')
        )
        chat_input = widgets.Textarea(
            value='',
            placeholder='Type your message here for direct chat...',
            description='Chat:',
            disabled=False,
            layout=widgets.Layout(width='100%', height='80px')
        )
        # --- Agent Selection Dropdown for Chat ---
        chat_agent_selector = widgets.Dropdown(
            options=[
                ("Direct Chat with Ollama Model", "direct_chat"),
                ("Chat via Action Agent Team", "action_agent")
            ],
            value="action_agent", # Default to action agent
            description="Chat Agent:",
            disabled=False,
        )
        chat_send_button = widgets.Button(
            description="Send Message",
            button_style='primary',
            tooltip='Send message to the selected agent',
            icon='paper-plane'
        )
        chat_clear_button = widgets.Button(
            description="Clear Chat",
            button_style='warning',
            tooltip='Clear the chat history',
            icon='trash'
        )

        # --- Event Handlers ---
        def on_run_task(b):
            task = task_input.value
            context = context_input.value
            if not task:
                with output_area:
                    print("⚠️ Please enter a task.")
                return

            with output_area:
                clear_output(wait=True)
                print(f"🚀 Running task: '{task}'")
                if context:
                    print(f"   Context: '{context}'")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        result_dict = box2_agent.run_task(task, context)
                        if result_dict.get("status") == "success":
                             print("✅ Task completed!")
                             print("\n📝 Plan:")
                             print(result_dict.get("plan", "N/A"))
                             print("\n💻 Generated Code:")
                             print(result_dict.get("code", "N/A"))
                             print("\n🔍 Review:")
                             print(result_dict.get("review", "N/A"))
                             print("\n--- Final Output ---")
                             print(result_dict.get("final_output", "N/A"))
                        else:
                             print(f"❌ Agent Error: {result_dict.get('message', 'Unknown error')}")
                    elif box2_api_accessible:
                         payload = {"task": task, "context": context}
                         response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                         if response.status_code == 200:
                             result_dict = response.json()
                             if result_dict.get("status") == "success":
                                  print("✅ Task completed!")
                                  print("\n📝 Plan:")
                                  print(result_dict.get("plan", "N/A"))
                                  print("\n💻 Generated Code:")
                                  print(result_dict.get("code", "N/A"))
                                  print("\n🔍 Review:")
                                  print(result_dict.get("review", "N/A"))
                                  print("\n--- Final Output ---")
                                  print(result_dict.get("final_output", "N/A"))
                             else:
                                  print(f"❌ Agent API Error: {result_dict.get('message', 'Unknown error from API')}")
                         else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Agent Core) is not accessible.")
                except Exception as e:
                    print(f"💥 ERROR running task: {e}")
                    traceback.print_exc()

        def on_execute_tool(b):
            tool_name = tool_name_dropdown.value
            tool_input_str = tool_input_textarea.value

            if tool_name == 'No tools loaded':
                 with output_area:
                     print("⚠️ No tools are available to execute.")
                 return

            try:
                if tool_input_str.strip():
                    tool_input_data = json.loads(tool_input_str)
                else:
                    tool_input_data = {}
            except json.JSONDecodeError as e:
                 with output_area:
                     print(f"❌ Invalid JSON input for tool: {e}")
                 return

            with output_area:
                clear_output(wait=True)
                print(f"🔧 Executing tool: '{tool_name}'")
                print(f"   Input: {json.dumps(tool_input_data, indent=2)}")
                print("-" * 20)

                try:
                    if box2_running_in_session:
                        if tool_name in box2_tool_registry:
                            result = box2_tool_registry[tool_name](**tool_input_data)
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result, indent=2) if isinstance(result, dict) else str(result))
                        else:
                            print(f"❌ Tool '{tool_name}' not found in registry.")
                    elif box2_api_accessible:
                        payload = {"tool_name": tool_name, "tool_input": tool_input_data}
                        response = requests.post(f"{box2_api_url}/mcp/tools/call", json=payload, timeout=60)
                        if response.status_code == 200:
                            res_data = response.json()
                            result_content = res_data.get("result", "No result field")
                            print("✅ Tool executed successfully!")
                            print(json.dumps(result_content, indent=2) if isinstance(result_content, dict) else str(result_content))
                        else:
                             print(f"❌ API Error ({response.status_code}): {response.text}")
                    else:
                         print("❌ Box 2 (Tool Execution) is not accessible.")
                except Exception as e:
                    print(f"💥 ERROR executing tool: {e}")
                    traceback.print_exc()

        def on_clear(b):
            output_area.clear_output()

        # --- Chat Event Handlers (Revised) ---
        def on_chat_send(b):
            user_message = chat_input.value.strip()
            selected_agent = chat_agent_selector.value
            if not user_message:
                with chat_history_area:
                    print("⚠️ Please enter a message to send.")
                return

            with chat_history_area:
                print(f"[You]: {user_message}")

            try:
                if selected_agent == "direct_chat" and box2_api_accessible:
                    # Call the new /mcp/chat endpoint for direct conversation
                    messages = [{"role": "user", "content": user_message}]
                    payload = {"messages": messages}
                    response = requests.post(f"{box2_api_url}/mcp/chat", json=payload, timeout=120, stream=True)
                    if response.status_code == 200:
                        full_response = ""
                        with chat_history_area:
                            print(f"[Assistant ({ollama_model})]: ", end="", flush=True)
                        for line in response.iter_lines():
                            if line:
                                try:
                                    chunk_data = json.loads(line)
                                    if chunk_data.get("event") == "message":
                                        content = chunk_data.get("data", {}).get("content", "")
                                        full_response += content
                                        with chat_history_area:
                                            print(content, end="", flush=True)
                                    elif chunk_data.get("event") == "end":
                                        break
                                    elif chunk_data.get("event") == "error":
                                        error_msg = chunk_data.get("data", {}).get("error", "Unknown stream error")
                                        with chat_history_area:
                                            print(f"\n❌ Stream Error: {error_msg}")
                                        break
                                except json.JSONDecodeError:
                                    with chat_history_area:
                                        print(f"\n⚠️ Malformed stream data received.")
                        with chat_history_area:
                             print() # Newline after streaming
                        chat_input.value = ''
                    else:
                         with chat_history_area:
                             print(f"\n❌ Chat API Error ({response.status_code}): {response.text}")
                         chat_input.value = ''

                elif selected_agent == "action_agent":
                    # Send message to the Action Agent Team
                    # Attempt to capture context from output_area
                    # Note: Directly reading from widgets.Output is tricky.
                    # We pass a generic context string. A more advanced version
                    # might store the last output in a variable.
                    context_from_output = "See previous output in the 'Output' section above for context."

                    with chat_history_area:
                        print(f"[Assistant (Action Agent Team)]: Thinking... ", end="", flush=True)

                    try:
                        if box2_running_in_session:
                            # Direct call to in-session agent
                            result_dict = box2_agent.run_task(goal=user_message, context=context_from_output)
                            final_reply = result_dict.get('final_output', '[Agent completed task but provided no final summary.]')
                        elif box2_api_accessible:
                            # Call Box 2 API
                            payload = {"task": user_message, "context": context_from_output}
                            api_response = requests.post(f"{box2_api_url}/mcp/agent/action", json=payload, timeout=120)
                            if api_response.status_code == 200:
                                result_dict = api_response.json()
                                final_reply = result_dict.get('final_output', '[Agent completed task but provided no final summary via API.]')
                            else:
                                final_reply = f"❌ API Error contacting Action Agent: {api_response.status_code} - {api_response.text}"
                        else:
                            final_reply = "❌ Neither direct session nor API access to Box 2 is available for the Action Agent."

                        with chat_history_area:
                            print() # Newline after "Thinking..."
                            print(f"[Assistant (Action Agent Team)]: {final_reply}")
                            print("-" * 20)

                    except Exception as e:
                         with chat_history_area:
                             print() # Newline after "Thinking..."
                             print(f"[System Error (Action Agent)]: {str(e)}")

                    chat_input.value = '' # Clear input after action agent call

                else:
                    # Fallback if direct chat is selected but API is not accessible
                    with chat_history_area:
                        print(f"\n❌ Selected agent '{selected_agent}' is not available in the current mode.")
                    chat_input.value = ''

            except Exception as e:
                 with chat_history_area:
                     print(f"\n💥 ERROR sending chat message: {e}")
                 chat_input.value = '' # Clear input on general error


        def on_chat_clear(b):
            chat_history_area.clear_output()
            chat_input.value = '' # Also clear the input field

        # Assign event handlers
        run_button.on_click(on_run_task)
        tool_run_button.on_click(on_execute_tool)
        clear_button.on_click(on_clear)
        chat_send_button.on_click(on_chat_send)
        chat_clear_button.on_click(on_chat_clear)

        # --- Display Layout ---
        ui_layout = widgets.VBox([
            widgets.HTML("<h1>🤖 Unified Manus MCP System (Jupyter - Ollama)</h1>"),
            widgets.HTML("<h2>Agent Task Execution</h2>"),
            task_input,
            context_input,
            run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Tool Execution</h2>"),
            tool_name_dropdown,
            tool_input_textarea,
            tool_run_button,
            widgets.HTML("<h2 style='margin-top: 20px;'>Direct Chat</h2>"),
            chat_agent_selector, # Add the agent selector dropdown
            chat_history_area,
            chat_input,
            widgets.HBox([chat_send_button, chat_clear_button]),
            widgets.HTML("<h2 style='margin-top: 20px;'>Output</h2>"),
            clear_button,
            output_area
        ])

        display(ui_layout)

    return create_jupyter_ui


print("🌐 Step 6: Setting up FastAPI server (Integrated or Proxy)...")

# --- Integrated or Proxy FastAPI App for Box 3 ---
def create_integrated_or_proxy_server():
    """Create the FastAPI app for Box 3, either integrated or proxying to Box 2"""
    app = FastAPI(
        title="Unified Manus MCP Server - Box 3 (Ollama)",
        description="Integrated server with GUI interfaces and potential proxying to Ollama-powered Box 2",
        version="7.0.x"
    )

    # CORS middleware
    app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"], # Adjust for production
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )

    # Static files
    site_dir = BASE_DIR / "site"
    if site_dir.exists():
        app.mount("/site", StaticFiles(directory=str(site_dir)), name="site")

    # --- Root endpoint ---
    @app.get("/")
    async def root():
        """Root endpoint"""
        index_file = site_dir / "index.html"
        if index_file.exists():
            return FileResponse(str(index_file))
        else:
            return {
                "message": "Unified Manus MCP System - Box 3 (Ollama-powered)",
                "version": "7.0.x",
                "box2_status": "Integrated" if box2_running_in_session else ("Proxying" if box2_api_accessible else "Unavailable"),
                "public_url": public_url,
                "agent_session": agent_session,
                "ollama_model": ollama_model
            }

    # --- Health check ---
    @app.get("/health")
    async def health():
        """Health check endpoint"""
        box2_status = "Unavailable"
        if box2_running_in_session:
            box2_status = "Integrated"
        elif box2_api_accessible:
            try:
                b2_response = requests.get(f"{box2_api_url}/health", timeout=2)
                if b2_response.status_code == 200:
                    b2_data = b2_response.json()
                    if b2_data.get("status") == "healthy":
                        box2_status = "Accessible (Healthy)"
                    else:
                        box2_status = f"Accessible (Degraded: {b2_data.get('status')})"
                else:
                    box2_status = f"Accessible (API Error: {b2_response.status_code})"
            except:
                box2_status = "Accessible (Health Check Failed)"

        return {
            "status": "healthy" if ("Healthy" in box2_status or box2_status == "Integrated") else "degraded",
            "box": 3,
            "version": "7.0.x",
            "timestamp": datetime.now().isoformat(),
            "box2_status": box2_status,
            "agent_session": agent_session,
            "ollama_model": ollama_model
        }

    # --- Proxy endpoints to Box 2 if it's running externally ---
    if not box2_running_in_session and box2_api_accessible:
        print("🔄 Setting up proxy endpoints to external Box 2...")

        @app.api_route("/mcp/tools/call", methods=["POST"])
        async def proxy_tool_call(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/tools/call", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/tools/list", methods=["GET"])
        async def proxy_tool_list(request: Request):
             try:
                 params = dict(request.query_params)
                 response = requests.get(f"{box2_api_url}/mcp/tools/list", params=params)
                 return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
             except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/action", methods=["POST"])
        async def proxy_agent_action(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                response = requests.post(f"{box2_api_url}/mcp/agent/action", data=body, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        @app.api_route("/mcp/agent/memory", methods=["GET", "POST"])
        async def proxy_agent_memory(request: Request):
            try:
                url = f"{box2_api_url}/mcp/agent/memory"
                if request.method == "POST":
                    url += "/clear"
                body = await request.body() if request.method in ["POST", "PUT"] else None
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                if body:
                    response = requests.request(request.method, url, data=body, headers=headers)
                else:
                     response = requests.request(request.method, url, headers=headers)
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                return JSONResponse(status_code=500, content={"error": f"Proxy error: {str(e)}"})

        # --- Proxy the new Chat endpoint ---
        @app.api_route("/mcp/chat", methods=["POST"])
        async def proxy_chat(request: Request):
            try:
                body = await request.body()
                headers = dict(request.headers)
                headers['Content-Type'] = 'application/json'
                # Use requests to forward the stream
                # Note: True proxying of SSE streams is complex with standard requests.
                # A more robust solution might use `httpx` or async libraries.
                # For now, we proxy the request and return the response.
                response = requests.post(f"{box2_api_url}/mcp/chat", data=body, headers=headers, stream=True)
                # Returning a streaming response correctly requires more setup.
                # This is a simplified proxy that might not handle streams perfectly.
                # Consider using `httpx` StreamResponse for better SSE proxying.
                return Response(content=response.content, status_code=response.status_code, headers=dict(response.headers))
            except Exception as e:
                 return JSONResponse(status_code=500, content={"error": f"Chat Proxy error: {str(e)}"})


    elif box2_running_in_session:
        print("🔗 Box 2 is integrated, no proxy needed for its endpoints.")
    else:
        print("⚠️ Box 2 is not accessible, proxy endpoints will not function.")

    return app

# Create the integrated/proxy app
integrated_app = create_integrated_or_proxy_server()
print("✅ FastAPI server (Box 3) configured")


print("💾 Step 7: Saving Box 3 configuration...")

def save_box3_state():
    """Save Box 3 state"""
    state = {
        "interfaces_available": ["jupyter", "gradio", "api"],
        "web_interface_ready": True,
        "plugin_manifests_created": True,
        "launch_modes": ["jupyter", "gradio", "api", "all"],
        "timestamp": datetime.now().isoformat(),
        "box2_connection": {
            "in_session": box2_running_in_session,
            "api_accessible": box2_api_accessible,
            "status": "Integrated" if box2_running_in_session else ("Accessible" if box2_api_accessible else "Unavailable")
        },
        "gradio_available": GRADIO_AVAILABLE,
        "jupyter_available": JUPYTER_AVAILABLE,
        "public_url": public_url,
        "dashboard_url": dashboard_url,
        "agent_session": agent_session,
        "ollama_model": ollama_model
    }
    config_file = BASE_DIR / "config" / "box3_exports.json"
    config_file.parent.mkdir(parents=True, exist_ok=True) # Ensure config dir exists
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 3 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 3 state: {e}")

save_box3_state()

print("🔍 Step 8: Final verification...")

def verify_box3_setup():
    """Verify Box 3 setup"""
    checks = {
        "Web Interface": (BASE_DIR / "site" / "index.html").exists(),
        "Plugin Manifests": (BASE_DIR / "site" / "ai-plugin.json").exists(),
        "Configuration Files": (BASE_DIR / "config" / "box3_exports.json").exists(),
        "Site Directory": (BASE_DIR / "site").exists(),
        "Box 1 Config": (BASE_DIR / "config" / "box1_exports.json").exists(),
        "Box 2 Config": (BASE_DIR / "config" / "box2_exports.json").exists(),
        "Box 2 Connection": box2_running_in_session or box2_api_accessible,
        "Gradio Availability": not GRADIO_AVAILABLE or (GRADIO_AVAILABLE and setup_gradio_interface() is not None),
        "Jupyter Availability": not JUPYTER_AVAILABLE or (JUPYTER_AVAILABLE and setup_jupyter_interface() is not None)
    }
    print("🔍 Box 3 verification:")
    all_good = True
    for check, status in checks.items():
        status_icon = "✅" if status else "❌"
        print(f" {status_icon} {check}: {'OK' if status else 'FAILED'}")
        if not status:
            all_good = False
    return all_good

verification_passed = verify_box3_setup()

if verification_passed:
    print("🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
    print("=" * 80)
    print("✅ ALL SYSTEMS VERIFIED AND READY!")
    print("🚀 LAUNCH OPTIONS:")
    print(" • Jupyter Interface: launch_system('jupyter')")
    print(" • Gradio Interface: launch_system('gradio')")
    print(" • API Server Only: launch_system('api')")
    print(" • ALL Interfaces: launch_system('all')")
    print(f"🌍 URLs:")
    print(f" • Main API: {public_url}")
    print(f" • Dashboard: {dashboard_url}")
    print(f" • Web Interface: {public_url}/site/")
    print(f" • API Docs: {public_url}/docs")
    print(f"📁 System Directories:")
    print(f" • Base: {BASE_DIR}")
    print(f" • Workspace: {WORKSPACE_DIR}")
    print(f" • Logs: {BASE_DIR / 'logs'}")
    print(f" • Site: {BASE_DIR / 'site'}")
    print(f"🔧 System Status:")
    print(f" • Environment: {'Google Colab' if IS_COLAB else 'Local'}")
    print(f" • Box 2 Status: {'Integrated' if box2_running_in_session else ('Proxying' if box2_api_accessible else 'Unavailable')}")
    print(f" • Ollama Model: {ollama_model}")
    print(f" • Gradio Ready: {'Yes' if GRADIO_AVAILABLE else 'No'}")
    print(f" • Jupyter Ready: {'Yes' if JUPYTER_AVAILABLE else 'No'}")
    print("=" * 80)
    print("🔄 Box 3 is ready for launch!")
else:
    print("❌ BOX 3 SETUP HAD ISSUES!")
    print("Please check the errors above.")

# --- Launch Functions ---
def launch_system(mode: str = "jupyter"):
    """
    Launch the system in different modes.
    Modes: 'jupyter', 'gradio', 'api', 'all'
    """
    global integrated_app # Use the app created earlier

    if mode == "jupyter":
        if JUPYTER_AVAILABLE:
            print("📓 Launching Jupyter interface...")
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_ui()
            else:
                print("❌ Failed to setup Jupyter interface.")
        else:
            print("❌ Jupyter is not available in this environment.")

    elif mode == "gradio":
        if GRADIO_AVAILABLE:
            print("🎨 Launching Gradio interface...")
            demo = setup_gradio_interface()
            if demo:
                demo.launch(share=True, server_name="0.0.0.0", server_port=7860)
                print(f"✅ Gradio launched. Access at: http://localhost:7860 (or the public Gradio link if shared)")
            else:
                print("❌ Failed to setup Gradio interface.")
        else:
            print("❌ Gradio is not available.")

    elif mode == "api":
        print("🌐 Launching FastAPI server (Box 3)...")
        uvicorn.run(integrated_app, host="0.0.0.0", port=8000)
        print(f"✅ FastAPI server launched. Access at: {public_url}")

    elif mode == "all":
        print("🔄 Launching all interfaces...")
        def run_api():
             uvicorn.run(integrated_app, host="0.0.0.0", port=8000)

        api_thread = threading.Thread(target=run_api)
        api_thread.daemon = True
        api_thread.start()
        print(f"✅ API Server started in background on port 8000")

        time.sleep(2)

        if JUPYTER_AVAILABLE:
            jupyter_ui = setup_jupyter_interface()
            if jupyter_ui:
                jupyter_thread = threading.Thread(target=jupyter_ui)
                jupyter_thread.start()
                print("✅ Jupyter interface launched")
            else:
                 print("⚠️ Jupyter interface setup failed")

        if GRADIO_AVAILABLE:
            demo = setup_gradio_interface()
            if demo:
                def run_gradio():
                    demo.launch(share=True, server_name="0.0.0.0", server_port=7860, prevent_thread_lock=True)
                    demo.block_thread()

                gradio_thread = threading.Thread(target=run_gradio)
                gradio_thread.daemon = True
                gradio_thread.start()
                print("✅ Gradio interface launched")
                time.sleep(3)
            else:
                 print("⚠️ Gradio interface setup failed")

        print("🎉 All requested interfaces started!")
        print(f"🌍 API: {public_url}")
        print(f"📊 Dashboard: {dashboard_url} (if applicable)")
        if GRADIO_AVAILABLE:
            print(f"🎨 Gradio: Check output above or http://localhost:7860")
        print("ℹ️ Main thread will now idle. Stop with KeyboardInterrupt (Ctrl+C).")

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("🛑 Shutting down all services...")

    else:
        print(f"❌ Unknown mode: {mode}")
        print("Available modes: jupyter, gradio, api, all")


# Auto-launch Jupyter interface if in a Jupyter environment and script run directly
if IS_COLAB or ('ipykernel' in sys.modules):
    print("📓 Auto-launching Jupyter interface...")
    launch_system('jupyter')

print("✅ Box 3 initialization complete!")
print("🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)")

# Export launch function
__all__ = ['launch_system']


✅ Box 3 initialization complete!
🚀 Use `launch_system('mode')` to start interfaces (e.g., `launch_system('gradio')`)


In [ ]:
# --- Diagnostic Test ---
# Run this in a new cell to check what the action_agent tool returns
try:
    from __main__ import TOOL_REGISTRY
    action_agent_tool = TOOL_REGISTRY.get("action_agent")
    if action_agent_tool:
        print("Testing action_agent tool directly...")
        test_result = action_agent_tool("Say hello once.")
        print(f"Type of result: {type(test_result)}")
        print(f"Result: {test_result}")
        if isinstance(test_result, dict):
            print("Status:", test_result.get("status", "No status found"))
            print("Final Output Preview:", test_result.get("final_output", "No final output")[:100])
        else:
            print("ERROR: Result is not a dictionary!")
    else:
        print("action_agent tool not found in TOOL_REGISTRY")
except Exception as e:
    print(f"Error during test: {e}")
    import traceback
    traceback.print_exc()

action_agent tool not found in TOOL_REGISTRY


In [ ]:
# Run this in a cell after Box 2 finishes
print("Checking for Box 2 components in session...")
print("Is 'agent' in globals()?", 'agent' in globals())
print("Is 'TOOL_REGISTRY' in globals()?", 'TOOL_REGISTRY' in globals())
if 'TOOL_REGISTRY' in globals():
    print("Tools registered in session:")
    print(list(TOOL_REGISTRY.keys()))
    print("Is 'action_agent' in TOOL_REGISTRY?", 'action_agent' in TOOL_REGISTRY)
else:
    print("TOOL_REGISTRY not found in session globals.")

Checking for Box 2 components in session...
Is 'agent' in globals()? True
Is 'TOOL_REGISTRY' in globals()? True
Tools registered in session:
['write_file', 'read_file', 'list_files', 'run_agent_task', 'get_agent_memory', 'clear_agent_memory', 'install_package', 'execute_python']
Is 'action_agent' in TOOL_REGISTRY? False


In [ ]:
print("Checking for Box 2 components in session...")
print("Is 'agent' in globals()?", 'agent' in globals())
print("Is 'TOOL_REGISTRY' in globals()?", 'TOOL_REGISTRY' in globals())
if 'TOOL_REGISTRY' in globals():
    print("Tools registered in session:")
    print(list(TOOL_REGISTRY.keys()))
    required_tools = ['list_files', 'action_agent'] # Add others you expect
    for tool in required_tools:
        print(f"Is '{tool}' in TOOL_REGISTRY?", tool in TOOL_REGISTRY)
else:
    print("TOOL_REGISTRY not found in session globals.")

Checking for Box 2 components in session...
Is 'agent' in globals()? True
Is 'TOOL_REGISTRY' in globals()? True
Tools registered in session:
['action_agent']
Is 'list_files' in TOOL_REGISTRY? False
Is 'action_agent' in TOOL_REGISTRY? True


In [ ]:
print("Checking for Box 2 components in session...")
print("Is 'agent' in globals()?", 'agent' in globals())
print("Is 'TOOL_REGISTRY' in globals()?", 'TOOL_REGISTRY' in globals())
if 'TOOL_REGISTRY' in globals():
    print("Tools registered in session:")
    print(list(TOOL_REGISTRY.keys()))
    required_tools = ['list_files', 'action_agent'] # Add others you expect
    for tool in required_tools:
        print(f"Is '{tool}' in TOOL_REGISTRY?", tool in TOOL_REGISTRY)
else:
    print("TOOL_REGISTRY not found in session globals.")

Checking for Box 2 components in session...
Is 'agent' in globals()? True
Is 'TOOL_REGISTRY' in globals()? True
Tools registered in session:
['clear_memory', 'action_agent']
Is 'list_files' in TOOL_REGISTRY? False
Is 'action_agent' in TOOL_REGISTRY? True


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 2: Agent Core and Tools with Ollama Integration & Chat Endpoint - v7.0.x                        ║
# ║ Integrates Ollama-powered ManusAgent with FastAPI tool endpoints and Human-in-the-Loop Chat            ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration & Chat...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import psutil
import requests
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable
from pydantic import BaseModel

# --- Configuration and Setup ---
print("📥 Step 1: Loading configurations from Box 1...")
# Assuming a standard path or loading from environment/arguments
config_dir = Path("./UnifiedManusSystem/config") # Adjust path as needed
config_dir.mkdir(parents=True, exist_ok=True)

# Load Box 1 config (assumed to exist)
box1_config_file = config_dir / "box1_exports.json"
if box1_config_file.exists():
    with open(box1_config_file, "r") as f:
        box1_config = json.load(f)
    print("✅ Box 1 configuration loaded")
else:
    print("⚠️ Box 1 config not found, using defaults")
    # Set default paths if Box 1 config is missing
    box1_config = {
        "BASE_DIR": str(Path.cwd() / "UnifiedManusSystem"),
        "WORKSPACE_DIR": str(Path.cwd() / "UnifiedManusSystem" / "workspace"),
        "LOG_FILE": str(Path.cwd() / "UnifiedManusSystem" / "logs" / "manus.log")
    }

# Extract configuration
BASE_DIR = Path(box1_config["BASE_DIR"])
WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
LOG_FILE = Path(box1_config["LOG_FILE"])

# Ensure directories exist
BASE_DIR.mkdir(parents=True, exist_ok=True)
WORKSPACE_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
LOG_FILE.touch(exist_ok=True) # Create log file if it doesn't exist

# --- Ollama Integration ---
print("🦙 Step 2: Setting up Ollama...")
DEFAULT_MODEL = "llama3" # Default model if not specified

def setup_ollama(model_name: str):
    """Ensure Ollama is running and the model is available."""
    try:
        # Check if Ollama process is running
        ollama_running = any(proc.info['name'] == 'ollama' for proc in psutil.process_iter(['name']))
        if not ollama_running:
            print("🔄 Starting Ollama service...")
            subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            time.sleep(5) # Give it a moment to start

        # Check if the model is available
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True)
        if model_name not in result.stdout:
            print(f"📥 Pulling Ollama model: {model_name}...")
            subprocess.run(["ollama", "pull", model_name], check=True)
        else:
            print(f"✅ Ollama model '{model_name}' is available.")
    except Exception as e:
        print(f"❌ Error setting up Ollama: {e}")
        raise

setup_ollama(DEFAULT_MODEL)
MODEL_NAME = DEFAULT_MODEL
print(f"✅ Ollama is ready with model: {MODEL_NAME}")

# --- ManusAgent Definition ---
print("🧠 Step 3: Defining ManusAgent...")

class ManusAgent:
    def __init__(self, model_name: str = MODEL_NAME):
        self.model_name = model_name
        self.session_id = f"session_{int(time.time())}"
        self.memory: List[Dict[str, Any]] = []
        self.log_queue = queue.Queue()
        self._log_lock = threading.Lock()

    def log_activity(self, source: str, message: str, details: Optional[Dict[str, Any]] = None):
        """Log an activity to memory and optionally to a file."""
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "source": source,
            "message": message,
            "details": details or {}
        }
        with self._log_lock:
            self.memory.append(log_entry)
        # Also put in queue for streaming
        self.log_queue.put(json.dumps(log_entry))
        # Write to file
        try:
            with open(LOG_FILE, "a") as f:
                f.write(json.dumps(log_entry) + "\n")
        except Exception as e:
            print(f"⚠️ Error writing to log file: {e}")

    def think(self, prompt: str, role: str = "user") -> str:
        """Generate a response using the Ollama model."""
        self.log_activity("agent", f"Thinking as '{role}'", {"prompt": prompt})
        try:
            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "stream": False
            }
            response = requests.post("http://localhost:11434/api/generate", json=payload)
            if response.status_code == 200:
                data = response.json()
                reply = data.get("response", "").strip()
                self.log_activity("agent", f"Thought as '{role}' complete", {"reply": reply})
                return reply
            else:
                error_msg = f"Ollama API error ({response.status_code}): {response.text}"
                self.log_activity("agent", "Thinking failed", {"error": error_msg})
                return f"❌ {error_msg}"
        except Exception as e:
            error_msg = f"Exception during thinking: {str(e)}"
            self.log_activity("agent", "Thinking failed", {"error": error_msg})
            return f"❌ {error_msg}"

    def plan(self, task: str, context: str = "") -> str:
        """Plan the approach for a task."""
        prompt = f"You are an AI assistant. Plan how to accomplish the following task.\n\nTask: {task}\nContext: {context}\n\nProvide a clear, step-by-step plan."
        return self.think(prompt, role="planner")

    def code(self, plan: str) -> str:
        """Generate code based on a plan."""
        prompt = f"You are a Python coding expert. Write Python code to implement the following plan.\n\nPlan: {plan}\n\nReturn ONLY the Python code, nothing else. No markdown, no explanations."
        return self.think(prompt, role="coder")

    def review(self, code: str) -> str:
        """Review generated code."""
        prompt = f"You are a senior Python developer. Review the following Python code for correctness, efficiency, and best practices.\n\nCode:\n{code}\n\nProvide feedback."
        return self.think(prompt, role="reviewer")

    def execute_code(self, code: str) -> str:
        """Safely execute Python code (placeholder - consider sandboxing)."""
        self.log_activity("agent", "Executing code", {"code_snippet": code[:100] + "..." if len(code) > 100 else code})
        try:
            # In a real scenario, use a secure sandbox like Docker or Pyodide
            local_vars = {}
            global_vars = {"__builtins__": __builtins__}
            exec(code, global_vars, local_vars)
            result = local_vars.get('result', 'Code executed, no result variable returned.')
            self.log_activity("agent", "Code execution successful", {"result": str(result)})
            return str(result)
        except Exception as e:
            error_msg = f"Execution error: {str(e)}"
            self.log_activity("agent", "Code execution failed", {"error": error_msg})
            return f"❌ {error_msg}"

    def run_task(self, task: str, context: Optional[str] = None) -> Dict[str, Any]:
        """Run the full agent workflow: Plan -> Code -> Review -> Execute."""
        self.log_activity("agent", "Starting task execution", {"task": task, "context": context})
        try:
            plan = self.plan(task, context or "")
            code = self.code(plan)
            review = self.review(code)
            # For safety, we might not auto-execute. Let Box 3 decide.
            # result = self.execute_code(code)

            output = {
                "plan": plan,
                "code": code,
                "review": review,
                # "result": result,
                "status": "completed"
            }
            self.log_activity("agent", "Task execution completed", {"output_summary": {k: "...(truncated)" for k in output}})
            return output
        except Exception as e:
            error_msg = f"Task execution failed: {str(e)}"
            self.log_activity("agent", "Task execution failed", {"error": error_msg})
            return {"status": "failed", "error": error_msg}

    def chat(self, message: str, history: Optional[List[Dict[str, str]]] = None) -> str:
        """Simple chat interaction."""
        self.log_activity("agent", "Chat message received", {"message": message})
        history_text = ""
        if history:
            history_text = "\n".join([f"{h['role']}: {h['content']}" for h in history])
        prompt = f"Conversation History:\n{history_text}\n\nUser: {message}\nAssistant:"
        reply = self.think(prompt, role="chatbot")
        self.log_activity("agent", "Chat reply sent", {"reply": reply})
        return reply

# Initialize the agent
agent = ManusAgent()
print(f"✅ ManusAgent initialized (Session ID: {agent.session_id})")

# --- Tool Registry ---
print("🛠️ Step 4: Setting up Tool Registry...")
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str, func: Callable):
    """Register a tool function."""
    TOOL_REGISTRY[name] = func
    print(f"✅ Tool '{name}' registered.")

def clear_agent_memory():
    """Tool to clear the agent's memory."""
    agent.memory.clear()
    agent.log_activity("system", "Memory cleared via tool")
    return "Agent memory cleared."

# Register core tools
register_tool("clear_memory", clear_agent_memory)

# Register the agent itself as a tool
def action_agent_tool(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """Tool to run the agent's main task workflow."""
    task = input_data.get("task", "")
    context = input_data.get("context")
    if not task:
        return {"error": "Missing 'task' in input_data for action_agent tool."}
    return agent.run_task(task, context)

register_tool("action_agent", action_agent_tool)

# --- FastAPI Server ---
print("🌐 Step 5: Setting up FastAPI server...")
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse, JSONResponse
import uvicorn

app = FastAPI(
    title="Unified Manus MCP Server - Box 2 (Ollama)",
    description="Ollama-powered agent core with tool endpoints and chat",
    version="7.0.x"
)

# CORS - Adjust for production
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Pydantic models for requests
class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

class ToolCallRequest(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class ChatMessage(BaseModel):
    role: str # 'user' or 'assistant'
    content: str

class ChatRequest(BaseModel):
    message: str
    history: Optional[List[ChatMessage]] = None

# --- API Endpoints ---
@app.get("/")
async def root():
    """Root endpoint."""
    return {
        "message": "Unified Manus MCP System - Box 2 (Ollama-powered)",
        "version": "7.0.x",
        "model": MODEL_NAME,
        "agent_session": agent.session_id
    }

@app.get("/health")
async def health():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "box": 2,
        "version": "7.0.x",
        "model": MODEL_NAME,
        "agent_session": agent.session_id,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/agent/action")
async def run_agent(request: TaskRequest):
    """Run the agent's main task workflow."""
    return agent.run_task(request.task, request.context)

@app.post("/mcp/tools/call")
async def call_tool(request: ToolCallRequest):
    """Call a registered tool."""
    tool_name = request.tool_name
    input_data = request.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found."})

    try:
        result = TOOL_REGISTRY[tool_name](input_data)
        agent.log_activity("system", f"Tool '{tool_name}' executed successfully")
        # Pretty print if result is a dict
        if isinstance(result, dict):
            return JSONResponse(content={"result": result})
        return JSONResponse(content={"result": str(result)})
    except Exception as e:
        error_msg = f"Error executing tool '{tool_name}': {str(e)}"
        agent.log_activity("system", "Tool execution failed", {"tool": tool_name, "error": error_msg})
        return JSONResponse(status_code=500, content={"error": error_msg})

@app.post("/mcp/chat")
async def chat_endpoint(request: ChatRequest):
    """Chat with the agent."""
    message = request.message
    history = [{"role": h.role, "content": h.content} for h in request.history] if request.history else None
    reply = agent.chat(message, history)
    return JSONResponse(content={"reply": reply})

@app.get("/mcp/stream")
async def stream_logs():
    """Stream agent logs using Server-Sent Events (SSE)."""
    async def event_generator():
        while True:
            try:
                # Use a timeout to allow periodic checks
                log_entry_str = agent.log_queue.get(timeout=1.0)
                yield f"data: {log_entry_str}\n\n"
            except queue.Empty:
                # Check if client disconnected (basic check)
                # yield ": ping\n\n" # Keep-alive might be needed
                pass
            except asyncio.CancelledError:
                print("Stream cancelled by client.")
                break
            except Exception as e:
                print(f"Error in stream generator: {e}")
                break
    return StreamingResponse(event_generator(), media_type="text/event-stream")

# --- Save Configuration for Box 3 ---
print("💾 Step 6: Saving Box 2 configuration for Box 3...")
API_URL = "http://localhost:8000" # Default for local run

def save_box2_state():
    """Save Box 2 state for Box 3 to consume."""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "api_endpoints": [
            "/",
            "/health",
            "/mcp/agent/action",
            "/mcp/tools/call",
            "/mcp/chat",
            "/mcp/stream",
            "/openapi.json", # Standard for FastAPI
            "/docs",         # Standard for FastAPI
            "/redoc"         # Standard for FastAPI
        ],
        "ollama_model": MODEL_NAME,
        "api_url": API_URL,
        "timestamp": datetime.now().isoformat()
    }
    config_file = config_dir / "box2_exports.json"
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 2 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 2 state: {e}")

save_box2_state()

# --- Finalization ---
print("=" * 60)
print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
print("=" * 60)
print(f"🤖 Agent Session: {agent.session_id}")
print(f"🦙 Ollama Model: {MODEL_NAME}")
print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)} (including 'action_agent')")
print(f"🧠 Memory Entries: {len(agent.memory)}")
print(f"🌐 API Ready at: {API_URL}")
print("=" * 60)
print("🔄 Ready for Box 3: Server Launch and GUI")
agent.log_activity("system", "Box 2 setup completed successfully", {
    "tools_count": len(TOOL_REGISTRY),
    "agent_session": agent.session_id,
    "ollama_model": MODEL_NAME
})

# --- Main Execution Block (REVISED FOR JUPYTER) ---
# This part handles starting the server correctly in Jupyter or as a script.

# Simple function to start the server directly with the app object
# This is the recommended way when running in Jupyter/Notebooks
def start_box2_server():
    """Starts the Box 2 FastAPI server."""
    print("🚀 Starting Box 2 FastAPI server on http://0.0.0.0:8000...")
    # Pass the 'app' object directly to uvicorn.run
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Check if we are likely in a Jupyter environment
# A common and effective way is to check for 'get_ipython' in the global namespace
if 'get_ipython' in globals():
    print("🔍 Detected Jupyter environment.")
    # In Jupyter, we usually want to start the server asynchronously
    # to avoid blocking the cell execution.
    # We will start it in a background thread.

    import threading
    import time

    def run_server_in_thread():
        """Wrapper to run the server function in a thread."""
        start_box2_server()

    # Create and start the thread
    server_thread = threading.Thread(target=run_server_in_thread, daemon=True)
    server_thread.start()

    # Give the server a moment to potentially fail or start
    time.sleep(1)

    if server_thread.is_alive():
        print("✅ Box 2 server thread is running.")
        print("   You can now proceed to Box 3 or access the API at http://localhost:8000/docs")
    else:
        print("❌ Box 2 server thread stopped unexpectedly. Check for errors in the logs above.")

# --- Main Execution Block (REVISED FOR JUPYTER COMPATIBILITY - Fix Event Loop) ---
# This section ensures the FastAPI server starts correctly in Jupyter environments
# by properly managing the asyncio event loop within a background thread.

def start_box2_server():
    """Encapsulates the server startup logic, handling event loops correctly."""
    print("🚀 Starting Box 2 FastAPI server on http://0.0.0.0:8000...")
    try:
        # Key Change 1: Pass the 'app' object directly, not a module string.
        # Key Change 2: Explicitly manage the event loop for the thread.
        import asyncio
        import uvicorn

        # Get the current event loop policy
        # If there isn't one, set a new one (usually needed in new threads)
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError:
            # No loop in this thread, create a new one
            asyncio.set_event_loop(asyncio.new_event_loop())
            loop = asyncio.get_event_loop()

        # Configure Uvicorn to use the loop we just ensured exists
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)

        # Run the server using the specific loop
        loop.run_until_complete(server.serve())

    except Exception as e:
        print(f"❌ Error starting Box 2 server: {e}")
        import traceback
        traceback.print_exc() # Print the full traceback for debugging

# --- Determine Execution Context and Start Server ---
# Check if we are likely in a Jupyter-like environment
if 'get_ipython' in globals():
    print("🔍 Detected Jupyter/IPython environment.")
    # --- Jupyter Path ---
    # Start the server in a background thread to prevent blocking the cell.
    import threading
    import time
    import traceback # Import for error handling in thread

    # Define a wrapper function for the thread
    def run_server_in_thread():
        start_box2_server()

    # Create and start the daemon thread
    # Using a more descriptive name can help with debugging
    server_thread = threading.Thread(target=run_server_in_thread, daemon=True, name="Box2ServerThread")
    server_thread.start()

    # Brief pause to allow potential startup errors to surface
    # Increase slightly to give the loop setup a moment
    time.sleep(2)

    # Report status
    if server_thread.is_alive():
        print("✅ Box 2 server thread is running in the background.")
        print("   You can now access the API at: http://localhost:8000")
        print("   API Documentation (Swagger UI): http://localhost:8000/docs")
        print("   (Note: If the server stops later, check the cell output for errors.)")
    else:
        print("❌ Box 2 server thread stopped shortly after starting.")
        print("   Check the output above for potential errors during startup.")

elif __name__ == "__main__":
    # --- Standalone Script Path ---
    # This is for when the script is saved as a .py file and run via command line.
    # e.g., python my_box2_script.py
    # Uvicorn needs to be able to import the module by name.
    print("🚀 Starting Box 2 FastAPI server as a standalone script...")
    # IMPORTANT: Ensure the string matches your actual filename.
    # If the file is named `my_box2_script.py`, this should be "my_box2_script:app"
    uvicorn.run("box2_agent_core_and_tools_v7_0_x:app", host="0.0.0.0", port=8000, reload=False)

else:
    # --- Module Import Path ---
    # If this script is imported into another Python script, do nothing automatically.
    # The importing script can access `app`, `agent`, `TOOL_REGISTRY` via the `__all__` export.
    pass

# Export key components for potential integration (e.g., with Box 3 if run in the same session/kernel)
__all__ = ['app', 'agent', 'TOOL_REGISTRY']



🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration & Chat...
📥 Step 1: Loading configurations from Box 1...
⚠️ Box 1 config not found, using defaults
🦙 Step 2: Setting up Ollama...
[OLLAMA] [GIN] 2025/07/27 - 14:52:58 | 200 |      27.187µs |       127.0.0.1 | HEAD     "/"
[OLLAMA] [GIN] 2025/07/27 - 14:52:58 | 200 |     331.524µs |       127.0.0.1 | GET      "/api/tags"
✅ Ollama model 'llama3' is available.
✅ Ollama is ready with model: llama3
🧠 Step 3: Defining ManusAgent...
✅ ManusAgent initialized (Session ID: session_1753627978)
🛠️ Step 4: Setting up Tool Registry...
✅ Tool 'clear_memory' registered.
✅ Tool 'action_agent' registered.
🌐 Step 5: Setting up FastAPI server...
💾 Step 6: Saving Box 2 configuration for Box 3...
✅ Box 2 state saved to UnifiedManusSystem/config/box2_exports.json
🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!
🤖 Agent Session: session_1753627978
🦙 Ollama Model: llama3
🛠️ Tools Registered: 2 (including 'action_agent')
🧠 Memory Entries: 0
🌐 API Ready at:

Exception in thread Thread-15 (run_server_in_thread):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-30-2574689039.py", line 431, in run_server_in_thread
  File "/tmp/ipython-input-30-2574689039.py", line 416, in start_box2_server
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_e

❌ Box 2 server thread stopped unexpectedly. Check for errors in the logs above.
🔍 Detected Jupyter/IPython environment.
🚀 Starting Box 2 FastAPI server on http://0.0.0.0:8000...


INFO:     Started server process [1139]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


❌ Box 2 server thread stopped shortly after starting.
   Check the output above for potential errors during startup.


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🤖 BOX 2: Agent Core and Tools with Ollama Integration & Chat Endpoint - v7.0.x                        ║
# ║ Integrates Ollama-powered ManusAgent with FastAPI tool endpoints and Human-in-the-Loop Chat            ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

print("🔧 BOX 2: Initializing Agent Core and Tools with Ollama Integration & Chat...")

import os
import sys
import time
import json
import asyncio
import threading
import subprocess
import traceback
import queue
import psutil
import requests
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any, Callable
from pydantic import BaseModel

# --- Configuration and Setup ---
print("📥 Step 1: Loading configurations from Box 1...")
# Assuming a standard path or loading from environment/arguments
config_dir = Path("./UnifiedManusSystem/config") # Adjust path as needed
config_dir.mkdir(parents=True, exist_ok=True)

# Load Box 1 config (assumed to exist)
box1_config_file = config_dir / "box1_exports.json"
if box1_config_file.exists():
    with open(box1_config_file, "r") as f:
        box1_config = json.load(f)
    print("✅ Box 1 configuration loaded")
else:
    print("⚠️ Box 1 config not found, using defaults")
    # Set default paths if Box 1 config is missing
    box1_config = {
        "BASE_DIR": str(Path.cwd() / "UnifiedManusSystem"),
        "WORKSPACE_DIR": str(Path.cwd() / "UnifiedManusSystem" / "workspace"),
        "LOG_FILE": str(Path.cwd() / "UnifiedManusSystem" / "logs" / "manus.log")
    }

# Extract configuration
BASE_DIR = Path(box1_config["BASE_DIR"])
WORKSPACE_DIR = Path(box1_config["WORKSPACE_DIR"])
LOG_FILE = Path(box1_config["LOG_FILE"])

# Ensure directories exist
BASE_DIR.mkdir(parents=True, exist_ok=True)
WORKSPACE_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
LOG_FILE.touch(exist_ok=True) # Create log file if it doesn't exist

# --- Ollama Integration ---
print("🦙 Step 2: Setting up Ollama...")
DEFAULT_MODEL = "llama3" # Default model if not specified

def setup_ollama(model_name: str):
    """Ensure Ollama is running and the model is available."""
    try:
        # Check if Ollama process is running
        ollama_running = any(proc.info['name'] == 'ollama' for proc in psutil.process_iter(['name']))
        if not ollama_running:
            print("🔄 Starting Ollama service...")
            subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            time.sleep(5) # Give it a moment to start

        # Check if the model is available
        result = subprocess.run(["ollama", "list"], capture_output=True, text=True)
        if model_name not in result.stdout:
            print(f"📥 Pulling Ollama model: {model_name}...")
            subprocess.run(["ollama", "pull", model_name], check=True)
        else:
            print(f"✅ Ollama model '{model_name}' is available.")
    except Exception as e:
        print(f"❌ Error setting up Ollama: {e}")
        raise

setup_ollama(DEFAULT_MODEL)
MODEL_NAME = DEFAULT_MODEL
print(f"✅ Ollama is ready with model: {MODEL_NAME}")

# --- ManusAgent Definition ---
print("🧠 Step 3: Defining ManusAgent...")

class ManusAgent:
    def __init__(self, model_name: str = MODEL_NAME):
        self.model_name = model_name
        self.session_id = f"session_{int(time.time())}"
        self.memory: List[Dict[str, Any]] = []
        self.log_queue = queue.Queue()
        self._log_lock = threading.Lock()

    def log_activity(self, source: str, message: str, details: Optional[Dict[str, Any]] = None):
        """Log an activity to memory and optionally to a file."""
        timestamp = datetime.now().isoformat()
        log_entry = {
            "timestamp": timestamp,
            "source": source,
            "message": message,
            "details": details or {}
        }
        with self._log_lock:
            self.memory.append(log_entry)
        # Also put in queue for streaming
        self.log_queue.put(json.dumps(log_entry))
        # Write to file
        try:
            with open(LOG_FILE, "a") as f:
                f.write(json.dumps(log_entry) + "\n")
        except Exception as e:
            print(f"⚠️ Error writing to log file: {e}")

    def think(self, prompt: str, role: str = "user") -> str:
        """Generate a response using the Ollama model."""
        self.log_activity("agent", f"Thinking as '{role}'", {"prompt": prompt})
        try:
            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "stream": False
            }
            response = requests.post("http://localhost:11434/api/generate", json=payload)
            if response.status_code == 200:
                data = response.json()
                reply = data.get("response", "").strip()
                self.log_activity("agent", f"Thought as '{role}' complete", {"reply": reply})
                return reply
            else:
                error_msg = f"Ollama API error ({response.status_code}): {response.text}"
                self.log_activity("agent", "Thinking failed", {"error": error_msg})
                return f"❌ {error_msg}"
        except Exception as e:
            error_msg = f"Exception during thinking: {str(e)}"
            self.log_activity("agent", "Thinking failed", {"error": error_msg})
            return f"❌ {error_msg}"

    def plan(self, task: str, context: str = "") -> str:
        """Plan the approach for a task."""
        prompt = f"You are an AI assistant. Plan how to accomplish the following task.\n\nTask: {task}\nContext: {context}\n\nProvide a clear, step-by-step plan."
        return self.think(prompt, role="planner")

    def code(self, plan: str) -> str:
        """Generate code based on a plan."""
        prompt = f"You are a Python coding expert. Write Python code to implement the following plan.\n\nPlan: {plan}\n\nReturn ONLY the Python code, nothing else. No markdown, no explanations."
        return self.think(prompt, role="coder")

    def review(self, code: str) -> str:
        """Review generated code."""
        prompt = f"You are a senior Python developer. Review the following Python code for correctness, efficiency, and best practices.\n\nCode:\n{code}\n\nProvide feedback."
        return self.think(prompt, role="reviewer")

    def execute_code(self, code: str) -> str:
        """Safely execute Python code (placeholder - consider sandboxing)."""
        self.log_activity("agent", "Executing code", {"code_snippet": code[:100] + "..." if len(code) > 100 else code})
        try:
            # In a real scenario, use a secure sandbox like Docker or Pyodide
            local_vars = {}
            global_vars = {"__builtins__": __builtins__}
            exec(code, global_vars, local_vars)
            result = local_vars.get('result', 'Code executed, no result variable returned.')
            self.log_activity("agent", "Code execution successful", {"result": str(result)})
            return str(result)
        except Exception as e:
            error_msg = f"Execution error: {str(e)}"
            self.log_activity("agent", "Code execution failed", {"error": error_msg})
            return f"❌ {error_msg}"

    def run_task(self, task: str, context: Optional[str] = None) -> Dict[str, Any]:
        """Run the full agent workflow: Plan -> Code -> Review -> Execute."""
        self.log_activity("agent", "Starting task execution", {"task": task, "context": context})
        try:
            plan = self.plan(task, context or "")
            code = self.code(plan)
            review = self.review(code)
            # For safety, we might not auto-execute. Let Box 3 decide.
            # result = self.execute_code(code)

            output = {
                "plan": plan,
                "code": code,
                "review": review,
                # "result": result,
                "status": "completed"
            }
            self.log_activity("agent", "Task execution completed", {"output_summary": {k: "...(truncated)" for k in output}})
            return output
        except Exception as e:
            error_msg = f"Task execution failed: {str(e)}"
            self.log_activity("agent", "Task execution failed", {"error": error_msg})
            return {"status": "failed", "error": error_msg}

    def chat(self, message: str, history: Optional[List[Dict[str, str]]] = None) -> str:
        """Simple chat interaction."""
        self.log_activity("agent", "Chat message received", {"message": message})
        history_text = ""
        if history:
            history_text = "\n".join([f"{h['role']}: {h['content']}" for h in history])
        prompt = f"Conversation History:\n{history_text}\n\nUser: {message}\nAssistant:"
        reply = self.think(prompt, role="chatbot")
        self.log_activity("agent", "Chat reply sent", {"reply": reply})
        return reply

# Initialize the agent
agent = ManusAgent()
print(f"✅ ManusAgent initialized (Session ID: {agent.session_id})")

# --- File System Manager and Tools (Add after ManusAgent definition) ---
print("📂 Step 3.5: Defining FileSystemManager and file tools...")

import subprocess
import sys
import pkg_resources

class FileSystemManager:
    """Manages file operations within the workspace."""
    def __init__(self, base_path: Path):
        self.base_path = base_path.resolve()
        self.base_path.mkdir(parents=True, exist_ok=True)

    def _get_full_path(self, filename: str) -> Path:
        """Resolve filename relative to base_path and ensure it's within bounds."""
        full_path = (self.base_path / filename).resolve()
        # Security: Ensure the path is within the base_path
        try:
            full_path.relative_to(self.base_path)
        except ValueError:
            raise PermissionError(f"Access denied: {filename} is outside the workspace.")
        return full_path

    def create_file(self, filename: str, content: str = "") -> str:
        """Create a new file."""
        try:
            full_path = self._get_full_path(filename)
            full_path.parent.mkdir(parents=True, exist_ok=True) # Ensure parent dirs exist
            with open(full_path, 'w') as f:
                f.write(content)
            agent.log_activity("fs_manager", f"File created: {filename}")
            return f"✅ File '{filename}' created successfully."
        except Exception as e:
            error_msg = f"❌ Failed to create file '{filename}': {str(e)}"
            agent.log_activity("fs_manager", "File creation failed", {"filename": filename, "error": error_msg})
            return error_msg

    def read_file(self, filename: str) -> str:
        """Read the content of a file."""
        try:
            full_path = self._get_full_path(filename)
            with open(full_path, 'r') as f:
                content = f.read()
            agent.log_activity("fs_manager", f"File read: {filename}", {"content_preview": content[:100]})
            return content
        except FileNotFoundError:
            error_msg = f"❌ File '{filename}' not found."
            agent.log_activity("fs_manager", "File read failed", {"filename": filename, "error": error_msg})
            return error_msg
        except Exception as e:
            error_msg = f"❌ Failed to read file '{filename}': {str(e)}"
            agent.log_activity("fs_manager", "File read failed", {"filename": filename, "error": error_msg})
            return error_msg

    def append_to_file(self, filename: str, content: str) -> str:
        """Append content to a file."""
        try:
            full_path = self._get_full_path(filename)
            with open(full_path, 'a') as f:
                f.write(content)
            agent.log_activity("fs_manager", f"Content appended to: {filename}")
            return f"✅ Content appended to '{filename}' successfully."
        except Exception as e:
            error_msg = f"❌ Failed to append to file '{filename}': {str(e)}"
            agent.log_activity("fs_manager", "File append failed", {"filename": filename, "error": error_msg})
            return error_msg

    def delete_file(self, filename: str) -> str:
        """Delete a file."""
        try:
            full_path = self._get_full_path(filename)
            full_path.unlink() # Raises FileNotFoundError if file doesn't exist
            agent.log_activity("fs_manager", f"File deleted: {filename}")
            return f"✅ File '{filename}' deleted successfully."
        except FileNotFoundError:
            error_msg = f"❌ File '{filename}' not found."
            agent.log_activity("fs_manager", "File deletion failed", {"filename": filename, "error": error_msg})
            return error_msg
        except Exception as e:
            error_msg = f"❌ Failed to delete file '{filename}': {str(e)}"
            agent.log_activity("fs_manager", "File deletion failed", {"filename": filename, "error": error_msg})
            return error_msg

    def list_files(self, directory: str = ".") -> List[str]:
        """List files in a directory relative to base_path."""
        try:
            full_dir_path = self._get_full_path(directory)
            if not full_dir_path.is_dir():
                 return [f"❌ '{directory}' is not a directory."]
            files = [f.name for f in full_dir_path.iterdir() if f.is_file()]
            agent.log_activity("fs_manager", f"Listed files in: {directory}", {"count": len(files)})
            return sorted(files)
        except PermissionError as e:
            error_msg = f"❌ Permission error listing '{directory}': {str(e)}"
            agent.log_activity("fs_manager", "File listing failed", {"directory": directory, "error": error_msg})
            return [error_msg]
        except Exception as e:
            error_msg = f"❌ Failed to list files in '{directory}': {str(e)}"
            agent.log_activity("fs_manager", "File listing failed", {"directory": directory, "error": error_msg})
            return [error_msg]

# Initialize the FileSystemManager
fs_manager = FileSystemManager(WORKSPACE_DIR)
print(f"✅ FileSystemManager initialized for workspace: {WORKSPACE_DIR}")

# --- Python Execution and Package Management Tools ---
print("🐍 Step 3.6: Defining Python execution and package tools...")

def execute_python(code: str) -> str:
    """Execute Python code safely (placeholder - consider sandboxing)."""
    agent.log_activity("python_executor", "Executing Python code", {"code_snippet": code[:100]})
    try:
        # In a real scenario, use a secure sandbox like Docker or Pyodide
        # Capturing stdout/stderr is complex in the same process.
        # For now, we'll use exec in a limited scope and return repr of 'result' var if it exists.
        local_vars = {}
        global_vars = {"__builtins__": __builtins__}
        exec(code, global_vars, local_vars)
        result = local_vars.get('result', 'Code executed, no "result" variable captured.')
        agent.log_activity("python_executor", "Python code executed successfully", {"result_preview": str(result)[:100]})
        return str(result)
    except Exception as e:
        error_msg = f"❌ Python execution error: {str(e)}"
        agent.log_activity("python_executor", "Python execution failed", {"error": error_msg})
        return error_msg

def install_package(package_name: str) -> str:
    """Install a Python package using pip."""
    agent.log_activity("package_manager", f"Installing package: {package_name}")
    try:
        # Use subprocess to run pip install
        # Note: sys.executable ensures we use the same Python interpreter
        result = subprocess.run([sys.executable, "-m", "pip", "install", package_name],
                                capture_output=True, text=True, timeout=300) # 5 min timeout
        if result.returncode == 0:
            agent.log_activity("package_manager", f"Package installed: {package_name}")
            return f"✅ Package '{package_name}' installed successfully.\n{result.stdout}"
        else:
            error_msg = f"❌ Failed to install package '{package_name}'.\nStderr:\n{result.stderr}\nStdout:\n{result.stdout}"
            agent.log_activity("package_manager", "Package installation failed", {"package": package_name, "error": error_msg})
            return error_msg
    except subprocess.TimeoutExpired:
        error_msg = f"❌ Installation of '{package_name}' timed out."
        agent.log_activity("package_manager", "Package installation timed out", {"package": package_name, "error": error_msg})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Exception during installation of '{package_name}': {str(e)}"
        agent.log_activity("package_manager", "Package installation failed", {"package": package_name, "error": error_msg})
        return error_msg

def list_packages() -> List[str]:
    """List installed Python packages."""
    agent.log_activity("package_manager", "Listing installed packages")
    try:
        # Get installed packages using pkg_resources (part of setuptools)
        installed_packages = [str(dist) for dist in list(pkg_resources.working_set)]
        agent.log_activity("package_manager", "Packages listed", {"count": len(installed_packages)})
        return sorted(installed_packages)
    except Exception as e:
        error_msg = f"❌ Failed to list packages: {str(e)}"
        agent.log_activity("package_manager", "Package listing failed", {"error": error_msg})
        return [error_msg]

# --- Dynamic Agent and Tool Injection Tools (Add after other tool definitions) ---

print("🔧 Step 3.7: Defining Agent and Tool Injection tools...")

# Import necessary modules for code execution (if not already imported at the top)
# We'll use the existing `execute_python` concept but need more control for defining functions.
# exec is used here, which is powerful but requires trust in the input code.
import types
import inspect

def inject_agent(role: str, task: str, tools_list: Optional[List[str]] = None, context: Optional[str] = None) -> str:
    """
    Creates and runs a temporary agent with a specific role and tool access.

    Args:
        role (str): The role or system prompt for the temporary agent.
        task (str): The task for the temporary agent to perform.
        tools_list (List[str], optional): A list of tool names the agent can use.
                                          If None, it has access to all tools.
                                          Use an empty list [] for no tools.
        context (str, optional): Additional context for the agent.

    Returns:
        str: The final output or result from the temporary agent.
    """
    agent.log_activity("injector", "Injecting temporary agent", {"role": role, "task": task, "tools_count": len(tools_list) if tools_list is not None else "All"})

    try:
        # Determine tool access
        available_tools = TOOL_REGISTRY # Default to all
        if tools_list is not None:
            # Filter tools based on the provided list
            available_tools = {name: func for name, func in TOOL_REGISTRY.items() if name in tools_list}
            if len(available_tools) != len(tools_list):
                missing = set(tools_list) - set(available_tools.keys())
                if missing:
                    agent.log_activity("injector", "Warning: Some requested tools for injected agent not found", {"missing": list(missing)})

        # Create a temporary agent instance (reusing ManusAgent but with a custom role/context if needed)
        # For simplicity, we'll use the main agent's model but log the custom role.
        # A more advanced version could create a distinct ManusAgent instance.
        temp_agent_prompt_prefix = f"[TEMP AGENT ROLE: {role}] "

        # Simple way to influence the task with the role for the main agent
        modified_task = f"{temp_agent_prompt_prefix} Task: {task}"
        if context:
             modified_task += f"\nContext: {context}"

        agent.log_activity("injector", "Running task with temporary agent context", {"modified_task_preview": modified_task[:100]})

        # Run the task using the main agent's `run_task` method, which will use its thinking process
        # The role/context is injected via the task prompt itself.
        # Note: This doesn't create a *fully* isolated agent with its own memory/parameters,
        # but it guides the main agent's behavior for this specific task based on the role.
        # Creating a truly independent agent instance is more complex.
        temp_result_dict = agent.run_task(modified_task) # Use main agent's method

        final_output = temp_result_dict.get("final_output", temp_result_dict.get("result", "No final output from injected agent task."))

        agent.log_activity("injector", "Temporary agent task completed", {"role": role, "final_output_preview": final_output[:100]})
        return f"[RESULT FROM TEMP AGENT ({role})]: {final_output}"

    except Exception as e:
        error_msg = f"❌ Error injecting/running temporary agent: {str(e)}"
        agent.log_activity("injector", "Temporary agent injection failed", {"role": role, "error": error_msg})
        return error_msg


def inject_tool(name: str, description: str, code: str, parameters: Optional[Dict[str, str]] = None) -> str:
    """
    Dynamically creates and registers a new tool function.

    Args:
        name (str): The name for the new tool.
        description (str): A description of what the tool does (becomes the function's docstring).
        code (str): The Python code for the function body. It must define a function named `tool_function`.
                    The function should return a string result.
                    Example code string:
                    "def tool_function(param1: str, param2: int) -> str:
                        return f'Processed {param1} and {param2}'"
        parameters (Dict[str, str], optional): A dictionary describing parameters for documentation/tool introspection.
                                               Not used for execution, but for potential future API/tool listing details.
                                               Format: {"param_name": "description"}

    Returns:
        str: A message indicating success or failure.
    """
    agent.log_activity("injector", "Injecting temporary tool", {"tool_name": name, "description": description})

    try:
        if name in TOOL_REGISTRY:
            warning_msg = f"⚠️ Tool '{name}' already exists in registry. It will be overwritten."
            agent.log_activity("injector", "Warning: Overwriting existing tool", {"tool_name": name})
            # Depending on requirements, you might want to prevent this or handle versioning.

        # Create a safe global namespace for executing the tool code
        # Provide access to common utilities if needed within the injected tool
        safe_globals = {
            "__builtins__": __builtins__,
            "os": os, # Be cautious granting OS access
            "sys": sys,
            "json": json,
            "requests": requests, # Allow web requests if needed
            # Add other safe modules as needed, or restrict further
            # Consider using a more restricted sandbox for production
        }

        # Execute the provided code string in the safe namespace
        # The code is expected to define a function named 'tool_function'
        local_vars = {}
        exec(code, safe_globals, local_vars)

        # Retrieve the defined function
        injected_func = local_vars.get('tool_function')

        if not injected_func or not isinstance(injected_func, types.FunctionType):
            error_msg = "❌ The provided code must define a function named 'tool_function'."
            agent.log_activity("injector", "Tool injection failed - no 'tool_function' defined", {"tool_name": name})
            return error_msg

        # Set the description as the function's docstring
        injected_func.__doc__ = description

        # Register the new function in the global TOOL_REGISTRY
        TOOL_REGISTRY[name] = injected_func
        agent.log_activity("injector", "Temporary tool injected and registered successfully", {"tool_name": name})
        return f"✅ Tool '{name}' injected and registered successfully."

    except SyntaxError as se:
        error_msg = f"❌ Syntax Error in injected tool code '{name}': {se}"
        agent.log_activity("injector", "Tool injection failed - Syntax Error", {"tool_name": name, "error": error_msg})
        return error_msg
    except Exception as e:
        error_msg = f"❌ Error injecting tool '{name}': {str(e)}"
        agent.log_activity("injector", "Tool injection failed", {"tool_name": name, "error": error_msg})
        return error_msg

print("✅ Agent and Tool Injection tools defined.")

# --- Tool Registry ---
print("🛠️ Step 4: Setting up Tool Registry...")
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(name: str, func: Callable):
    """Register a tool function."""
    TOOL_REGISTRY[name] = func
    print(f"✅ Tool '{name}' registered.")

def clear_agent_memory():
    """Tool to clear the agent's memory."""
    agent.memory.clear()
    agent.log_activity("system", "Memory cleared via tool")
    return "Agent memory cleared."

# Register core tools
register_tool("clear_memory", clear_agent_memory)

# Register file system tools (using fs_manager methods)
register_tool("create_file", fs_manager.create_file)
register_tool("read_file", fs_manager.read_file)
register_tool("append_to_file", fs_manager.append_to_file)
register_tool("delete_file", fs_manager.delete_file)
register_tool("list_files", fs_manager.list_files)

# Register Python execution and package tools
register_tool("execute_python", execute_python)
register_tool("install_package", install_package)
register_tool("list_packages", list_packages)

# Register the agent itself as a tool
def acti# --- Additional Tools for Log Retrieval and Filtering (Add after other tool definitions) ---

print("📜 Step 3.8: Defining Log Retrieval and Filtering tools...")

# Ensure 'collections' is imported at the top if not already
# from collections import deque # Not strictly needed for slicing list, but good for efficient fixed-size queues

def get_recent_logs(count: int = 10) -> List[Dict[str, Any]]:
    """
    Retrieves the most recent log entries from the agent's memory.

    Args:
        count (int): The number of recent log entries to retrieve. Defaults to 10.

    Returns:
        List[Dict[str, Any]]: A list of the most recent log entry dictionaries.
                              Returns an error message list if count is invalid.
    """
    agent.log_activity("log_tool", f"Fetching {count} recent logs")
    try:
        count = int(count)
        if count <= 0:
            return [{"error": "Count must be a positive integer."}]

        # agent.memory is a list, slice the last 'count' items
        recent_logs = agent.memory[-count:]
        agent.log_activity("log_tool", f"Retrieved {len(recent_logs)} recent logs")
        return recent_logs
    except (ValueError, TypeError):
        return [{"error": "Count must be a valid integer."}]
    except Exception as e:
        error_msg = f"❌ Error fetching recent logs: {str(e)}"
        agent.log_activity("log_tool", "Fetching recent logs failed", {"error": error_msg})
        return [{"error": error_msg}]

def get_filtered_logs(filter_text: str) -> List[Dict[str, Any]]:
    """
    Retrieves log entries that contain a specific text in their message or details.

    Args:
        filter_text (str): The text to search for within log entries.

    Returns:
        List[Dict[str, Any]]: A list of log entry dictionaries that match the filter.
    """
    agent.log_activity("log_tool", f"Fetching logs filtered by: '{filter_text}'")
    try:
        if not isinstance(filter_text, str):
             return [{"error": "Filter text must be a string."}]

        filtered_logs = []
        for entry in agent.memory:
            # Check message
            if filter_text.lower() in entry.get("message", "").lower():
                filtered_logs.append(entry)
                continue # Avoid adding duplicates if both message and details match
            # Check details (convert dict to string for searching)
            details_str = json.dumps(entry.get("details", {}), default=str)
            if filter_text.lower() in details_str.lower():
                filtered_logs.append(entry)

        agent.log_activity("log_tool", f"Retrieved {len(filtered_logs)} logs matching filter")
        return filtered_logs
    except Exception as e:
        error_msg = f"❌ Error filtering logs: {str(e)}"
        agent.log_activity("log_tool", "Filtering logs failed", {"error": error_msg, "filter": filter_text})
        return [{"error": error_msg}]

def get_recent_filtered_logs(count: int = 10, filter_text: str = "") -> List[Dict[str, Any]]:
    """
    Retrieves the most recent log entries, optionally filtered by text.

    Args:
        count (int): The number of recent log entries to consider. Defaults to 10.
        filter_text (str): Optional text to filter the recent logs by.

    Returns:
        List[Dict[str, Any]]: A list of the most recent (and optionally filtered) log entries.
    """
    # Re-use existing logic
    recent_logs = get_recent_logs(count)
    # Check if get_recent_logs returned an error
    if isinstance(recent_logs, list) and len(recent_logs) > 0 and isinstance(recent_logs[0], dict) and 'error' in recent_logs[0]:
        return recent_logs # Propagate error

    agent.log_activity("log_tool", f"Filtering last {count} logs by: '{filter_text}'")
    try:
        if not filter_text:
            return recent_logs # No filter, return recent logs

        filtered_logs = []
        for entry in recent_logs:
             if filter_text.lower() in entry.get("message", "").lower():
                filtered_logs.append(entry)
                continue
             details_str = json.dumps(entry.get("details", {}), default=str)
             if filter_text.lower() in details_str.lower():
                filtered_logs.append(entry)

        agent.log_activity("log_tool", f"Retrieved {len(filtered_logs)} recent logs matching filter")
        return filtered_logs
    except Exception as e:
        error_msg = f"❌ Error filtering recent logs: {str(e)}"
        agent.log_activity("log_tool", "Filtering recent logs failed", {"error": error_msg, "filter": filter_text, "count": count})
        return [{"error": error_msg}]

def read_log_file(lines: int = 20) -> str:
    """
    Reads the last N lines from the agent's log file.

    Args:
        lines (int): The number of lines from the end of the file to read. Defaults to 20.

    Returns:
        str: The last N lines of the log file as a single string, or an error message.
    """
    agent.log_activity("log_tool", f"Reading last {lines} lines from log file")
    try:
        lines = int(lines)
        if lines <= 0:
            return "❌ Error: Number of lines must be positive."

        # Reading last N lines efficiently
        # Adapted approach for simplicity, might not be optimal for huge files
        with open(LOG_FILE, 'r') as f:
            all_lines = f.readlines()

        last_lines = all_lines[-lines:] if len(all_lines) >= lines else all_lines
        result = ''.join(last_lines)
        agent.log_activity("log_tool", f"Read {len(last_lines)} lines from log file")
        return result
    except (ValueError, TypeError):
        return "❌ Error: Number of lines must be a valid integer."
    except FileNotFoundError:
        return f"❌ Error: Log file '{LOG_FILE}' not found."
    except Exception as e:
        error_msg = f"❌ Error reading log file: {str(e)}"
        agent.log_activity("log_tool", "Reading log file failed", {"error": error_msg})
        return error_msg

print("✅ Log Retrieval and Filtering tools defined.")


on_agent_tool(input_data: Dict[str, Any]) -> Dict[str, Any]:
    """Tool to run the agent's main task workflow."""
    task = input_data.get("task", "")
    context = input_data.get("context")
    if not task:
        return {"error": "Missing 'task' in input_data for action_agent tool."}
    return agent.run_task(task, context)

register_tool("action_agent", action_agent_tool)

# --- Register the new injection tools ---
register_tool("inject_agent", inject_agent)
register_tool("inject_tool", inject_tool)

# --- FastAPI Server ---
print("🌐 Step 5: Setting up FastAPI server...")
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse, JSONResponse
import uvicorn

app = FastAPI(
    title="Unified Manus MCP Server - Box 2 (Ollama)",
    description="Ollama-powered agent core with tool endpoints and chat",
    version="7.0.x"
)

# CORS - Adjust for production
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Pydantic models for requests
class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

class ToolCallRequest(BaseModel):
    tool_name: str
    tool_input: Optional[Dict[str, Any]] = None

class ChatMessage(BaseModel):
    role: str # 'user' or 'assistant'
    content: str

class ChatRequest(BaseModel):
    message: str
    history: Optional[List[ChatMessage]] = None

# --- API Endpoints ---
@app.get("/")
async def root():
    """Root endpoint."""
    return {
        "message": "Unified Manus MCP System - Box 2 (Ollama-powered)",
        "version": "7.0.x",
        "model": MODEL_NAME,
        "agent_session": agent.session_id
    }

@app.get("/health")
async def health():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "box": 2,
        "version": "7.0.x",
        "model": MODEL_NAME,
        "agent_session": agent.session_id,
        "timestamp": datetime.now().isoformat()
    }

@app.post("/mcp/agent/action")
async def run_agent(request: TaskRequest):
    """Run the agent's main task workflow."""
    return agent.run_task(request.task, request.context)

@app.post("/mcp/tools/call")
async def call_tool(request: ToolCallRequest):
    """Call a registered tool."""
    tool_name = request.tool_name
    input_data = request.tool_input or {}

    if tool_name not in TOOL_REGISTRY:
        return JSONResponse(status_code=404, content={"error": f"Tool '{tool_name}' not found."})

    try:
        # Ensure input_data is a dictionary for **kwargs
        if not isinstance(input_data, dict):
             return JSONResponse(status_code=400, content={"error": "tool_input must be a JSON object (dictionary)."})

        result = TOOL_REGISTRY[tool_name](**input_data)
        agent.log_activity("system", f"Tool '{tool_name}' executed successfully")
        # Pretty print if result is a dict
        if isinstance(result, dict):
            return JSONResponse(content={"result": result})
        return JSONResponse(content={"result": str(result)})
    except Exception as e:
        error_msg = f"Error executing tool '{tool_name}': {str(e)}"
        agent.log_activity("system", "Tool execution failed", {"tool": tool_name, "error": error_msg})
        return JSONResponse(status_code=500, content={"error": error_msg})

@app.post("/mcp/chat")
async def chat_endpoint(request: ChatRequest):
    """Chat with the agent."""
    message = request.message
    history = [{"role": h.role, "content": h.content} for h in request.history] if request.history else None
    reply = agent.chat(message, history)
    return JSONResponse(content={"reply": reply})

@app.get("/mcp/stream")
async def stream_logs():
    """Stream agent logs using Server-Sent Events (SSE)."""
    async def event_generator():
        while True:
            try:
                # Use a timeout to allow periodic checks
                log_entry_str = agent.log_queue.get(timeout=1.0)
                yield f"data: {log_entry_str}\n\n"
            except queue.Empty:
                # Check if client disconnected (basic check)
                # yield ": ping\n\n" # Keep-alive might be needed
                pass
            except asyncio.CancelledError:
                print("Stream cancelled by client.")
                break
            except Exception as e:
                print(f"Error in stream generator: {e}")
                break
    return StreamingResponse(event_generator(), media_type="text/event-stream")

# --- Save Configuration for Box 3 ---
print("💾 Step 6: Saving Box 2 configuration for Box 3...")
API_URL = "http://localhost:8000" # Default for local run

def save_box2_state():
    """Save Box 2 state for Box 3 to consume."""
    state = {
        "tools_registered": list(TOOL_REGISTRY.keys()),
        "agent_session": agent.session_id,
        "api_endpoints": [
            "/",
            "/health",
            "/mcp/agent/action",
            "/mcp/tools/call",
            "/mcp/chat",
            "/mcp/stream",
            "/openapi.json", # Standard for FastAPI
            "/docs",         # Standard for FastAPI
            "/redoc"         # Standard for FastAPI
        ],
        "ollama_model": MODEL_NAME,
        "api_url": API_URL,
        "timestamp": datetime.now().isoformat()
    }
    config_file = config_dir / "box2_exports.json"
    try:
        with open(config_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"✅ Box 2 state saved to {config_file}")
    except Exception as e:
        print(f"❌ Failed to save Box 2 state: {e}")

save_box2_state()

# --- Finalization ---
print("=" * 60)
print("🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
print("=" * 60)
print(f"🤖 Agent Session: {agent.session_id}")
print(f"🦙 Ollama Model: {MODEL_NAME}")
print(f"📂 Workspace Directory: {WORKSPACE_DIR}")
print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)}")
tool_names = list(TOOL_REGISTRY.keys())
# Print tools in a more readable way
for i in range(0, len(tool_names), 5): # 5 tools per line
    print(f"   - {', '.join(tool_names[i:i+5])}")
print(f"🧠 Memory Entries: {len(agent.memory)}")
print(f"🌐 API Ready at: {API_URL}")
print("=" * 60)
print("🔄 Ready for Box 3: Server Launch and GUI")
agent.log_activity("system", "Box 2 setup completed successfully", {
    "tools_count": len(TOOL_REGISTRY),
    "agent_session": agent.session_id,
    "ollama_model": MODEL_NAME
})

# --- Main Execution Block (REVISED FOR JUPYTER COMPATIBILITY - Fix Event Loop) ---
# This section ensures the FastAPI server starts correctly in Jupyter environments
# by properly managing the asyncio event loop within a background thread.

def start_box2_server():
    """Encapsulates the server startup logic, handling event loops correctly."""
    print("🚀 Starting Box 2 FastAPI server on http://0.0.0.0:8000...")
    try:
        # Key Change 1: Pass the 'app' object directly, not a module string.
        # Key Change 2: Explicitly manage the event loop for the thread.
        import asyncio
        import uvicorn

        # Get the current event loop policy
        # If there isn't one, set a new one (usually needed in new threads)
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError:
            # No loop in this thread, create a new one
            asyncio.set_event_loop(asyncio.new_event_loop())
            loop = asyncio.get_event_loop()

        # Configure Uvicorn to use the loop we just ensured exists
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)

        # Run the server using the specific loop
        loop.run_until_complete(server.serve())

    except Exception as e:
        print(f"❌ Error starting Box 2 server: {e}")
        import traceback
        traceback.print_exc() # Print the full traceback for debugging

# --- Determine Execution Context and Start Server ---
# Check if we are likely in a Jupyter-like environment
if 'get_ipython' in globals():
    print("🔍 Detected Jupyter/IPython environment.")
    # --- Jupyter Path ---
    # Start the server in a background thread to prevent blocking the cell.
    import threading
    import time
    import traceback # Import for error handling in thread

    # Define a wrapper function for the thread
    def run_server_in_thread():
        start_box2_server()

    # Create and start the daemon thread
    # Using a more descriptive name can help with debugging
    server_thread = threading.Thread(target=run_server_in_thread, daemon=True, name="Box2ServerThread")
    server_thread.start()

    # Brief pause to allow potential startup errors to surface
    # Increase slightly to give the loop setup a moment
    time.sleep(2)

    # Report status
    if server_thread.is_alive():
        print("✅ Box 2 server thread is running in the background.")
        print("   You can now access the API at: http://localhost:8000")
        print("   API Documentation (Swagger UI): http://localhost:8000/docs")
        print("   (Note: If the server stops later, check the cell output for errors.)")
    else:
        print("❌ Box 2 server thread stopped shortly after starting.")
        print("   Check the output above for potential errors during startup.")

elif __name__ == "__main__":
    # --- Standalone Script Path ---
    # This is for when the script is saved as a .py file and run via command line.
    # e.g., python my_box2_script.py
    # Uvicorn needs to be able to import the module by name.
    print("🚀 Starting Box 2 FastAPI server as a standalone script...")
    # IMPORTANT: Ensure the string matches your actual filename.
    # If the file is named `my_box2_script.py`, this should be "my_box2_script:app"
    uvicorn.run("box2_agent_core_and_tools_v7_0_x:app", host="0.0.0.0", port=8000, reload=False)

else:
    # --- Module Import Path ---
    # If this script is imported into another Python script, do nothing automatically.
    # The importing script can access `app`, `agent`, `TOOL_REGISTRY` via the `__all__` export.
    pass

# Export key components for potential integration (e.g., with Box 3 if run in the same session/kernel)
__all__ = ['app', 'agent', 'TOOL_REGISTRY']


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/importer.py", line 19, in import_from_string
    module = importlib.import_module(module_str)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1140, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'box2_agent_core_and_tools_v7_0_x'

During handling of the above exception, another exception occu

SyntaxError: expected '(' (ipython-input-33-244305455.py, line 543)